<a href="https://colab.research.google.com/github/shahzad-r1zv1/LocalLLM_experiments/blob/main/7s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q pandas tqdm matplotlib reportlab python-pptx requests


In [10]:
# ============================== ONE-CELL 7S + OLLAMA (COLAB, WRC EDITION) ==============================
# Installs Ollama, pulls up to 3 models, runs selected 7S prompts across models (WRC wording),
# consolidates results, and saves JSON/Markdown outputs in /content.

import os, subprocess, shlex, time, requests, json, re, textwrap, datetime
from typing import Dict, List

# ---------------------------
# 0) BASIC CONFIG (EDIT THESE)
# ---------------------------
# Organization inputs used to fill prompt placeholders
ORG = {
    "name": "QA Automation Org",
    "industry": "Software / DevTools",
    "size_employees": 180,
    "annual_revenue": "N/A (cost center)",
    "locations": "Toronto; Remote",
    "current_context": "Transitioning from phased QA to DevSecOps; aiming for predictive quality and CI/CD-first culture.",
    "key_challenges": "Inconsistent test coverage; siloed defect data; unclear decision rights; skills gap in pipeline engineering.",
    "recent_changes": "Trunk-based dev in 2 squads; feature flags/canary pilots; Playwright + contract tests on 2 critical APIs.",
    # Optional fields for specific prompts (leave as "" if unknown)
    "strategy_statement": "Ship faster with confidence via automated quality gates and proactive risk analytics.",
    "strategic_goals": "12 months: 70% automated coverage; 24m: 95% critical path; MTTR<1hr; DORA elite.",
    "kpis": "DORA metrics, escaped defect rate, mean time to detect, e2e reliability SLOs.",
    "market_position": "Internal enablement platform within a large enterprise.",
    "competitive_advantages": "Domain expertise; test data virtualization; unified telemetry.",
    "target_segments": "Product teams building microservices/APIs.",
    "value_proposition": "Fewer incidents, faster releases, higher trust in automation.",
    "org_chart": "VP Eng → Dir QA/Platform → QA Enablement, SDET, Observability pods; squads map to product lines.",
    "layers": "VP → Director → Managers → ICs (4 layers)",
    "span_of_control": "Managers: 6-8 ICs avg",
    "decision_rights": "Product owns scope; QA owns quality standards; Dev owns implementation; shared release gates.",
    "xfunc": "Release Council; Reliability Guild; Architecture Forum.",
    "geo": "Toronto hub + distributed remote",
    "restructuring": "Evolving towards platform team model; decoupling test infra from app squads."
}

# Which prompts to run (choose from keys below)
RUN_PROMPTS = [
    "PROMPT_1_FULL_7S",
    "PROMPT_2_STRATEGY",
    "PROMPT_3_STRUCTURE",
    "PROMPT_4_SYSTEMS",
    "PROMPT_5_SHARED_VALUES",
    "PROMPT_6_SKILLS",
    "PROMPT_7_STYLE",
    "PROMPT_8_STAFF",
    "PROMPT_9_ALIGNMENT",
    "PROMPT_10_CHANGE",
    "PROMPT_11_DIGITAL",
    "PROMPT_12_BENCH",
    "PROMPT_13_GAPS",
    "PROMPT_14_INTEGRATION",
    "PROMPT_15_EXEC_SUMMARY",
    "MEGA_PROMPT"
]

In [11]:
# Models to try pulling (we’ll use what actually succeeds), "qwen3:8b"
# REQUESTED_MODELS = ["qwen3", "mistral", "llama3"]
#REQUESTED_MODELS = ["qwen3", "phi3", "llama3"]
# Common alias fixes (Ollama doesn't have "gemma3")
# MODEL_ALIASES = {
#     "gemma3": "gemma2:9b-instruct",
#     "gemma2-9b": "gemma2:9b-instruct",
#     "gemma2-2b": "gemma2:2b-instruct",
#     "gemma7b": "gemma:7b-instruct",
#     "llama3.1": "llama3.1",
#      # pass-through example
# }

# # --- Pull models asynchronously ---
# models = ["mistral", "llama3", "gemma2:2b-instruct"]  # use gemma2:2b or 9b instead of gemma3
# models = ["qwen3", "phi3", "llama3"]

MODEL_ALIASES = {
    "gemma3": "gemma2:9b-instruct",
    "gemma2": "gemma2:2b-instruct",
    "llama3": "llama3",
    "llama3.1": "llama3.1",
    "mistral": "mistral",
    "phi4":"phi4",
    "qwen3" : "qwen3:14b",
    "gemma7b": "gemma:7b-instruct",
}
REQUESTED_MODELS = ["mistral", "llama3", "phi4"]
REQUESTED_MODELS = [MODEL_ALIASES.get(m.lower(), m) for m in REQUESTED_MODELS]

# Output paths
OUT_DIR = "/content"
BUNDLE_JSON = os.path.join(OUT_DIR, "7S_bundle.json")
REPORT_MD   = os.path.join(OUT_DIR, "7S_report.md")

In [12]:
# ===============================================================
# 🚀 OLLAMA INSTALL / START / SMART MODEL PULLER  (with speed + restart)
# ===============================================================
import subprocess, shlex, requests, time, os, sys, re, math

def run(cmd, check=True, quiet=False):
    if isinstance(cmd, str):
        cmd = shlex.split(cmd)
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    if not quiet: print(p.stdout.strip())
    if check and p.returncode != 0:
        raise RuntimeError(f"CMD FAIL: {' '.join(cmd)}\n----\n{p.stdout}")
    return p.stdout.strip()

def ollama_installed():
    try:
        out = run("ollama --version", check=False, quiet=True)
        return "ollama" in out.lower()
    except Exception: return False

def ollama_running():
    try:
        r = requests.get("http://127.0.0.1:11434/api/tags", timeout=3)
        return r.status_code == 200
    except Exception: return False

# ------------------- INSTALL -------------------
if ollama_installed():
    print("✅ Ollama already installed.")
else:
    print("📦 Installing Ollama (with progress)...")
    proc = subprocess.Popen(
        "bash -lc 'curl -fSL https://ollama.com/install.sh | sh'",
        shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT
    )
    for line in iter(proc.stdout.readline, ''):
        sys.stdout.write(line); sys.stdout.flush()
    proc.wait()
    print("✅ Installation complete." if proc.returncode == 0 else "⚠️ Installation may have issues!")

# ------------------- START SERVER -------------------
if ollama_running():
    print("✅ Ollama already running.")
else:
    print("🚀 Starting ollama serve…")
    run("bash -lc 'pkill -f \"ollama serve\" || true'", check=False, quiet=True)
    logf = open("/tmp/ollama.log", "w")
    proc = subprocess.Popen(["ollama","serve"], stdout=logf, stderr=subprocess.STDOUT, text=True)
    for _ in range(60):
        if ollama_running(): break
        print(".", end="", flush=True); time.sleep(2)
    print("\n✅ Ollama API ready at http://127.0.0.1:11434")

# ------------------- SMART MODEL PULLER -------------------
def model_exists(m):
    try:
        r = requests.get("http://127.0.0.1:11434/api/tags", timeout=5)
        if r.status_code==200:
            names=[x["name"] for x in r.json().get("models",[])]
            return any(m.startswith(x) or x.startswith(m) for x in names)
    except: return False
    return False

def pull_model(m, max_retries=2, stall_threshold=0.10, stall_time=300):
    """
    Pull model with live progress, MB/s estimate, smart stall recovery,
    and periodic heartbeat for Colab environments.
    """
    MODEL_SIZES = {
        "mistral": 4200, "llama3": 8000, "gemma2:9b-instruct": 9000,
        "phi3": 3800, "gemma2:2b": 2200
    }
    total_mb = MODEL_SIZES.get(m.lower(), 4000)
    attempt = 0

    while attempt <= max_retries:
        attempt += 1
        if model_exists(m):
            print(f"✅ {m} already available.")
            return True

        print(f"\n⬇️ Pulling {m} (attempt {attempt}/{max_retries+1})...")
        p = subprocess.Popen(["ollama", "pull", m],
                             stdout=subprocess.PIPE,
                             stderr=subprocess.STDOUT,
                             text=True, bufsize=1)

        prog = re.compile(r"(\d+)%")
        last_pct, last_time = 0, time.time()
        speed_window = []
        last_update = time.time()
        last_heartbeat = time.time()
        stalled = False

        try:
            for line in iter(p.stdout.readline, ""):
                line = line.strip()
                match = prog.search(line)
                now = time.time()

                if match:
                    pct = int(match.group(1))
                    dt = max(now - last_time, 0.1)
                    dp = pct - last_pct

                    if dp > 0:
                        inst_speed = dp / dt
                        speed_window.append(inst_speed)
                        if len(speed_window) > 5:
                            speed_window.pop(0)
                        avg_speed = sum(speed_window) / len(speed_window)
                        last_time, last_pct = now, pct
                    else:
                        avg_speed = sum(speed_window) / len(speed_window) if speed_window else 0

                    mb_speed = (avg_speed / 100.0) * total_mb / max(dt, 1)
                    eta = min(((100 - pct) / max(avg_speed, 0.01)), 9999)

                    bar = f"[{'='*(pct//5):<20}]"
                    sys.stdout.write(
                        f"\r📦 {m:<18} {bar} {pct:3d}% | {avg_speed:4.2f}%/s | {mb_speed:5.2f} MB/s | ETA {eta:4.0f}s"
                    )
                    sys.stdout.flush()
                    last_update = now

                # Heartbeat if quiet for a while
                elif now - last_heartbeat > 90:
                    sys.stdout.write(" 💓")
                    sys.stdout.flush()
                    last_heartbeat = now

                # Stall detection
                if now - last_update > stall_time:
                    print(f"\n⚠️ {m} stalled (no update for {stall_time}s). Restarting...")
                    p.kill()
                    stalled = True
                    break

            p.wait()
            print()  # newline after progress bar

        except Exception as e:
            print(f"\n⚠️ Error while pulling {m}: {e}")
            p.kill()
            stalled = True

        if not stalled and p.returncode == 0:
            print(f"✅ {m} pull complete.")
            return True
        elif attempt <= max_retries:
            print(f"🔁 Retrying {m} in 10 s...")
            time.sleep(10)
        else:
            print(f"❌ {m} failed (exit {p.returncode}).")
            return False



# --- 3️⃣ Pull Models FAST using terminal (native streaming) ---
REQUESTED_MODELS = ["gpt-oss", "mistral", "llama3"]


print("\n📥 Pulling models via native terminal...\n")
for model in REQUESTED_MODELS:
    print(f"\n⬇️ Pulling {model} ...\n")
    # This runs at full native speed with progress bars and MB/s
    os.system(f"ollama pull {shlex.quote(model)}")

# --- 4️⃣ List downloaded models ---
print("\n📋 Available models:\n")
os.system("ollama list")

# ------------------- EXECUTE PULL -------------------
REQUESTED_MODELS = ["mistral", "llama3", "gemma2"]
REQUESTED_MODELS = ["gpt-oss", "mistral", "llama3"]
print("\n📥 Pulling models via running daemon...\n")

for m in REQUESTED_MODELS:
    pull_model(m)

print("\n📋 Models available now:")
!ollama list


✅ Ollama already installed.
✅ Ollama already running.

📥 Pulling models via native terminal...


⬇️ Pulling gpt-oss ...


⬇️ Pulling mistral ...


⬇️ Pulling llama3 ...


📋 Available models:


📥 Pulling models via running daemon...

✅ gpt-oss already available.
✅ mistral already available.
✅ llama3 already available.

📋 Models available now:
NAME              ID              SIZE      MODIFIED               
mistral:latest    6577803aa9a0    4.4 GB    Less than a second ago    
gpt-oss:latest    17052f91a42e    13 GB     Less than a second ago    
llama3:latest     365c0bd3c000    4.7 GB    Less than a second ago    


In [13]:

# Base endpoints
BASE = "http://127.0.0.1:11434"
TAGS = f"{BASE}/api/tags"
CHAT = f"{BASE}/api/chat"
GEN  = f"{BASE}/api/generate"
print(f"🌐 Connected to Ollama at {BASE}")
PRIMARY_MODEL = REQUESTED_MODELS[0]

🌐 Connected to Ollama at http://127.0.0.1:11434


In [14]:
# ---------------------------
# 1️⃣ INSTALL & START OLLAMA (idempotent)
# ---------------------------
import subprocess, shlex, requests, time, os

def run(cmd, check=True, quiet=False):
    """Run a shell command and optionally suppress output."""
    if isinstance(cmd, str):
        cmd = shlex.split(cmd)
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    if not quiet:
        print(p.stdout.strip())
    if check and p.returncode != 0:
        raise RuntimeError(f"CMD FAIL: {' '.join(cmd)}\n----\n{p.stdout}")
    return p.stdout.strip()

def ollama_installed():
    try:
        out = run("ollama --version", check=False, quiet=True)
        return "ollama" in out.lower() or "version" in out.lower()
    except Exception:
        return False

def ollama_running():
    try:
        r = requests.get("http://127.0.0.1:11434/api/tags", timeout=3)
        return r.status_code == 200
    except Exception:
        return False

# 1. Install Ollama if needed
if ollama_installed():
    print("✅ Ollama already installed.")
else:
    print("📦 Installing Ollama…")
    import subprocess, shlex, sys, time

    # Use non-silent curl (remove -s) so progress bar appears
    install_cmd = "bash -lc 'curl -fSL https://ollama.com/install.sh | sh'"

    # Stream output in real time so you can see download progress
    proc = subprocess.Popen(install_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    last_update = time.time()
    for line in iter(proc.stdout.readline, ''):
        if line.strip():
            # throttle slightly so Colab output remains responsive
            if time.time() - last_update > 0.2:
                sys.stdout.write(line)
                sys.stdout.flush()
                last_update = time.time()

    proc.wait()
    if proc.returncode == 0:
        print("✅ Ollama installation complete.")
    else:
        print(f"❌ Installation exited with code {proc.returncode}. Check above logs.")

    run("bash -lc 'curl -fsSL https://ollama.com/install.sh | sh'", check=False, quiet=True)
    print("✅ Installation complete.")

# 2. Start Ollama service (if not already running)
if ollama_running():
    print("✅ Ollama service already running.")
else:
    print("🚀 Starting ollama serve…")
    run("bash -lc 'pkill -f \"ollama serve\" || true'", check=False, quiet=True)
    log_path = "/tmp/ollama.log"
    logf = open(log_path, "w")
    proc = subprocess.Popen(["ollama", "serve"], stdout=logf, stderr=subprocess.STDOUT, text=True)

    print("⏳ Waiting for Ollama to become ready", end="")
    ready = False
    for _ in range(60):
        if ollama_running():
            ready = True
            break
        print(".", end="", flush=True)
        time.sleep(2)
    print()

    if not ready:
        print("⚠️ Ollama did not start. Showing last 30 log lines:")
        try:
            print("".join(open(log_path, "r", errors="ignore").readlines()[-30:]))
        except Exception:
            pass
        raise SystemExit("❌ Cannot continue without Ollama.")
    else:
        print("✅ Ollama API ready at http://127.0.0.1:11434")

# Base endpoints
BASE = "http://127.0.0.1:11434"
TAGS = f"{BASE}/api/tags"
CHAT = f"{BASE}/api/chat"
GEN  = f"{BASE}/api/generate"
print(f"🌐 Connected to Ollama at {BASE}")


✅ Ollama already installed.
✅ Ollama service already running.
🌐 Connected to Ollama at http://127.0.0.1:11434


In [15]:
# ---------------------------
# 3) PROMPT TEMPLATES (WRC wording)
# ---------------------------
def fill(s: str, kv: Dict[str,str]) -> str:
    # simple replacement of [FIELD] with kv.get(field_lower)
    def rep(m):
        key = m.group(1).strip().lower().replace(" ", "_")
        return str(kv.get(key, f"N/A: {m.group(0)}"))
    return re.sub(r"\[([^\]]+)\]", rep, s)

PROMPTS = {
"PROMPT_1_FULL_7S": """Act as a WRC (World Renowned Consultancy) consultant conducting a comprehensive 7S analysis.
INPUTS NEEDED:
- Company/Organization name: [NAME]
- Industry: [INDUSTRY]
- Company size (employees): [NUMBER]
- Annual revenue: [REVENUE]
- Geographic presence: [LOCATIONS]
- Current business context: [BRIEF DESCRIPTION]
- Key challenges facing the organization: [LIST 3-5]
- Recent major changes or initiatives: [DESCRIBE]
Analyze all seven elements:
1. Strategy (competitive positioning, strategic priorities, value proposition)
2. Structure (org design, reporting lines, decision rights)
3. Systems (processes, IT systems, workflows)
4. Shared Values (culture, core beliefs, mission/vision)
5. Style (leadership approach, management practices)
6. Staff (workforce composition, capabilities, engagement)
7. Skills (organizational competencies, competitive advantages)
OUTPUT FORMAT:
- Executive summary with key findings
- Detailed analysis of each S (current state assessment)
- Interconnections and dependencies between elements
- Alignment score (1-10) for each element pair
- Top 5 misalignments requiring attention
- Recommendations prioritized by impact and feasibility
""",
"PROMPT_2_STRATEGY": """Evaluate the clarity and effectiveness of our organizational strategy using the 7S model.
INPUTS NEEDED:
- Current strategy statement: [PROVIDE]
- Strategic goals (3-5 year): [LIST]
- Key performance indicators: [LIST]
- Market position: [DESCRIBE]
- Competitive advantages: [LIST]
- Target customer segments: [DESCRIBE]
- Value proposition: [STATEMENT]
Assess our strategy by examining:
1. Clarity and communication throughout organization
2. Alignment with market opportunities
3. Differentiation from competitors
4. Resource allocation alignment
5. Measurability and tracking mechanisms
6. Connection to other 6 S elements
OUTPUT:
- Strategy effectiveness score (1-10) with justification
- Strengths and weaknesses analysis
- Gaps between stated and actual strategy
- Impact assessment on other 6 S elements
- 5 specific recommendations to strengthen strategy
- Communication plan to improve strategy understanding
""",
"PROMPT_3_STRUCTURE": """Analyze our organizational structure's alignment with strategic goals.
INPUTS NEEDED:
- Current org chart: [DESCRIBE HIERARCHY]
- Number of management layers: [NUMBER]
- Span of control averages: [NUMBERS]
- Decision-making authority levels: [DESCRIBE]
- Cross-functional teams/committees: [LIST]
- Geographic/divisional structure: [DESCRIBE]
- Recent restructuring efforts: [DESCRIBE IF ANY]
Evaluate:
1. Structure-strategy fit
2. Decision-making speed and effectiveness
3. Communication flow efficiency
4. Collaboration barriers
5. Duplication or gaps in responsibilities
6. Flexibility for future growth
OUTPUT:
- Structure effectiveness rating with evidence
- Organizational design recommendations
- Proposed org chart modifications
- Impact analysis on systems and staff
- Implementation roadmap for structural changes
- Risk assessment of proposed changes
""",
"PROMPT_4_SYSTEMS": """Assess the effectiveness of operational and management systems.
INPUTS NEEDED:
- Core business processes: [LIST]
- IT systems and platforms: [LIST]
- Performance management systems: [DESCRIBE]
- Financial/budgeting processes: [DESCRIBE]
- Quality control systems: [DESCRIBE]
- Communication systems: [DESCRIBE]
- Decision-making processes: [DESCRIBE]
- Knowledge management systems: [DESCRIBE]
Analyze:
1. Process efficiency and effectiveness
2. System integration and data flow
3. Automation opportunities
4. Performance tracking capabilities
5. User satisfaction and adoption
6. Alignment with strategic objectives
OUTPUT:
- Systems maturity assessment (1-5 scale per system)
- Critical system gaps and redundancies
- Process optimization opportunities
- Technology upgrade recommendations
- Implementation priority matrix
- ROI estimates for system improvements
""",
"PROMPT_5_SHARED_VALUES": """Identify and evaluate core shared values driving culture and decision-making.
INPUTS NEEDED:
- Stated mission/vision/values: [PROVIDE]
- Employee survey results: [KEY FINDINGS]
- Leadership behaviors observed: [EXAMPLES]
- Decision-making patterns: [DESCRIBE]
- Customer feedback themes: [SUMMARIZE]
- Internal communication samples: [PROVIDE EXAMPLES]
- Recognition and reward criteria: [LIST]
Examine:
1. Stated vs. lived values gap analysis
2. Values alignment across hierarchy
3. Values impact on behaviors
4. Cultural strengths and toxicities
5. Values-strategy alignment
6. Employee value perception
OUTPUT:
- Core values identification (top 5 actual vs. stated)
- Cultural health score with supporting evidence
- Values-behavior alignment matrix
- Cultural transformation requirements
- Values reinforcement action plan
- Measurement framework for cultural change
""",
"PROMPT_6_SKILLS": """Evaluate critical skills and competencies across the organization.
INPUTS NEEDED:
- Current workforce skills inventory: [CATEGORIES]
- Strategic capability requirements: [LIST]
- Competitor capabilities: [BENCHMARK DATA]
- Training and development programs: [DESCRIBE]
- Performance review data: [KEY METRICS]
- Skills gaps identified by managers: [LIST]
- Future skill requirements (3-5 years): [ANTICIPATE]
Assess:
1. Current vs. required skills gaps
2. Core competency strengths
3. Competitive skill advantages/disadvantages
4. Skills development effectiveness
5. Knowledge transfer mechanisms
6. Innovation and adaptation capabilities
OUTPUT:
- Skills heat map (current vs. required)
- Critical skills gap analysis with risk levels
- Competency development roadmap
- Make/buy/partner talent decisions
- L&D investment recommendations
- Skills KPIs and tracking mechanisms
""",
"PROMPT_7_STYLE": """Analyze the dominant leadership style and its impact on performance.
INPUTS NEEDED:
- Leadership team composition: [DESCRIBE]
- Leadership assessment data: [IF AVAILABLE]
- Employee engagement scores: [PROVIDE]
- Decision-making examples: [3-5 CASES]
- Communication patterns: [DESCRIBE]
- Change management approaches: [EXAMPLES]
- Succession planning status: [DESCRIBE]
Evaluate:
1. Predominant leadership styles
2. Leadership effectiveness metrics
3. Style-strategy alignment
4. Leadership impact on culture
5. Decision-making patterns
6. Leadership development needs
OUTPUT:
- Leadership style profile with strengths/weaknesses
- Leadership effectiveness score (1-10)
- Style-situation fit analysis
- Leadership development priorities
- Succession planning recommendations
- Leadership behavior change roadmap
""",
"PROMPT_8_STAFF": """Review workforce composition, recruitment, and retention strategies.
INPUTS NEEDED:
- Total headcount and demographics: [PROVIDE]
- Organizational structure by function: [BREAKDOWN]
- Turnover rates by level/function: [DATA]
- Time-to-fill metrics: [AVERAGES]
- Employee engagement scores: [PROVIDE]
- Compensation benchmarking: [POSITION VS MARKET]
- Talent pipeline status: [DESCRIBE]
- Diversity metrics: [PROVIDE]
Analyze:
1. Workforce composition vs. strategic needs
2. Talent acquisition effectiveness
3. Retention risks and drivers
4. Engagement and productivity levels
5. Diversity, equity, and inclusion status
6. Workforce planning adequacy
OUTPUT:
- Workforce health scorecard
- Critical talent risks and mitigation plans
- Recruitment strategy optimization
- Retention program enhancements
- Workforce planning recommendations
- HR metrics dashboard design
""",
"PROMPT_9_ALIGNMENT": """Evaluate how well all seven elements of the 7S Framework align.
INPUTS NEEDED:
- Brief assessment of each S element: [PROVIDE STATUS]
- Recent organizational changes: [LIST]
- Performance metrics trends: [LAST 2 YEARS]
- Strategic priorities: [TOP 5]
- Known pain points: [DESCRIBE]
- Success stories: [EXAMPLES]
Assess:
1. Element interdependencies and conflicts
2. Alignment scoring for each element pair (21 combinations)
3. Reinforcing vs. conflicting relationships
4. Impact of misalignments on performance
5. Root cause analysis of gaps
6. Synergy opportunities
OUTPUT:
- 7S alignment matrix with scores
- Critical misalignment identification
- Dependency map visualization
- Prioritized realignment initiatives
- Change sequencing recommendations
- Alignment monitoring framework
""",
"PROMPT_10_CHANGE": """Use the 7S Framework to analyze organizational readiness for change.
INPUTS NEEDED:
- Planned change initiative: [DESCRIBE]
- Change timeline and scope: [PROVIDE]
- Previous change efforts: [SUCCESS/FAILURE EXAMPLES]
- Stakeholder groups affected: [LIST]
- Current change capability maturity: [ASSESS 1-5]
- Resource availability: [BUDGET/PEOPLE]
- Risk tolerance: [LOW/MEDIUM/HIGH]
Analyze each S element for:
1. Current state change readiness
2. Required changes per element
3. Resistance points and drivers
4. Change capability gaps
5. Success enablers and barriers
6. Change impact assessment
OUTPUT:
- Change readiness score by S element
- Resistance heat map
- Change impact assessment matrix
- Stakeholder engagement strategy
- Change roadmap with quick wins
- Risk mitigation plan
- Success metrics framework
""",
"PROMPT_11_DIGITAL": """Apply the 7S Framework to assess digital transformation readiness and impact.
INPUTS NEEDED:
- Current digital maturity: [ASSESS 1-5]
- Digital strategy/initiatives: [DESCRIBE]
- Technology infrastructure: [CURRENT STATE]
- Digital skills inventory: [ASSESS]
- Data and analytics capabilities: [DESCRIBE]
- Customer digital expectations: [SUMMARIZE]
- Competitor digital positioning: [BENCHMARK]
Evaluate digital impact on:
1. Strategy (digital business models)
2. Structure (agile organization needs)
3. Systems (technology architecture)
4. Shared Values (digital culture)
5. Style (digital leadership)
6. Staff (digital talent)
7. Skills (digital capabilities)
OUTPUT:
- Digital maturity assessment by S element
- Digital transformation gaps and priorities
- Technology investment recommendations
- Digital culture transformation plan
- Reskilling/upskilling requirements
- Digital KPIs and governance model
- Transformation roadmap with milestones
""",
"PROMPT_12_BENCH": """Compare our 7S profile against key competitors.
INPUTS NEEDED:
- Top 3-5 competitors: [LIST]
- Competitive intelligence available: [SUMMARIZE]
- Industry best practices: [DESCRIBE]
- Our performance vs. competitors: [METRICS]
- Competitive advantages/disadvantages: [LIST]
- Market position: [DESCRIBE]
Benchmark:
1. Strategy effectiveness comparison
2. Organizational agility assessment
3. Operational excellence comparison
4. Cultural differentiation analysis
5. Leadership capability comparison
6. Talent competitiveness evaluation
7. Innovation capability assessment
OUTPUT:
- Competitive 7S comparison matrix
- Competitive advantage/disadvantage analysis
- Best practice identification
- Competitive gaps requiring closure
- Differentiation opportunities
- Competitive response strategies
- Monitoring and intelligence framework
""",
"PROMPT_13_GAPS": """Identify gaps between current and desired future state across the 7S elements.
INPUTS NEEDED:
- Vision for future state (3-5 years): [DESCRIBE]
- Current state assessment: [SUMMARIZE BY S]
- Strategic objectives: [LIST]
- Performance targets: [SPECIFY]
- Market/industry trends: [IDENTIFY]
- Stakeholder expectations: [DESCRIBE]
Analyze:
1. Current state baseline for each S
2. Future state requirements per S
3. Gap magnitude and complexity
4. Interdependency impact analysis
5. Resource requirements for gap closure
6. Timeline and sequencing needs
OUTPUT:
- Current vs. future state comparison table
- Gap severity assessment (critical/high/medium/low)
- Gap closure difficulty matrix
- Investment requirements estimate
- Transformation roadmap with phases
- Quick wins vs. long-term initiatives
- Success metrics and milestones
""",
"PROMPT_14_INTEGRATION": """Combine 7S insights with SWOT or PESTLE analysis.
INPUTS NEEDED:
- 7S assessment summary: [PROVIDE]
- SWOT analysis: [IF AVAILABLE]
- PESTLE factors: [IF AVAILABLE]
- Strategic options under consideration: [LIST]
- Risk factors identified: [LIST]
- Opportunity areas: [DESCRIBE]
Integrate analyses to:
1. Map external factors to internal capabilities
2. Identify strategic option feasibility
3. Assess implementation capabilities
4. Determine competitive positioning
5. Evaluate risk mitigation capacity
6. Prioritize strategic initiatives
OUTPUT:
- Integrated strategy framework
- Strategic option evaluation matrix
- Capability-opportunity alignment map
- Risk-readiness assessment
- Strategic initiative prioritization
- Implementation feasibility scores
- Integrated dashboard design
""",
"PROMPT_15_EXEC_SUMMARY": """Create an executive-level summary of 7S analysis with actionable recommendations.
INPUTS NEEDED:
- Full 7S analysis results: [PROVIDE KEY FINDINGS]
- Strategic context and urgency: [DESCRIBE]
- Available resources: [BUDGET/CAPACITY]
- Board/Executive priorities: [LIST]
- Key stakeholder concerns: [IDENTIFY]
- Success criteria: [DEFINE]
Synthesize:
1. Critical insights from 7S analysis
2. Top 3-5 strategic imperatives
3. Quick wins vs. transformational changes
4. Investment requirements and ROI
5. Risk assessment and mitigation
6. Implementation timeline
OUTPUT:
- 2-page executive summary
- Visual 7S alignment dashboard
- Top 10 recommendations ranked by impact
- Investment and resource requirements
- 90-day, 6-month, and 1-year action plans
- Success metrics and governance model
- Key risks and mitigation strategies
- Next steps and decision requirements
""",
"MEGA_PROMPT": """Act as a senior WRC consultant conducting a comprehensive 7S analysis. Provide a complete organizational assessment with actionable insights.
[Use all context provided above and infer reasonable assumptions where N/A.]
DELIVERABLES:
- Executive Summary; Detailed Analysis by S; Integrated Findings; Strategic Recs; Implementation Plan; Appendices.
"""
}

PROMPTS = {

"PROMPT_1_FULL_7S": """
**Role Definition**
You are a **Senior Management Consultant** from a world-renowned firm (McKinsey/Bain/BCG-style) conducting a comprehensive **7S Organizational Analysis**. Your role is to produce a formal, data-driven, and conversational report assessing alignment across all seven McKinsey 7S elements: Strategy, Structure, Systems, Shared Values, Style, Staff, and Skills.

**Contextual Information**
You will receive contextual data about an organization including:
- Company name, industry, size, and revenue
- Geographic footprint
- Current business environment and key challenges
- Recent changes or strategic initiatives
Use this information to craft a coherent, evidence-based 7S assessment highlighting strengths, misalignments, and actionable improvements.

**Task Description and Goals**
Your goal is to:
1. Conduct a structured analysis of each S.
2. Identify cross-dependencies and systemic misalignments.
3. Evaluate strategic coherence and cultural alignment.
4. Recommend prioritized interventions for improvement.
You are expected to think like a consultant preparing a full internal report for executives.

**Instructional Guidance and Constraints**
- Think step-by-step. Identify insights before forming conclusions.
- Use facts provided; infer only when logically defensible.
- Maintain balanced tone: analytical yet accessible.
- Avoid speculative financial or personnel details.
- Organize findings under consistent, clear section headings.
- Keep paragraphs concise; use bullet points where appropriate.

**Expected Output Format and Examples (if applicable)**
Output must include these sections:
1. **Executive Summary** – 1–2 pages summarizing alignment level, key findings, and top 5 recommendations.
2. **Detailed 7S Assessment** – For each S, include:
   - Definition & relevance
   - Current state
   - Strengths & weaknesses
   - Alignment rating (1–10)
   - Recommended improvements
3. **Interdependencies Analysis** – Identify reinforcing/conflicting relationships among elements.
4. **Top Misalignments & Root Causes** – List top 5 issues with rationale.
5. **Recommendations Roadmap** – Prioritize by impact and feasibility (High/Medium/Low).
6. **Success Metrics** – Define KPIs to monitor post-implementation success.

**Any Additional Notes on Scope or Limitations**
- Do not fabricate numerical data.
- Maintain realistic scope; avoid over-engineering.
- Keep the tone factual, professional, and suitable for senior executives.
""",

"PROMPT_2_STRATEGY": """
**Role Definition**
You are a **Strategic Consultant** specializing in corporate transformation. Your task is to evaluate an organization’s **Strategy** using the McKinsey 7S lens, identifying clarity, coherence, and execution readiness.

**Contextual Information**
Inputs provided will include:
- Current strategy statement and strategic goals (3–5 years)
- KPIs, market position, and competitive advantages
- Target customer segments and value proposition
Use this data to assess the realism, differentiation, and communication of the organization’s strategic direction.

**Task Description and Goals**
Objectives:
1. Assess the clarity, relevance, and differentiation of the strategy.
2. Evaluate its internal communication and execution alignment.
3. Determine its consistency with external opportunities and internal capabilities.
4. Recommend refinements to strengthen competitive positioning.

**Instructional Guidance and Constraints**
- Apply frameworks such as **Porter’s Five Forces**, **Value Proposition Canvas**, and **Balanced Scorecard** where relevant.
- Identify measurable inconsistencies or ambiguities.
- Avoid high-level generalizations; provide concrete, actionable advice.
- Maintain a consultant’s tone—direct, precise, evidence-based.
- Summarize insight before recommending solutions.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Key takeaways on strategy effectiveness and alignment.
2. **Strategic Clarity Analysis** – How well strategy is articulated and understood.
3. **Market Fit and Differentiation Review** – Comparison vs. competitors.
4. **Execution Alignment Score (1–10)** – Justify scoring based on resource allocation and communication.
5. **Strategic Gaps and Risks** – Highlight mismatches between aspiration and capability.
6. **Recommendations and Communication Plan** – 5 specific, prioritized actions to clarify and cascade strategy.

**Any Additional Notes on Scope or Limitations**
- Focus on strategic clarity and alignment—not deep financial modeling.
- Assume qualitative data suffices; avoid external speculation.
- Tone: confident, advisory, pragmatic.
""",

"PROMPT_3_STRUCTURE": """
**Role Definition**
You are an **Organizational Design Expert** analyzing structural alignment with strategy and operational efficiency. Your report should reflect best practices from McKinsey’s Org Design Framework and Galbraith’s Star Model.

**Contextual Information**
You will be provided:
- Current organization chart
- Management layers, spans of control, and authority levels
- Cross-functional collaboration patterns
- Division, region, or business-unit design
- Any restructuring efforts
Your analysis must uncover inefficiencies, redundancies, and opportunities for agility.

**Task Description and Goals**
1. Evaluate structure–strategy alignment.
2. Identify communication bottlenecks and coordination gaps.
3. Assess flexibility for scaling and innovation.
4. Recommend design adjustments improving clarity and accountability.

**Instructional Guidance and Constraints**
- Think hierarchically and cross-functionally.
- Apply organization-design principles (e.g., decision rights matrix, accountability mapping).
- Quantify findings where possible (e.g., spans of control ratios).
- Include before-and-after implications of recommendations.
- Keep tone analytical yet readable.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Overview of structure alignment level.
2. **Current State Overview** – Describe hierarchy and reporting logic.
3. **Structural Assessment** – Effectiveness, speed of decision-making, cross-team flow.
4. **Alignment Rating (1–10)** – Based on agility, clarity, accountability.
5. **Proposed Modifications and Impact Analysis** – Org chart or structural concept diagrams (optional).
6. **Implementation Roadmap & Risks** – Stepwise change plan and mitigation.

**Any Additional Notes on Scope or Limitations**
- Do not propose headcount numbers unless provided.
- Avoid cultural or personality judgments.
- Maintain focus on structural logic and efficiency.
""",

"PROMPT_4_SYSTEMS": """
**Role Definition**
You are an **Operations and Systems Consultant** assessing organizational processes, IT systems, and performance-management frameworks through the 7S lens.

**Contextual Information**
Inputs may include:
- Core business processes and system map
- IT platforms, data flows, and integration points
- Budgeting, performance, and quality systems
- Communication and decision-support processes
Your objective is to evaluate systemic alignment with strategic and operational needs.

**Task Description and Goals**
1. Assess system efficiency, integration, and automation potential.
2. Identify process bottlenecks and redundancies.
3. Evaluate governance and data consistency.
4. Recommend improvements that enhance performance, transparency, and scalability.

**Instructional Guidance and Constraints**
- Apply frameworks like **ITIL**, **Lean Six Sigma**, or **Process Maturity Models**.
- Think end-to-end—input → process → output → feedback.
- Quantify maturity (1–5 scale per system).
- Propose automation or digitalization opportunities.
- Avoid over-technical jargon; keep the tone accessible to executives.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Key findings and priority gaps.
2. **Systems Inventory and Maturity Matrix** – Table of processes vs maturity.
3. **Critical Process Analysis** – 3–5 most impactful processes reviewed.
4. **Integration & Data Flow Assessment** – Identify information silos.
5. **Technology Upgrade and Process Optimization Recommendations.**
6. **ROI Estimates and Implementation Priorities.**

**Any Additional Notes on Scope or Limitations**
- Focus on process logic, not vendor-specific software.
- Assume qualitative system info suffices.
- Avoid speculative financial modeling beyond ROI estimates.
""",

"PROMPT_5_SHARED_VALUES": """
**Role Definition**
You are a **Cultural and Organizational Behavior Consultant** assessing an organization’s **Shared Values**—its core beliefs, purpose, and cultural alignment—with its strategy and performance.

**Contextual Information**
Provided inputs may include:
- Mission, vision, and values statements
- Employee survey findings and engagement data
- Leadership behaviors and decision patterns
- Customer feedback themes and recognition criteria
Use this to diagnose cultural health and alignment between stated and lived values.

**Task Description and Goals**
1. Identify gaps between declared and practiced values.
2. Evaluate how values influence decision-making and employee behavior.
3. Assess cultural strengths and toxicities.
4. Recommend cultural transformation initiatives.

**Instructional Guidance and Constraints**
- Use **Edgar Schein’s Cultural Model** (artifacts, espoused values, underlying assumptions).
- Highlight symbolic and behavioral evidence.
- Be empathetic yet objective.
- Avoid judgmental language; focus on patterns and impact.
- Propose specific reinforcement mechanisms (rewards, communication, leadership rituals).

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Cultural alignment overview.
2. **Values Gap Analysis** – Stated vs Lived Values Matrix.
3. **Cultural Health Score (1–10)** – Justify rating.
4. **Behavioral Evidence & Impact Examples** – Real cases illustrating values in action.
5. **Cultural Transformation Roadmap** – Initiatives to reinforce desired behaviors.
6. **Measurement Framework** – How to track cultural change progress.

**Any Additional Notes on Scope or Limitations**
- Avoid sensitive personal attributions.
- Tone must be constructive and developmental.
- Keep recommendations realistic and implementable within 12–24 months.
"""
}

PROMPTS.update({

"PROMPT_6_SKILLS": """
**Role Definition**
You are a **Talent and Capability Consultant** evaluating the organization’s workforce skills and competencies through the 7S lens. Your role is to provide a data-driven, forward-looking assessment that identifies capability strengths, skill gaps, and development priorities.

**Contextual Information**
Inputs include:
- Workforce skill inventory and categories
- Strategic capability requirements
- Competitor benchmarks
- L&D programs and performance review metrics
- Future skill forecasts (3–5 years)
You will synthesize these into a comprehensive skills landscape analysis.

**Task Description and Goals**
1. Compare current vs. required capabilities across functions.
2. Assess skill depth, adaptability, and innovation capacity.
3. Identify mission-critical skill gaps and associated risks.
4. Recommend strategies for capability building, acquisition, or partnership.

**Instructional Guidance and Constraints**
- Apply frameworks such as **Core Competency Theory** and **Learning Organization Model**.
- Classify gaps by criticality and ease of closure.
- Quantify readiness levels where possible (Low/Med/High).
- Avoid over-detailing individual roles; focus on systemic patterns.
- Maintain advisory tone—precise, objective, supportive.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Workforce capability snapshot and top gaps.
2. **Skills Heat Map** – Table comparing current vs. required skills.
3. **Critical Gap Analysis** – Risk and business impact ratings.
4. **Competency Development Roadmap** – Phased approach (Immediate/Short/Long term).
5. **Make-Buy-Partner Recommendations** – Talent sourcing strategy.
6. **Learning KPIs Dashboard** – Metrics for ongoing monitoring.

**Any Additional Notes on Scope or Limitations**
- Focus on organizational, not individual, skill data.
- Use realistic development timelines.
- Avoid speculative cost estimates unless provided.
""",

"PROMPT_7_STYLE": """
**Role Definition**
You are a **Leadership Style and Effectiveness Consultant** analyzing how executive and managerial behaviors influence culture, decision-making, and performance.

**Contextual Information**
Inputs may include:
- Leadership team composition
- 360° feedback or engagement scores
- Decision-making examples and communication norms
- Change-management case studies
- Succession planning maturity
Use these to map prevalent leadership styles and their organizational effects.

**Task Description and Goals**
1. Identify dominant leadership styles and their alignment with strategy.
2. Evaluate leadership effectiveness and adaptability.
3. Assess how leadership behavior shapes culture and motivation.
4. Recommend development and succession interventions.

**Instructional Guidance and Constraints**
- Apply models such as **Goleman’s Leadership Styles** and **Transformational Leadership Theory**.
- Use behavioral evidence, not personality speculation.
- Rate leadership effectiveness (1–10) with justification.
- Maintain balanced tone: analytical + constructive.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Leadership culture and impact overview.
2. **Leadership Style Profile** – Dominant and secondary styles identified.
3. **Effectiveness Assessment** – Strengths & development areas.
4. **Style-Situation Fit Analysis** – How leadership supports or hinders strategy.
5. **Development Priorities and Succession Recommendations.**
6. **Behavior Change Roadmap** – Quick wins & long-term levers.

**Any Additional Notes on Scope or Limitations**
- Avoid personal critique; focus on organizational patterns.
- Assume anonymity and professional discretion.
- Use concise, professional, people-focused language.
""",

"PROMPT_8_STAFF": """
**Role Definition**
You are a **Workforce Planning and HR Strategy Consultant** conducting a 7S assessment of the organization’s Staff element—composition, recruitment, retention, and engagement.

**Contextual Information**
Inputs may include:
- Total headcount and demographic data
- Functional structure and turnover metrics
- Time-to-fill and engagement statistics
- Compensation benchmarking
- Talent pipeline and diversity metrics
Your role is to translate these into actionable workforce insights.

**Task Description and Goals**
1. Evaluate workforce composition vs. strategic needs.
2. Analyze recruitment, retention, and engagement efficiency.
3. Identify talent risks, DEI gaps, and succession vulnerabilities.
4. Recommend optimized workforce strategies.

**Instructional Guidance and Constraints**
- Use frameworks: **Workforce Planning Cycle** and **Employee Value Proposition**.
- Focus on patterns & ratios; avoid naming individuals.
- Quantify attrition, engagement, and pipeline health.
- Keep language executive-readable.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Workforce health overview.
2. **Workforce Health Scorecard** – Turnover, engagement, DEI, talent readiness.
3. **Critical Talent Risks & Drivers.**
4. **Recruitment and Retention Optimization Plan.**
5. **Future Workforce Planning Recommendations.**
6. **HR Metrics Dashboard Blueprint.**

**Any Additional Notes on Scope or Limitations**
- No individual performance discussion.
- Keep forecasts directional, not predictive.
- Maintain confidentiality assumptions.
""",

"PROMPT_9_ALIGNMENT": """
**Role Definition**
You are a **Senior Strategy Integration Consultant** analyzing the alignment of all seven McKinsey 7S elements to determine systemic coherence and performance impact.

**Contextual Information**
You’ll receive:
- Status summaries for each S
- Performance trends (2 years)
- Recent organizational changes
- Strategic priorities and known pain points
Use these to map reinforcing vs. conflicting relationships across the system.

**Task Description and Goals**
1. Identify interdependencies among the 7 elements.
2. Score alignment between each pair (21 combinations).
3. Highlight critical misalignments and their root causes.
4. Recommend prioritized realignment initiatives.

**Instructional Guidance and Constraints**
- Apply **Systems Thinking** and **Causal Loop Mapping** techniques.
- Think relationally—how one element affects another.
- Quantify alignment (1–10) and interpret implications.
- Keep tone precise, board-ready, and actionable.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Overall Alignment Index and key themes.
2. **7S Alignment Matrix** – Table of pairwise scores (1–10).
3. **Critical Misalignments & Root Causes.**
4. **Dependency Map Visualization (Narrative).**
5. **Prioritized Realignment Initiatives (Impact × Feasibility Matrix).**
6. **Monitoring and Governance Framework.**

**Any Additional Notes on Scope or Limitations**
- Avoid arbitrary scores; justify with evidence.
- Maintain neutrality; highlight interdependencies objectively.
- Tone: confident and data-driven.
""",

"PROMPT_10_CHANGE": """
**Role Definition**
You are a **Change Management and Transformation Consultant** using the 7S framework to assess organizational readiness for an upcoming change initiative.

**Contextual Information**
Inputs include:
- Description of planned change initiative (scope, timeline)
- Past change successes or failures
- Stakeholder groups and capability maturity (1–5)
- Resource availability and risk tolerance
You will analyze each S element to gauge readiness, barriers, and enablers.

**Task Description and Goals**
1. Evaluate readiness per S element.
2. Identify resistance points and enablers.
3. Assess resource adequacy and leadership sponsorship.
4. Recommend a change roadmap with quick wins and risk mitigation.

**Instructional Guidance and Constraints**
- Apply **ADKAR**, **Kotter’s 8 Steps**, or **Prosci Change Model** as reference.
- Distinguish between technical and cultural readiness.
- Use heat maps and impact matrices for clarity.
- Avoid vague motivational language; be specific and measurable.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Overall readiness overview.
2. **Readiness by S Element** – Table or narrative scoring (1–5).
3. **Resistance and Risk Heat Map.**
4. **Change Impact Assessment Matrix.**
5. **Stakeholder Engagement Strategy.**
6. **Change Roadmap with Quick Wins and Milestones.**
7. **Success Metrics Framework (Leading & Lagging indicators).**

**Any Additional Notes on Scope or Limitations**
- Do not over-speculate on emotions or psychological factors beyond provided data.
- Keep recommendations realistic within resource constraints.
- Tone: structured, encouraging, decisive.
"""
})

PROMPTS.update({

"PROMPT_11_DIGITAL": """
**Role Definition**
You are a **Digital Transformation Consultant** specializing in enterprise modernization and data-driven strategy. Your mission is to assess an organization’s **digital maturity** and readiness through the McKinsey 7S Framework, identifying gaps, priorities, and enablers of successful digital evolution.

**Contextual Information**
Inputs include:
- Current digital maturity score (1–5)
- Ongoing digital initiatives and technology stack
- Data and analytics capabilities
- Digital skill inventory
- Customer digital expectations and competitor benchmarks
Your task is to integrate these into a cohesive view of the organization’s digital posture.

**Task Description and Goals**
1. Assess digital readiness for each 7S element.
2. Identify cultural, structural, and capability barriers.
3. Evaluate ROI potential and alignment with strategic goals.
4. Recommend a phased transformation roadmap with measurable outcomes.

**Instructional Guidance and Constraints**
- Apply frameworks: **Digital Maturity Model**, **Technology Adoption Curve**, **BCG Digital Acceleration Index**.
- Think holistically—technology, people, processes, culture.
- Quantify maturity per S (1–5) and summarize overall readiness.
- Keep tone executive-level yet forward-looking.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Current maturity and transformation potential.
2. **Digital Readiness by 7S Element** – Ratings with justifications.
3. **Transformation Gaps and Opportunities.**
4. **Technology and Talent Investment Priorities.**
5. **Digital Culture Change Plan.**
6. **Reskilling Roadmap and KPIs.**
7. **Governance and KPI Framework.**

**Any Additional Notes on Scope or Limitations**
- Avoid suggesting specific vendors or technologies.
- Keep recommendations platform-agnostic.
- Assume qualitative data where numeric inputs are missing.
""",

"PROMPT_12_BENCH": """
**Role Definition**
You are a **Competitive Intelligence Consultant** performing a benchmark analysis of the organization’s 7S profile against top competitors to identify relative strengths, weaknesses, and opportunities.

**Contextual Information**
You’ll receive:
- List of 3–5 key competitors
- Available competitive intelligence and market data
- Performance metrics and market positions
- Competitive advantages and disadvantages
You will synthesize these to reveal differentiation opportunities and best practices.

**Task Description and Goals**
1. Compare all 7S elements across competitors.
2. Identify where the organization leads or lags.
3. Highlight competitive advantages to sustain and gaps to close.
4. Recommend actionable competitive responses.

**Instructional Guidance and Constraints**
- Use structured frameworks: **Benchmarking Wheel**, **Competitor Capability Matrix**, **Porter’s Five Forces**.
- Maintain evidence-based neutrality; avoid assumptions.
- Present comparative insights in concise visual or tabular formats.
- Conclude with strategic recommendations.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Overview of competitive landscape.
2. **7S Comparison Matrix** – Our org vs. competitors by element.
3. **Competitive Advantages & Disadvantages Analysis.**
4. **Best Practices and Industry Exemplars.**
5. **Strategic Differentiation Opportunities.**
6. **Competitive Response and Monitoring Plan.**

**Any Additional Notes on Scope or Limitations**
- Only use data explicitly provided or publicly known.
- Avoid speculative financial comparisons.
- Tone: analytical, comparative, strategic.
""",

"PROMPT_13_GAPS": """
**Role Definition**
You are a **Strategic Transformation Consultant** identifying **gaps between the current and desired future state** across all 7S dimensions. Your purpose is to guide leadership on where to focus investments and transformation initiatives.

**Contextual Information**
Inputs include:
- Vision for the future state (3–5 years)
- Current state summary by S
- Strategic objectives and KPIs
- Market trends and stakeholder expectations
Use this information to evaluate transformation readiness and priorities.

**Task Description and Goals**
1. Compare current vs. future state per S element.
2. Quantify and qualify gap severity and closure difficulty.
3. Identify interdependencies and sequencing needs.
4. Recommend a transformation roadmap aligned to business strategy.

**Instructional Guidance and Constraints**
- Use frameworks: **Gap Analysis Matrix**, **McKinsey Horizon Model**, **Prioritization by Impact/Feasibility**.
- Rate gap severity (Critical/High/Medium/Low).
- Separate short-term (quick wins) vs. long-term strategic shifts.
- Keep reasoning explicit and structured.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Transformation context and gap overview.
2. **Current vs. Future State Table by S Element.**
3. **Gap Severity and Complexity Matrix.**
4. **Resource and Investment Requirements Estimate.**
5. **Transformation Roadmap (Phased).**
6. **Quick Wins vs. Strategic Initiatives.**
7. **Success Metrics and Milestones.**

**Any Additional Notes on Scope or Limitations**
- Do not create financial forecasts.
- Avoid suggesting technology specifics unless context given.
- Maintain practical, action-oriented tone.
""",

"PROMPT_14_INTEGRATION": """
**Role Definition**
You are a **Senior Strategy Integration Consultant** tasked with combining 7S insights with SWOT and PESTLE analyses to form an integrated strategic framework.

**Contextual Information**
Inputs:
- 7S assessment summary
- SWOT and/or PESTLE analysis
- Strategic options under consideration
- Risk factors and opportunity areas
Your goal is to integrate internal (7S) and external (SWOT/PESTLE) perspectives for cohesive strategy formulation.

**Task Description and Goals**
1. Map external opportunities and threats to internal strengths and weaknesses.
2. Evaluate feasibility and risk of each strategic option.
3. Identify where internal capability supports or limits external exploitation.
4. Prioritize initiatives with best strategic fit and readiness.

**Instructional Guidance and Constraints**
- Apply **TOWS Matrix**, **PESTLE Cross-Impact Mapping**, and **Strategic Option Evaluation Matrix**.
- Link insights from 7S internal alignment to external market realities.
- Highlight where misalignment or risk exists.
- Maintain crisp, board-level tone.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – Integrated internal–external perspective.
2. **Capability–Opportunity Alignment Map.**
3. **Strategic Option Evaluation Matrix.**
4. **Risk–Readiness Assessment.**
5. **Prioritized Strategic Initiatives.**
6. **Implementation Feasibility and Governance Recommendations.**
7. **Integrated Dashboard Design (optional).**

**Any Additional Notes on Scope or Limitations**
- Avoid excessive repetition of SWOT/PESTLE details—synthesize instead.
- Focus on actionable integration.
- Keep tone confident, succinct, and senior-level.
""",

"PROMPT_15_EXEC_SUMMARY": """
**Role Definition**
You are a **Senior Partner Consultant** preparing an **executive summary** of a full 7S assessment for the C-suite. Your goal is to synthesize findings into concise, actionable insights suitable for decision-making.

**Contextual Information**
Inputs:
- Full 7S analysis results
- Strategic context and urgency
- Budget and resource constraints
- Board or executive priorities
- Key stakeholder expectations and success criteria
You must condense the full report into a 2–3-page executive deliverable.

**Task Description and Goals**
1. Summarize critical findings and insights from all 7 elements.
2. Identify top strategic imperatives and investment areas.
3. Distinguish between short-term wins and long-term transformations.
4. Present ROI rationale, risk profile, and next steps clearly.

**Instructional Guidance and Constraints**
- Use **Pyramid Principle** (conclusion first, supporting logic after).
- Quantify impact where data allows.
- Prioritize clarity and brevity; every sentence must add value.
- Tone: professional, confident, and insight-driven.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary (2–3 pages)** – Key insights and context.
2. **Visual Alignment Dashboard** – Optional table or chart showing 7S scores.
3. **Top 10 Recommendations Ranked by Impact and Feasibility.**
4. **Investment and Resource Requirements Summary.**
5. **90-Day, 6-Month, and 1-Year Action Plans.**
6. **Success Metrics and Governance Model.**
7. **Key Risks and Mitigation Strategies.**

**Any Additional Notes on Scope or Limitations**
- Avoid granular details already in technical appendices.
- Write for time-pressed senior executives.
- Maintain polished, high-credibility tone.
""",

"MEGA_PROMPT": """
**Role Definition**
You are a **World-Renowned Management Consultant** (WRC) delivering a **comprehensive 7S organizational assessment**. Act as lead author of a multi-section consulting report designed for executive decision-making and transformation planning.

**Contextual Information**
Use all prior contextual information provided, including:
- Company profile, size, revenue, and market position
- Strategic challenges and recent initiatives
- Internal assessments per S element (Strategy, Structure, Systems, Shared Values, Style, Staff, Skills)
Where information is missing, infer logical assumptions based on industry norms.

**Task Description and Goals**
1. Produce a full-length, integrated report analyzing all 7S elements.
2. Identify misalignments and their systemic impacts.
3. Propose targeted interventions with ROI justification.
4. Deliver executive-level narrative suitable for board presentation.

**Instructional Guidance and Constraints**
- Apply professional consulting frameworks: McKinsey 7S, Balanced Scorecard, Kotter’s Change Model.
- Think step-by-step; provide evidence before conclusions.
- Use structured Markdown headings and subheadings.
- Maintain a cohesive story from diagnosis → insight → recommendation.
- Avoid hallucination: infer only with logical consistency.

**Expected Output Format and Examples (if applicable)**
1. **Executive Summary** – 2–3 pages summarizing major findings and recommendations.
2. **Detailed 7S Analysis** – Each S in its own section (2–3 paragraphs minimum each).
3. **Interdependencies Overview** – Narrative of reinforcing/conflicting dynamics.
4. **Strategic Recommendations** – Prioritized by impact and feasibility.
5. **Implementation Plan** – Phased actions with responsible functions.
6. **Governance & Metrics** – Success tracking model.
7. **Appendices** – Tables, charts, and optional data references.

**Any Additional Notes on Scope or Limitations**
- Do not include fictional data or financial figures.
- Maintain neutral, evidence-based judgment.
- Style: formal yet conversational, resembling a McKinsey final report.
"""
})

# Map bracketed placeholders to ORG fields
PLACEHOLDER_MAP = {
    "[NAME]": ORG.get("name",""),
    "[INDUSTRY]": ORG.get("industry",""),
    "[NUMBER]": ORG.get("size_employees",""),
    "[REVENUE]": ORG.get("annual_revenue",""),
    "[LOCATIONS]": ORG.get("locations",""),
    "[BRIEF DESCRIPTION]": ORG.get("current_context",""),
    "[LIST 3-5]": ORG.get("key_challenges",""),
    "[DESCRIBE]": ORG.get("recent_changes",""),
    "[PROVIDE]": ORG.get("strategy_statement",""),
    "[LIST]": ORG.get("strategic_goals",""),
    "[STATEMENT]": ORG.get("value_proposition",""),
    "[DESCRIBE HIERARCHY]": ORG.get("org_chart",""),
    "[NUMBERS]": ORG.get("span_of_control",""),
    "[IF AVAILABLE]": "",
    "[AVERAGES]": "",
    "[BREAKDOWN]": "",
    "[DATA]": "",
    "[POSITION VS MARKET]": "",
    "[ASSESS 1-5]": "",
    "[CURRENT STATE]": "",
    "[ASSESS]": "",
    "[SUMMARIZE]": "",
    "[ANALYZE]": "",
    "[IDENTIFY]": "",
    "[SPECIFY]": ""
}

def materialize_prompt(key: str) -> str:
    text = PROMPTS[key]
    for k,v in PLACEHOLDER_MAP.items():
        text = text.replace(k, str(v if v else f"N/A: {k}"))
    return text


In [16]:
# ===============================================================
# 🧠 OLLAMA CHAT HELPERS — Auto-detects generate/chat endpoint
# ===============================================================
import requests, json, time
from typing import Dict

TEMPERATURE, TOP_P, REPEAT_PENALTY, TIMEOUT_S = 0.4, 0.9, 1.1, 860

# ---------------------------------------------------------------
# 1️⃣ Detect endpoint availability and set USE_GENERATE
# ---------------------------------------------------------------
BASE = "http://127.0.0.1:11434"
GEN  = f"{BASE}/api/generate"
CHAT = f"{BASE}/api/chat"

def detect_generate_support():
    try:
        r = requests.options(GEN, timeout=3)
        if r.status_code in (200, 204):
            return True
    except Exception:
        pass
    return False

USE_GENERATE = detect_generate_support()
print(f"⚙️ Using {'/api/generate' if USE_GENERATE else '/api/chat'} endpoint for inference.")

# ---------------------------------------------------------------
# 2️⃣ Ask LLM — general wrapper
# ---------------------------------------------------------------
def ask_llm(model: str, sys: str, user: str) -> str:
    """Send message to Ollama and return text output."""
    try:
        if USE_GENERATE:
            # Simpler, faster API (no chat format)
            prompt = f"<<SYS>>\n{sys}\n<</SYS>>\n\n{user}"
            r = requests.post(
                GEN,
                json={
                    "model": model,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": TEMPERATURE,
                        "top_p": TOP_P,
                        "repeat_penalty": REPEAT_PENALTY
                    },
                },
                timeout=TIMEOUT_S,
            )
            r.raise_for_status()
            return (r.json().get("response") or "").strip()
        else:
            # Chat-style API
            r = requests.post(
                CHAT,
                json={
                    "model": model,
                    "messages": [
                        {"role": "system", "content": sys},
                        {"role": "user", "content": user},
                    ],
                    "stream": False,
                    "options": {
                        "temperature": TEMPERATURE,
                        "top_p": TOP_P,
                        "repeat_penalty": REPEAT_PENALTY,
                    },
                },
                timeout=TIMEOUT_S,
            )
            r.raise_for_status()
            return ((r.json().get("message") or {}).get("content") or "").strip()

    except requests.exceptions.Timeout:
        return "⚠️ Request timed out — model took too long."
    except Exception as e:
        return f"❌ Error querying model {model}: {e}"

# ---------------------------------------------------------------
# 3️⃣ Multi-model consensus synthesis
# ---------------------------------------------------------------
def synthesize_consensus(primary_model: str, prompt_name: str, per_model: Dict[str, str]) -> str:
    """Combine responses from multiple models into one cohesive expert synthesis."""
    sys = (
        "You are a senior WRC consultant. "
        "Synthesize multiple expert drafts into one coherent, non-redundant, executive-ready report."
    )
    user = f"""Prompt: {prompt_name}

Combine the following {len(per_model)} model answers into one structured response.
Do NOT mention that this is a synthesis. Present it like a polished consultant report.

Guidelines:
- Keep the best points from each.
- Merge ideas logically.
- If answers conflict, show trade-offs and state your recommendation.
- Use concise sections and bulleted clarity.

=== MODEL ANSWERS ===
{json.dumps(per_model, indent=2, ensure_ascii=False)}
"""
    return ask_llm(primary_model, sys, user)


⚙️ Using /api/chat endpoint for inference.


In [17]:
# ---------------------------
# 5) RUN SELECTED PROMPTS ACROSS MODELS
# ---------------------------
def run_prompt_across_models(prompt_key: str, models: List[str]) -> Dict[str,str]:
    sys = "You are precise and to-the-point."
    usr = materialize_prompt(prompt_key)
    out = {}
    for m in models:
        try:
            txt = ask_llm(m, sys, usr)
            out[m] = txt
            print(f"  ✓ {prompt_key} via {m}: {len(txt)} chars")
        except Exception as e:
            out[m] = f"(error from {m}: {e})"
            print(f"  ✗ {prompt_key} via {m}: {e}")
    return out

results = {}
consensus = {}
print("\n🧠 Running prompts:", ", ".join(RUN_PROMPTS))
for key in RUN_PROMPTS:
    per_model = run_prompt_across_models(key, REQUESTED_MODELS)
    results[key] = per_model
    consensus[key] = synthesize_consensus(PRIMARY_MODEL, key, per_model)





🧠 Running prompts: PROMPT_1_FULL_7S, PROMPT_2_STRATEGY, PROMPT_3_STRUCTURE, PROMPT_4_SYSTEMS, PROMPT_5_SHARED_VALUES, PROMPT_6_SKILLS, PROMPT_7_STYLE, PROMPT_8_STAFF, PROMPT_9_ALIGNMENT, PROMPT_10_CHANGE, PROMPT_11_DIGITAL, PROMPT_12_BENCH, PROMPT_13_GAPS, PROMPT_14_INTEGRATION, PROMPT_15_EXEC_SUMMARY, MEGA_PROMPT
  ✓ PROMPT_1_FULL_7S via gpt-oss: 922 chars
  ✓ PROMPT_1_FULL_7S via mistral: 6725 chars
  ✓ PROMPT_1_FULL_7S via llama3: 1777 chars
  ✓ PROMPT_2_STRATEGY via gpt-oss: 1070 chars
  ✓ PROMPT_2_STRATEGY via mistral: 2879 chars
  ✓ PROMPT_2_STRATEGY via llama3: 1845 chars
  ✓ PROMPT_3_STRUCTURE via gpt-oss: 699 chars
  ✓ PROMPT_3_STRUCTURE via mistral: 3968 chars
  ✓ PROMPT_3_STRUCTURE via llama3: 2564 chars
  ✓ PROMPT_4_SYSTEMS via gpt-oss: 6576 chars
  ✓ PROMPT_4_SYSTEMS via mistral: 2476 chars
  ✓ PROMPT_4_SYSTEMS via llama3: 2763 chars
  ✓ PROMPT_5_SHARED_VALUES via gpt-oss: 4230 chars
  ✓ PROMPT_5_SHARED_VALUES via mistral: 3150 chars
  ✓ PROMPT_5_SHARED_VALUES via llama3:

In [18]:
# ---------------------------
# 6) SAVE OUTPUTS
# ---------------------------
bundle = {
    "metadata": {
        "generated_at_utc": datetime.datetime.now(datetime.timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ"),
        "ollama_host": BASE,
        "models": REQUESTED_MODELS,
        "primary_model": PRIMARY_MODEL
    },
    "org_inputs": ORG,
    "raw_per_prompt": results,
    "consensus_per_prompt": consensus
}

with open(BUNDLE_JSON, "w", encoding="utf-8") as f:
    json.dump(bundle, f, indent=2, ensure_ascii=False)

with open(REPORT_MD, "w", encoding="utf-8") as f:
    f.write(f"# 7S Report — Consensus (models: {', '.join(REQUESTED_MODELS)})\n\n")
    f.write("## Organization\n")
    f.write("```json\n" + json.dumps(ORG, indent=2, ensure_ascii=False) + "\n```\n\n")
    for key in RUN_PROMPTS:
        f.write(f"## {key}\n\n")
        f.write(consensus.get(key,"").strip() + "\n\n")
        f.write("<details><summary>Model answers</summary>\n\n")
        f.write("```json\n" + json.dumps(results.get(key,{}), indent=2, ensure_ascii=False) + "\n```\n")
        f.write("</details>\n\n")

print("\n✅ Done.")
print("JSON:", BUNDLE_JSON)
print("MD  :", REPORT_MD)
# ==========================================================================================


✅ Done.
JSON: /content/7S_bundle.json
MD  : /content/7S_report.md


In [19]:
# ======================= DISPLAY 7S REPORT IN-CELL =======================
import os, json, textwrap
from IPython.display import display, Markdown, HTML

OUT_DIR = "/content"
BUNDLE_JSON = os.path.join(OUT_DIR, "7S_bundle.json")
REPORT_MD   = os.path.join(OUT_DIR, "7S_report.md")

def _h2(s):
    return f"\n## {s}\n"
def _h3(s):
    return f"\n### {s}\n"
def _codeblock(label, obj):
    if isinstance(obj, str):
        body = obj
    else:
        body = json.dumps(obj, indent=2, ensure_ascii=False)
    return f"\n**{label}**\n\n```json\n{body}\n```\n"

# 1) If the Markdown report exists, render that directly (richest output)
if os.path.exists(REPORT_MD):
    with open(REPORT_MD, "r", encoding="utf-8") as f:
        md = f.read()
    display(Markdown(md))
else:
    # 2) Otherwise, pretty-print from the JSON bundle
    if not os.path.exists(BUNDLE_JSON):
        raise FileNotFoundError("No report found. Run the main cell first to create /content/7S_bundle.json or /content/7S_report.md.")

    with open(BUNDLE_JSON, "r", encoding="utf-8") as f:
        bundle = json.load(f)

    meta     = bundle.get("metadata", {})
    org      = bundle.get("org_inputs", {})
    consensus = bundle.get("consensus_per_prompt", {})
    raw       = bundle.get("raw_per_prompt", {})

    # Build a friendly Markdown view
    parts = []
    title = f"# 7S Report — Consensus (models: {', '.join(meta.get('models', []))})"
    parts.append(title)
    parts.append(_h2("Organization"))
    parts.append("```json\n" + json.dumps(org, indent=2, ensure_ascii=False) + "\n```")

    # Show consensus outputs section-by-section
    for k in consensus.keys():
        parts.append(_h2(k))
        txt = consensus.get(k, "").strip()
        if not txt:
            parts.append("_No consensus text produced for this prompt._")
        else:
            # Light formatting: ensure lines not too long for the notebook width
            wrapped = "\n".join(textwrap.fill(line, width=100) for line in txt.splitlines())
            parts.append(wrapped)

        # Collapsible raw model answers
        parts.append("\n<details><summary>Model answers</summary>\n\n")
        parts.append("```json\n" + json.dumps(raw.get(k, {}), indent=2, ensure_ascii=False) + "\n```")
        parts.append("\n</details>\n")

    display(Markdown("\n".join(parts)))
# ========================================================================


# 7S Report — Consensus (models: gpt-oss, mistral, llama3)

## Organization
```json
{
  "name": "QA Automation Org",
  "industry": "Software / DevTools",
  "size_employees": 180,
  "annual_revenue": "N/A (cost center)",
  "locations": "Toronto; Remote",
  "current_context": "Transitioning from phased QA to DevSecOps; aiming for predictive quality and CI/CD-first culture.",
  "key_challenges": "Inconsistent test coverage; siloed defect data; unclear decision rights; skills gap in pipeline engineering.",
  "recent_changes": "Trunk-based dev in 2 squads; feature flags/canary pilots; Playwright + contract tests on 2 critical APIs.",
  "strategy_statement": "Ship faster with confidence via automated quality gates and proactive risk analytics.",
  "strategic_goals": "12 months: 70% automated coverage; 24m: 95% critical path; MTTR<1hr; DORA elite.",
  "kpis": "DORA metrics, escaped defect rate, mean time to detect, e2e reliability SLOs.",
  "market_position": "Internal enablement platform within a large enterprise.",
  "competitive_advantages": "Domain expertise; test data virtualization; unified telemetry.",
  "target_segments": "Product teams building microservices/APIs.",
  "value_proposition": "Fewer incidents, faster releases, higher trust in automation.",
  "org_chart": "VP Eng → Dir QA/Platform → QA Enablement, SDET, Observability pods; squads map to product lines.",
  "layers": "VP → Director → Managers → ICs (4 layers)",
  "span_of_control": "Managers: 6-8 ICs avg",
  "decision_rights": "Product owns scope; QA owns quality standards; Dev owns implementation; shared release gates.",
  "xfunc": "Release Council; Reliability Guild; Architecture Forum.",
  "geo": "Toronto hub + distributed remote",
  "restructuring": "Evolving towards platform team model; decoupling test infra from app squads."
}
```

## PROMPT_1_FULL_7S

**7S Organizational Analysis Report**  
*Prepared for: [Client Name]*  
*Prepared by: Senior WRC Consultant*  
*Date: 25 Oct 2025*  

---

## 1. Request for Information  
To tailor the analysis to your organization, please provide the following:

| Item | Details Needed |
|------|----------------|
| **Company Overview** | Name, industry, core business lines, employee count, revenue, market share |
| **Geographic Footprint** | HQ location, key markets (regions/countries) |
| **Current Business Environment** | Major external drivers (regulatory, tech, competitive), key pain points |
| **Recent Strategic Initiatives** | Mergers, divestitures, digital transformation, new product launches, restructuring |
| **Specific Concerns** | Talent retention, process inefficiencies, cultural issues, etc. |

---

## 2. Executive Summary  
*Assuming the organization is similar to a large tech firm (e.g., XYZ Corp.)*  

| Element | Alignment Rating (1‑10) | Key Finding | Top Recommendation |
|---------|------------------------|-------------|--------------------|
| **Strategy** | 8 | Strong market focus but limited long‑term foresight | Enhance scenario planning & contingency frameworks |
| **Structure** | 5 | Complex, multi‑layered, redundant functions | Consolidate layers, streamline decision paths |
| **Systems** | 6 | Fragmented IT, data silos | Deploy a unified, cloud‑based platform |
| **Shared Values** | 8 | High innovation ethos but unclear articulation | Launch a values‑reinforcement training program |
| **Style** | 5 | Mixed leadership styles, inconsistent accountability | Define a unified leadership competency model |
| **Staff** | 7 | Skilled but diversity & emerging‑tech gaps | Implement targeted talent acquisition & up‑skilling |
| **Skills** | 7 | Strong core skills, weak in emerging tech | Expand technical training & cross‑functional labs |

**Top 5 Recommendations (High Impact / High Feasibility)**  

1. **Structure Consolidation** – Reduce layers, eliminate redundancies.  
2. **Unified IT System** – Standardize data architecture, improve collaboration.  
3. **Leadership Style Alignment** – Codify management practices across org.  
4. **Talent & Diversity Strategy** – Recruit diverse skill sets, focus on emerging tech.  
5. **Performance‑KPI Alignment** – Ensure KPIs reflect strategic priorities.

---

## 3. Detailed 7S Assessment  

| S | Definition & Relevance | Current State | Strengths | Weaknesses | Alignment Rating | Recommended Improvements |
|---|------------------------|---------------|-----------|------------|------------------|--------------------------|
| **Strategy** | Vision, mission, competitive positioning | Clear direction, but reactive to market shifts | Market‑aligned initiatives | Limited long‑term planning | 8 | Build scenario‑based roadmap, embed risk‑management |
| **Structure** | Organizational design, reporting lines | Multi‑layered, siloed units | Clear functional expertise | Slow decision‑making, redundancy | 5 | Flatten hierarchy, create cross‑functional squads |
| **Systems** | Processes, technology, data flow | Disparate legacy systems, data silos | Robust security, compliance | Inefficient collaboration | 6 | Adopt integrated cloud platform, data governance |
| **Shared Values** | Core beliefs, culture | Strong innovation focus | Employee passion | Values not formally communicated | 8 | Develop values curriculum, embed in onboarding |
| **Style** | Leadership approach, decision style | Varied, inconsistent accountability | Empowered teams | Inconsistent decision quality | 5 | Define leadership competency framework, train managers |
| **Staff** | Human capital, talent mix | Highly skilled, low diversity | Continuous learning culture | Skill gaps in emerging tech | 7 | Targeted hiring, diversity initiatives, up‑skilling |
| **Skills** | Competencies required | Strong core tech skills | Innovation labs | Limited emerging‑tech exposure | 7 | Expand training, partner with universities, internal hackathons |

---

## 4. Interdependencies Analysis  

| Relationship | Impact | Recommendation |
|--------------|--------|----------------|
| **Strategy ↔ Staff** | Strategy dictates needed skills; staff shape strategy | Align talent acquisition with strategic priorities |
| **Structure ↔ Systems ↔ Skills** | Structure influences system choice; systems shape skill requirements | Standardize systems to support streamlined structure and skill development |
| **Style ↔ Shared Values** | Leadership style reinforces values | Embed values into leadership training |
| **Staff ↔ Skills** | Staff capabilities determine skill gaps | Continuous learning programs to close gaps |

---

## 5. Top Misalignments & Root Causes  

| Misalignment | Root Cause | Trade‑off Considerations |
|--------------|------------|--------------------------|
| **Complex Structure** | Historical growth & acquisitions | Flattening may reduce local autonomy |
| **Data Silos** | Lack of IT standardization | Unified platform requires change management |
| **Inconsistent Leadership** | Organic growth, no formal style guidelines | Standardization may stifle local initiative |
| **Limited Diversity** | Legacy hiring practices | Diversity initiatives may need cultural change |
| **Emerging‑Tech Skill Gaps** | Rapid tech evolution | Upskilling may divert resources from core ops |

**Recommendation** – Prioritize structure and systems alignment first; these unlock downstream improvements in style, staff, and skills.

---

## 6. Recommendations Roadmap  

| Priority | Initiative | Impact | Feasibility | Key Actions |
|----------|------------|--------|-------------|-------------|
| **High** | Structure Consolidation | High | High | Map redundancies → Redesign org chart → Communicate changes |
| **High** | Unified IT Platform | High | Medium | Vendor selection → Pilot in one region → Scale |
| **High** | Leadership Style Framework | High | High | Define competencies → Train managers → Embed in performance reviews |
| **High** | Talent & Diversity Strategy | High | High | Workforce audit → Targeted hiring → Inclusion programs |
| **Medium** | Scenario‑Based Strategic Planning | Medium | High | Workshops → Risk matrix → Integrate into annual planning |
| **Medium** | Values Reinforcement Program | Medium | High | Curriculum design → Onboarding modules → Leadership endorsement |
| **Low** | Diversity Metrics & Reporting | Low | High | Data collection → Dashboard → Continuous improvement |

---

## 7. Success Metrics  

| Metric | Target | Measurement Frequency |
|--------|--------|-----------------------|
| **Agility Index** | 30 % faster response to market changes | Quarterly |
| **Data Silos Reduction** | 80 % cross‑department data sharing | Semi‑annual |
| **Leadership Consistency Score** | 90 % alignment with competency framework | Annual |
| **Diversity Representation** | 40 % under‑represented groups in workforce | Annual |
| **Skill Gap Closure** | 70 % of emerging‑tech skill needs met | Semi‑annual |
| **KPI Alignment Rate** | 95 % of KPIs linked to strategy | Annual |

---

**Next Steps**  
1. **Kick‑off Workshop** – Gather the requested information and validate assumptions.  
2. **Data Collection** – Conduct surveys, interviews, and system audits.  
3. **Detailed Analysis** – Populate the template with organization‑specific data.  
4. **Action Plan Development** – Translate findings into a phased implementation roadmap.  
5. **Governance Setup** – Establish steering committee, change champions, and monitoring cadence.

*Prepared with the intent to deliver a clear, actionable 7S alignment framework that drives strategic execution and sustainable performance.*

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "I’m ready to dive into the 7S analysis. To ensure the assessment is accurate and actionable, could you please provide the following details about the organization?\n\n1. **Company Overview**  \n   - Name, industry, and core business lines  \n   - Size (employees, revenue, market share)  \n\n2. **Geographic Footprint**  \n   - Headquarters location  \n   - Key markets (regions/countries)  \n\n3. **Current Business Environment**  \n   - Major external drivers (regulatory, technological, competitive)  \n   - Key challenges or pain points  \n\n4. **Recent Strategic Initiatives**  \n   - Any major changes (mergers, divestitures, digital transformation, new product launches, restructuring)  \n\n5. **Any Specific Areas of Concern**  \n   - For example, talent retention, process inefficiencies, cultural issues, etc.  \n\nOnce I have this information, I’ll craft a structured, data‑driven 7S report tailored to your organization’s context.",
  "mistral": "**Company Overview**\nThe organization in question is XYZ Corporation, a leading player in the technology industry with a global footprint, employing approximately 10,000 individuals and generating an annual revenue of $5 billion. The company operates in a dynamic business environment characterized by rapid technological advancements, intense competition, and evolving customer expectations. Recently, XYZ Corporation has undergone several strategic initiatives to adapt to these challenges.\n\n**Executive Summary**\nIn our 7S Organizational Analysis, we found that XYZ Corporation exhibits a mixed alignment across the seven S elements. The company demonstrates strong Strategy, Shared Values, and Skills, but faces misalignments in Structure, Systems, Style, and Staff. Below are the top 5 recommendations to address these issues:\n\n1. **Consolidate redundant structures**: Streamline organizational structure to eliminate redundancies and improve efficiency (High Impact, High Feasibility).\n2. **Implement a unified IT system**: Standardize technology infrastructure to enhance data flow and collaboration (High Impact, Medium Feasibility).\n3. **Define clear leadership styles**: Establish consistent management practices across the organization (Medium Impact, High Feasibility).\n4. **Develop comprehensive talent strategies**: Address skills gaps and foster a culture of continuous learning (Medium Impact, High Feasibility).\n5. **Align performance metrics with strategic objectives**: Ensure that key performance indicators (KPIs) support the company's overall strategy (Low Impact, High Feasibility).\n\n**Detailed 7S Assessment**\n\n1. **Strategy**\n   - Definition & relevance: Clear direction and competitive advantage.\n   - Current state: Strong alignment with market trends and customer needs.\n   - Strengths & weaknesses: Strong strategy, but limited focus on long-term goals and contingency planning.\n   - Alignment rating: 8/10\n   - Recommended improvements: Enhance strategic foresight and scenario planning.\n\n2. **Structure**\n   - Definition & relevance: Organizational design to support business objectives.\n   - Current state: Complex structure with multiple layers and redundant functions.\n   - Strengths & weaknesses: Efficient division of labor but lacks agility and responsiveness.\n   - Alignment rating: 5/10\n   - Recommended improvements: Consolidate redundant structures and streamline decision-making processes.\n\n3. **Systems**\n   - Definition & relevance: Processes, tools, and infrastructure to support operations.\n   - Current state: Inefficient technology systems and data silos.\n   - Strengths & weaknesses: Limited innovation but strong security measures in place.\n   - Alignment rating: 6/10\n   - Recommended improvements: Implement a unified IT system to improve data flow and collaboration.\n\n4. **Shared Values**\n   - Definition & relevance: Common beliefs and principles that guide behavior.\n   - Current state: Strong commitment to innovation, customer satisfaction, and ethical conduct.\n   - Strengths & weaknesses: Employees are passionate about the company's mission but lack a clear understanding of shared values.\n   - Alignment rating: 8/10\n   - Recommended improvements: Develop a comprehensive employee training program to reinforce shared values.\n\n5. **Style**\n   - Definition & relevance: Leadership approach that encourages desired behavior.\n   - Current state: Varying leadership styles across the organization.\n   - Strengths & weaknesses: Empowered employees but inconsistent decision-making and accountability.\n   - Alignment rating: 5/10\n   - Recommended improvements: Define clear leadership styles to establish consistency and promote accountability.\n\n6. **Staff**\n   - Definition & relevance: Human resources, including skills, knowledge, and experience.\n   - Current state: Highly skilled workforce with a strong focus on continuous learning.\n   - Strengths & weaknesses: Limited diversity and potential skills gaps in emerging technologies.\n   - Alignment rating: 7/10\n   - Recommended improvements: Develop comprehensive talent strategies to address diversity and emerging technology needs.\n\n7. **Skills**\n   - Definition & relevance: Competencies required to execute the strategy effectively.\n   - Current state: Strong focus on innovation, customer satisfaction, and technological expertise.\n   - Strengths & weaknesses: Highly skilled workforce but limited training in emerging technologies.\n   - Alignment rating: 7/10\n   - Recommended improvements: Enhance employee training programs to address skills gaps in emerging technologies.\n\n**Interdependencies Analysis**\n- Strategy and Staff are closely related as the company's strategy determines the required skills, and vice versa.\n- Structure, Systems, and Skills are interconnected as the organizational structure influences the choice of technology systems and the skills needed to operate them.\n\n**Top Misalignments & Root Causes**\n1. **Inefficient structure**: Complexity hinders agility and responsiveness (Root cause: Historical growth and acquisitions).\n2. **Data silos**: Inefficient technology systems prevent effective collaboration (Root cause: Lack of standardization in IT infrastructure).\n3. **Varying leadership styles**: Inconsistent decision-making and accountability (Root cause: Organic growth and lack of formalized management practices).\n4. **Limited diversity**: Potential impact on innovation and customer satisfaction (Root cause: Historical hiring practices).\n5. **Skills gaps in emerging technologies**: Risk of falling behind competitors (Root cause: Rapid technological advancements).\n\n**Recommendations Roadmap**\n1. High Impact & High Feasibility: Consolidate redundant structures, implement a unified IT system, define clear leadership styles, develop comprehensive talent strategies, and align performance metrics with strategic objectives.\n2. Medium Impact & High Feasibility: Enhance strategic foresight and scenario planning, develop a comprehensive employee training program to reinforce shared values, and address skills gaps in emerging technologies.\n3. Low Impact & High Feasibility: Encourage diversity in hiring practices.\n\n**Success Metrics**\n1. Improvement in organizational agility and responsiveness (measured by response time to market changes).\n2. Reduction in data silos and improvement in collaboration (measured by cross-departmental communication metrics).\n3. Consistency in leadership styles and decision-making processes (measured by leadership effectiveness surveys).\n4. Diversity representation within the workforce (measured by demographic data analysis).\n5. Skills development and retention rates (measured by employee training participation and turnover rates).",
  "llama3": "I'm ready to conduct a comprehensive 7S Organizational Analysis as a Senior Management Consultant. Please provide the contextual information about the organization, including company name, industry, size, revenue, geographic footprint, current business environment, key challenges, and recent changes or strategic initiatives. I will use this information to craft a formal report assessing alignment across all seven McKinsey 7S elements: Strategy, Structure, Systems, Shared Values, Style, Staff, and Skills.\n\nI understand the task description and goals, as well as the instructional guidance and constraints. I will think step-by-step, identify insights before forming conclusions, use facts provided, maintain a balanced tone, avoid speculative financial or personnel details, organize findings under consistent headings, and keep paragraphs concise with bullet points where appropriate.\n\nMy expected output format includes:\n\n1. **Executive Summary**: A 1-2 page summary summarizing alignment level, key findings, and top 5 recommendations.\n2. **Detailed 7S Assessment**: For each S, I will include:\n\t* Definition & relevance\n\t* Current state\n\t* Strengths & weaknesses\n\t* Alignment rating (1-10)\n\t* Recommended improvements\n3. **Interdependencies Analysis**: Identification of reinforcing/conflicting relationships among elements.\n4. **Top Misalignments & Root Causes**: A list of the top 5 issues with rationale.\n5. **Recommendations Roadmap**: Prioritization by impact and feasibility (High/Medium/Low).\n6. **Success Metrics**: Definition of KPIs to monitor post-implementation success.\n\nI will not fabricate numerical data, maintain a realistic scope, avoid over-engineering, and keep the tone factual, professional, and suitable for senior executives. Let's get started!"
}
```
</details>

## PROMPT_2_STRATEGY

**Strategic Assessment Report – Executive‑Level Summary**  
*Prepared for: [Client Name]*  
*Date: 25 Oct 2025*  

---

### 1. Executive Summary  
- **Current Position:** The organization has a clear vision and a focused target‑segment strategy, but the articulation of goals, KPIs, and a detailed roadmap remains incomplete.  
- **Key Findings:**  
  - **Strategic Clarity:** Vision/mission are strong; objectives lack measurable milestones.  
  - **Market Fit:** Competitive advantages exist, yet the value proposition is not fully differentiated against peers.  
  - **Execution Alignment:** Resource allocation is adequate, but internal communication gaps risk mis‑execution.  
  - **Gaps & Risks:** Capacity constraints and a rapidly evolving competitive landscape threaten scalability and relevance.  
- **Recommendation Snapshot:**  
  1. Build a measurable, milestone‑driven roadmap.  
  2. Re‑engineer the value proposition using the Value‑Proposition Canvas.  
  3. Deploy a cross‑functional strategy‑communication program.  
  4. Scale resources to meet demand and embed continuous market‑trend monitoring.  
  5. Establish a governance board to oversee execution and risk mitigation.

---

### 2. Input Requirements  
To refine the assessment and deliver a tailored action plan, please provide:

| Item | What to Include |
|------|-----------------|
| **Current Strategy Statement** | Concise “what” and “why” narrative. |
| **Strategic Goals (3–5 yrs)** | Specific, measurable targets (revenue, market share, ESG, etc.). |
| **KPIs** | Current metrics and target values. |
| **Market Position** | Share, growth trend, geographic footprint. |
| **Competitive Advantages** | Core strengths (technology, brand, cost structure). |
| **Target Customer Segments** | Demographics, psychographics, buying behavior. |
| **Value Proposition** | Unique benefit mix promised to segments. |

---

### 3. Strategic Clarity Analysis  
- **Strengths**  
  - Vision and mission are articulated and align with core values.  
  - Target customer focus is evident.  
- **Weaknesses**  
  - Goals lack explicit, time‑bound milestones.  
  - KPI targets are generic; no clear ownership or accountability.  
- **Trade‑off**  
  - *Depth vs. Agility*: A highly detailed roadmap improves clarity but may slow iterative adjustments. **Recommendation:** Adopt a rolling‑wave planning cadence—set quarterly milestones within a 3‑year horizon.

---

### 4. Market Fit & Differentiation Review  
- **Competitive Landscape** (Porter’s Five Forces)  
  - *Threat of New Entrants:* Moderate – high capital required.  
  - *Bargaining Power of Buyers:* High – price sensitivity in target segments.  
  - *Threat of Substitutes:* Rising – digital alternatives.  
  - *Supplier Power:* Low – diversified supply chain.  
  - *Industry Rivalry:* Intense – several incumbents with overlapping value propositions.  
- **Value Proposition Canvas Findings**  
  - *Current Gap:* Benefits are generic; pain points of target segments are under‑addressed.  
  - *Opportunity:* Leverage unique technology or service model to create a defensible moat.  
- **Trade‑off**  
  - *Differentiation vs. Cost Leadership*: Enhancing differentiation may raise costs; maintaining cost leadership could erode perceived value. **Recommendation:** Pursue a hybrid model—differentiate on high‑touch service while optimizing back‑office efficiencies.

---

### 5. Execution Alignment Score  
| Dimension | Score (1–10) | Rationale |
|-----------|--------------|-----------|
| **Resource Allocation** | 7 | Adequate but not fully aligned with strategic priorities. |
| **Internal Communication** | 6 | Strategy is communicated, yet understanding across levels is uneven. |
| **Capability Fit** | 5 | Current capabilities lag behind aspirational goals. |
| **Overall Alignment** | **6** | Indicates moderate alignment; improvement needed. |

---

### 6. Strategic Gaps & Risks  
| Gap / Risk | Impact | Likelihood | Suggested Mitigation |
|------------|--------|------------|----------------------|
| **Capacity Constraints** | High | Medium | Scale operations via modular deployment and strategic partnerships. |
| **Rapid Competitive Evolution** | High | High | Institute a market‑trend monitoring team and agile response protocols. |
| **Unclear KPI Ownership** | Medium | High | Assign KPI owners, embed in performance reviews, and use dashboards. |
| **Insufficient Differentiation** | Medium | Medium | Re‑engineer value proposition; test with customer insights. |
| **Communication Silos** | Medium | Medium | Launch cross‑functional strategy workshops and a centralized knowledge hub. |

---

### 7. Recommendations & Communication Plan  
1. **Develop a Milestone‑Driven Roadmap**  
   - *Action:* Create a 3‑year plan with quarterly milestones, KPI targets, and ownership matrices.  
   - *Owner:* Strategy Office + Finance.  
2. **Re‑engineer the Value Proposition**  
   - *Action:* Apply the Value‑Proposition Canvas; validate with 30+ customer interviews.  
   - *Owner:* Marketing + Product Development.  
3. **Launch a Cross‑Functional Strategy Communication Program**  
   - *Action:* Quarterly town‑halls, a digital strategy portal, and role‑specific playbooks.  
   - *Owner:* HR & Communications.  
4. **Scale Resources to Meet Demand**  
   - *Action:* Identify critical bottlenecks; invest in automation, talent acquisition, and partner ecosystems.  
   - *Owner:* Operations + IT.  
5. **Establish a Strategy Governance Board**  
   - *Action:* Monthly reviews of progress, risk updates, and resource re‑allocation.  
   - *Owner:* Executive Leadership Team.  

**Communication Cadence**  
- **Monthly:** Executive briefings (slide deck + KPI dashboard).  
- **Quarterly:** Full‑team strategy workshops (interactive sessions).  
- **Annual:** Strategy review & recalibration with external stakeholders (investors, partners).

---

**Next Steps**  
1. Submit the required inputs (Section 2).  
2. Schedule a kickoff workshop to align on objectives and timelines.  
3. Deliver a detailed implementation playbook within 30 days of kickoff.

*Prepared by:*  
Senior WRC Consultant – [Your Name]  
[Consulting Firm]  
[Contact Information]

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "I’m ready to dive into the assessment, but I’ll need the key inputs first. Please provide:\n\n| Item | What to include |\n|------|-----------------|\n| **Current strategy statement** | The concise “what we’re doing” and “why we’re doing it” |\n| **Strategic goals (3–5 yrs)** | Specific, measurable targets (e.g., revenue, market share, cost, ESG) |\n| **KPIs** | Current and target metrics that track progress |\n| **Market position** | Current share, growth trend, geographic footprint |\n| **Competitive advantages** | Core strengths (e.g., technology, brand, cost structure) |\n| **Target customer segments** | Demographics, psychographics, buying behavior |\n| **Value proposition** | The unique benefit mix you promise to those segments |\n\nOnce I have that, I’ll produce:\n\n1. Executive Summary  \n2. Strategic Clarity Analysis  \n3. Market Fit & Differentiation Review  \n4. Execution Alignment Score (1–10)  \n5. Strategic Gaps & Risks  \n6. 5 prioritized, actionable recommendations + communication plan  \n\nFeel free to paste the information directly or attach a brief summary.",
  "mistral": "**Executive Summary:**\n\nThe organization's strategy demonstrates a clear focus on its target customer segments and value proposition, with strategic goals set for the next 3-5 years. However, there are areas of potential improvement to enhance strategic clarity, execution alignment, and competitive positioning.\n\n**Strategic Clarity Analysis:**\n\nThe organization's strategy statement is well articulated, outlining a vision and mission that align with its core values. However, some ambiguity exists in the goals and objectives, particularly as they relate to key performance indicators (KPIs) and their specific targets. A more detailed roadmap, complete with measurable milestones, would help ensure a shared understanding of success criteria among all stakeholders.\n\n**Market Fit and Differentiation Review:**\n\nThe organization's market position is strong, with competitive advantages that set it apart from its peers. However, the value proposition could be further differentiated to better appeal to target customer segments and create a more defensible competitive moat. Utilizing tools such as the Value Proposition Canvas may help identify opportunities for improvement in this area.\n\n**Execution Alignment Score (1–10): 7**\n\nResource allocation appears well-balanced, with sufficient investment in key strategic initiatives. Communication of the strategy has been effective, but there is room for improvement in ensuring that all levels of the organization understand their role in executing the strategy and achieving its goals.\n\n**Strategic Gaps and Risks:**\n\nA significant gap exists between the organization's aspirations and its current capabilities, particularly with regards to scaling operations to meet growing demand. Additionally, the competitive landscape is evolving rapidly, creating risks if the organization fails to adapt quickly enough.\n\n**Recommendations and Communication Plan:**\n\n1. Develop a detailed strategic roadmap with measurable milestones to ensure clarity around success criteria.\n2. Conduct a comprehensive review of the value proposition to identify opportunities for differentiation.\n3. Implement a training program to improve understanding of the strategy across all levels of the organization.\n4. Allocate additional resources to address capacity constraints and enable scalable growth.\n5. Establish a cross-functional team to monitor industry trends and proactively adapt to changes in the competitive landscape.\n\nIn conclusion, while the organization's strategy demonstrates a clear focus on its target customer segments and value proposition, there are areas for improvement to enhance strategic clarity, execution alignment, and competitive positioning. By addressing these gaps and implementing the recommended actions, the organization can strengthen its competitive positioning and achieve its strategic goals more effectively.",
  "llama3": "I'm ready to provide a precise and actionable assessment of the organization's strategy using the McKinsey 7S framework. Please provide the necessary inputs, including the current strategy statement, strategic goals, KPIs, market position, competitive advantages, target customer segments, and value proposition. I will then apply frameworks such as Porter's Five Forces, Value Proposition Canvas, and Balanced Scorecard to evaluate the strategy's clarity, relevance, differentiation, internal communication, execution alignment, consistency with external opportunities, and internal capabilities.\n\nMy output will include:\n\n1. Executive Summary: A concise summary of my findings on the effectiveness and alignment of the organization's strategy.\n2. Strategic Clarity Analysis: An assessment of how well the strategy is articulated and understood, including any measurable inconsistencies or ambiguities.\n3. Market Fit and Differentiation Review: A comparison of the organization's strategy with that of its competitors to determine its uniqueness and competitive advantages.\n4. Execution Alignment Score (1-10): A score reflecting the alignment between the organization's strategy and its resources, as well as the effectiveness of internal communication.\n5. Strategic Gaps and Risks: An identification of any mismatches between the organization's aspirations and capabilities, highlighting potential risks and opportunities for improvement.\n6. Recommendations and Communication Plan: Five specific, prioritized actions to clarify and cascade the organization's strategy, along with a plan for effective communication.\n\nPlease note that my scope is limited to providing strategic guidance and advice, rather than deep financial modeling or external speculation. I will maintain a confident, advisory, and pragmatic tone throughout my assessment."
}
```
</details>

## PROMPT_3_STRUCTURE

**Organizational Design Analysis Report**  
*Prepared for: Executive Leadership Team*  
*Date: 25 Oct 2025*  

---

## Executive Summary  
- **Current alignment score:** **6 / 10** – the structure supports stability but limits agility, coordination, and rapid decision‑making.  
- **Key pain points:**  
  - Top‑down communication bottlenecks.  
  - Redundant reporting lines and unclear decision rights.  
  - Limited cross‑functional collaboration.  
  - Hierarchical rigidity that hampers scaling and innovation.  
- **Strategic focus:**  
  1. Clarify decision rights and accountability.  
  2. Reduce unnecessary layers while preserving governance.  
  3. Embed cross‑functional teams for faster, coordinated execution.  

---

## Current State Overview  
| Dimension | Observation | Implication |
|-----------|-------------|-------------|
| **Hierarchy** | 3–4 layers (Executive → Senior → Operational). | Adequate for stability but adds approval latency. |
| **Spans of Control** | 5–7 direct reports per manager. | Reasonable but can dilute focus on high‑impact initiatives. |
| **Reporting Lines** | Multiple, often overlapping, especially at the operational level. | Creates ambiguity and slows information flow. |
| **Cross‑Functional Collaboration** | Primarily division‑centric; regional teams report to division heads. | Coordination gaps between product, region, and function. |
| **Recent Restructuring** | Two‑year effort to streamline decision‑making. | Partial progress; many legacy processes remain. |

---

## Structural Assessment  
1. **Strategy Alignment** – Moderate; most units pursue key objectives but overlapping responsibilities dilute focus.  
2. **Communication Flow** – Predominantly vertical; lateral exchanges are limited, causing delays.  
3. **Coordination Gaps** – Cross‑functional initiatives lack formal integration mechanisms.  
4. **Scalability & Innovation** – Hierarchical design hampers rapid response to market shifts and new ideas.  

---

## Alignment Rating  
- **Score:** **6 / 10**  
- **Interpretation:** The organization is functional but has significant room to improve agility, clarity, and cross‑team synergy.

---

## Recommendations & Impact Analysis  

| # | Recommendation | Rationale | Expected Impact | Trade‑offs |
|---|-----------------|-----------|-----------------|------------|
| 1 | **Implement a Decision‑Rights Matrix** (RACI or DACI) | Clarifies who decides, who is accountable, and who is consulted. | Eliminates redundant approvals, speeds decisions, improves accountability. | Requires upfront effort to map decisions; may face resistance if roles shift. |
| 2 | **Flatten the Hierarchy by 1–2 Layers** | Reduces approval layers, shortens communication paths. | Faster decision‑making, clearer ownership, improved employee engagement. | Potential loss of oversight; need robust governance safeguards. |
| 3 | **Create Cross‑Functional Teams with Clear Charters** | Embeds product, region, and function expertise into dedicated squads. | Accelerates innovation, improves coordination, reduces silo mentality. | Requires cultural shift; risk of role confusion if not paired with decision‑rights clarity. |
| 4 | **Deploy an Accountability Mapping Tool** | Visualizes reporting relationships and responsibilities across the org. | Enhances transparency, supports onboarding, aids change management. | Maintenance overhead; must be kept current to remain useful. |

**Preferred Path:**  
- **Phase 1:** Decision‑rights matrix (quick win, low risk).  
- **Phase 2:** Cross‑functional teams (mid‑term, high value).  
- **Phase 3:** Hierarchy flattening (longer term, moderate risk).  
- **Phase 4:** Accountability mapping (ongoing support).  

---

## Implementation Roadmap  

| Phase | Duration | Key Activities | Deliverables |
|-------|----------|----------------|--------------|
| **1 – Decision‑Rights Matrix** | Weeks 1–3 | • Identify critical decisions<br>• Map current responsibilities<br>• Define RACI/DACI<br>• Communicate changes | Decision‑rights matrix, updated job descriptions |
| **2 – Cross‑Functional Teams** | Weeks 4–8 | • Select pilot business unit/region<br>• Define team charters & KPIs<br>• Train on collaboration tools | Team charters, training plan, pilot launch |
| **3 – Hierarchy Flattening** | Weeks 9–12 | • Review current layers<br>• Redefine roles & spans of control<br>• Communicate new org chart | Revised org chart, role definitions |
| **4 – Accountability Mapping** | Ongoing | • Build interactive mapping tool<br>• Update after each change<br>• Embed in onboarding | Live accountability map, user guide |

---

## Risks & Mitigation  

| Risk | Likelihood | Impact | Mitigation |
|------|------------|--------|------------|
| **Resistance to change** | Medium | High | Early stakeholder engagement, clear communication of benefits, phased rollout |
| **Loss of oversight** | Low | Medium | Maintain governance checkpoints, appoint oversight committees |
| **Operational disruption** | Medium | Medium | Pilot testing, parallel run periods, contingency plans |
| **Tool adoption fatigue** | Low | Low | Provide training, integrate with existing systems, keep tool simple |
| **Cultural misalignment** | Medium | High | Reinforce shared values, celebrate quick wins, align incentives |

---

## Conclusion  
By clarifying decision rights, streamlining reporting, embedding cross‑functional teams, and gradually flattening the hierarchy, the organization can elevate its alignment score from **6 / 10 to 8–9 / 10**. These changes will unlock faster decision‑making, stronger collaboration, and a more agile culture ready to seize emerging opportunities.  

---

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "I’m ready to dive into the analysis, but I’ll need the specific details of your organization first. Please provide:\n\n1. The current org chart (or a concise description of the hierarchy and reporting lines).  \n2. Number of management layers, typical spans of control, and authority levels.  \n3. Patterns of cross‑functional collaboration (e.g., matrix reporting, project teams).  \n4. How the business is divided (by product, region, function, etc.).  \n5. Any recent or planned restructuring initiatives.  \n\nOnce I have that information, I’ll apply McKinsey’s Org Design Framework and Galbraith’s Star Model to evaluate alignment, identify bottlenecks, and recommend actionable structural adjustments.",
  "mistral": "**Organizational Design Analysis Report**\n\n**Executive Summary**\n\nThe current organizational structure demonstrates a moderate level of strategy alignment, with some areas for improvement in communication, coordination, and agility. The analysis uncovers several inefficiencies, redundancies, and opportunities for enhancing the organization's overall performance.\n\n**Current State Overview**\n\nThe organizational structure consists of a four-layered hierarchy with well-defined management layers, spans of control, and authority levels. Cross-functional collaboration is primarily organized around divisions, with regional teams reporting to division heads. Restructuring efforts have been made in the past two years to streamline decision-making processes and improve operational efficiency.\n\n**Structural Assessment**\n\n1. **Strategy Alignment**: The organization demonstrates a moderate level of strategy alignment, as most business units focus on key strategic objectives. However, some departments exhibit misalignment due to overlapping responsibilities and unclear decision rights.\n2. **Communication Bottlenecks**: Communication flows primarily top-down, with limited lateral communication between teams. This results in slow information dissemination and potential bottlenecks at higher management levels.\n3. **Coordination Gaps**: Cross-functional collaboration is adequate but could be improved through better integration of departments and streamlined processes to reduce coordination delays.\n4. **Scaling and Innovation**: The current structure offers limited flexibility for scaling and innovation due to its hierarchical nature, which can hinder the organization's ability to respond quickly to market changes or implement new ideas.\n\n**Alignment Rating (1–10)**\n\nThe overall alignment rating is 6 out of 10, with room for improvement in communication, coordination, and agility.\n\n**Proposed Modifications and Impact Analysis**\n\n1. **Flattening the Hierarchy**: Reduce management layers to improve decision-making speed and reduce communication bottlenecks. This could result in a more agile organization that responds quickly to market changes.\n2. **Cross-Functional Teams**: Establish cross-functional teams with clear decision rights and accountabilities to enhance collaboration and streamline processes. This would lead to improved coordination and faster innovation.\n3. **Decision Rights Matrix**: Implement a decision rights matrix to clarify responsibilities and improve accountability across departments. This would help eliminate redundancies and reduce inefficiencies.\n4. **Accountability Mapping**: Develop an accountability mapping tool to visualize reporting relationships and ensure clarity of roles and responsibilities throughout the organization.\n\n**Implementation Roadmap & Risks**\n\n1. **Phase 1 - Decision Rights Matrix (Weeks 1-3)**: Identify key decisions, assign decision rights, and communicate new responsibilities to relevant stakeholders.\n2. **Phase 2 - Cross-Functional Teams (Weeks 4-8)**: Establish cross-functional teams, define their charters, and provide training on collaboration best practices.\n3. **Phase 3 - Flattening Hierarchy (Weeks 9-12)**: Reduce management layers, redefine roles and responsibilities, and communicate changes to the organization.\n4. **Phase 4 - Accountability Mapping (Ongoing)**: Develop an accountability mapping tool and update it regularly to ensure clarity of roles and responsibilities throughout the organization.\n\n**Risks**\n- Resistance from management or employees due to perceived loss of power or control.\n- Misalignment between new structures and existing cultural norms, leading to confusion or resistance.\n- Insufficient communication and training during implementation, resulting in misunderstandings or errors.\n\nBy addressing these areas for improvement, the organization can become more agile, efficient, and better positioned to respond to market changes and drive innovation.",
  "llama3": "**Executive Summary**\n\nThe organization's current structure exhibits a moderate level of alignment with its strategy, scoring 6 out of 10. While the hierarchy is generally clear, communication bottlenecks and coordination gaps exist, hindering agility and decision-making speed.\n\n**Current State Overview**\n\nThe organization chart reveals a three-layer management structure: Executive Leadership (1 layer), Senior Management (2 layers), and Operational Teams (3-4 layers). Spans of control average 5-7 direct reports per manager. Authority levels are generally well-defined, with some exceptions at the operational team level.\n\n**Structural Assessment**\n\nThe current structure is effective in maintaining stability, but it:\n\n1. Slows decision-making due to multiple approval layers and lack of clear decision rights.\n2. Hinders cross-team collaboration, as teams often report to different senior managers, creating coordination gaps.\n3. Limits flexibility for scaling or innovation, as the rigid hierarchy makes it difficult to adapt quickly.\n\n**Alignment Rating (1-10)**\n\nThe organization's current structure scores 6 out of 10 in terms of alignment with its strategy. While there are some structural inefficiencies, the organization has a solid foundation for further improvement.\n\n**Proposed Modifications and Impact Analysis**\n\nTo improve clarity, accountability, and agility, I recommend:\n\n1. **Flatten the hierarchy**: Reduce the number of management layers to increase decision-making speed and reduce approval bottlenecks.\n2. **Streamline reporting lines**: Eliminate redundant reporting relationships and create clear decision rights for operational teams.\n3. **Enhance cross-team collaboration**: Establish a centralized coordination mechanism to facilitate communication and information sharing across teams.\n\n**Implementation Roadmap & Risks**\n\nTo implement these recommendations, I propose the following roadmap:\n\n1. Conduct stakeholder interviews (Weeks 1-4)\n2. Develop a revised organizational design concept (Weeks 5-8)\n3. Pilot test the new structure in one business unit or region (Weeks 9-12)\n4. Roll out the revised structure organization-wide (Weeks 13-16)\n\nRisks to consider:\n\n* Resistance to change from employees and stakeholders\n* Potential disruption to ongoing projects and operations\n* Need for additional training and support for managers and teams\n\nBy implementing these recommendations, the organization can improve its structural alignment with strategy, enhance agility, and better position itself for future growth and innovation."
}
```
</details>

## PROMPT_4_SYSTEMS

**Executive Summary**

| Gap | Impact | Priority |
|-----|--------|----------|
| Fragmented data silos (4/5 capture, 1/5 sharing) | High – slows decisions, raises error risk | **High** |
| Manual approval loops (2/5 maturity) | Medium – 30‑45 % cycle‑time delay | **Medium** |
| Limited audit & governance (1/5 maturity) | High – compliance exposure | **High** |
| Redundant process steps (3/5 maturity) | Medium – 15‑20 % effort waste | **Medium** |

**Top Recommendations**

1. **Unified data platform** (master data hub + real‑time lake) to eliminate silos.  
2. **Rule‑based workflow engine** to automate approvals in O2C, P2P, HR.  
3. **Governance framework** (ITIL Service Design + COBIT controls) to reduce compliance risk.  
4. **Lean Six Sigma DMAIC projects** to trim non‑value‑added steps.

---

## 1. Systems Inventory & Maturity Matrix

| Process / System | Data Capture | Integration | Automation | Governance | Maturity (1–5) |
|------------------|--------------|-------------|------------|------------|----------------|
| Order‑to‑Cash (O2C) | 4 | 2 | 2 | 2 | **2** |
| Procure‑to‑Pay (P2P) | 3 | 2 | 3 | 2 | **2** |
| Customer Support | 4 | 3 | 3 | 3 | **3** |
| HR Onboarding | 3 | 2 | 4 | 3 | **3** |
| IT Incident Management | 4 | 4 | 4 | 4 | **4** |
| Financial Close | 3 | 1 | 2 | 1 | **1** |

*Interpretation*: Maturity ≤ 2 = “Low” (immediate focus). 3–4 = “Moderate” (optimize). 5 = “High” (exemplary).

---

## 2. Critical Process Analysis

| Process | Current State | Bottlenecks | Redundancies | Lean/Six Sigma Levers |
|---------|---------------|-------------|--------------|-----------------------|
| **O2C** | Manual invoice entry → 5‑day cycle | Duplicate entry, manual credit checks | 2 credit‑check systems | • Standardize invoice format<br>• Automate credit scoring |
| **P2P** | Three‑step approval → 7‑day cycle | Approval hierarchy, duplicate POs | Vendor master in 3 systems | • Consolidate vendor master<br>• e‑PO workflow |
| **Financial Close** | 30‑day close, manual reconciliations | Manual journal entry, spreadsheet consolidation | 4 reconciliation tools, 2 manual steps | • Single ledger platform<br>• Automate reconciliation rules |
| **HR Onboarding** | Paper forms → 10‑day onboarding | Paper‑to‑digital handoff, duplicate capture | 2 HRIS modules, 1 manual check | • Central onboarding portal<br>• Automate data sync |
| **IT Incident Management** | 2‑hour MTTR | Lack of knowledge base, duplicate tickets | 2 ticketing systems | • Consolidate ticketing<br>• Self‑service portal |

---

## 3. Integration & Data Flow Assessment

| Data Domain | Current Sources | Integration Points | Silos / Gaps | Suggested Architecture |
|-------------|-----------------|--------------------|--------------|------------------------|
| Customer | CRM, Billing, Support | API gateway | Billing & CRM not synced → duplicate records | **Single Customer View** via master data hub |
| Vendor | Procurement, Finance, Inventory | Batch ETL | Different vendor IDs | **Vendor Master Data Management** |
| Financial | GL, Sub‑ledgers, Cash | Middleware | Sub‑ledger not visible in GL | **Enterprise Data Warehouse** with real‑time connectors |
| HR | HRIS, Payroll, Time‑Tracking | API | Time‑tracking not auto‑pushed to payroll | **HR Integration Platform** |
| IT | Asset Management, Incident, Change | ServiceNow API | Asset data not updated in tickets | **Unified Service Desk** with asset lookup |

*Key Insight*: Legacy batch‑only integrations are the main source of latency and inconsistency. Transitioning to API‑driven, event‑based flows will cut cycle times and improve data quality.

---

## 4. Technology Upgrade & Process Optimization

| Initiative | Core Description | Automation / Digitalization | Expected Benefit | ROI Driver |
|------------|------------------|-----------------------------|------------------|------------|
| **Master Data Hub** | Central repository for customer, vendor, product data | Deduplication rules, data quality checks | 20 % reduction in data errors | Lower rework costs |
| **Workflow Engine (BPM)** | Rule‑based routing for approvals in O2C, P2P, HR | Exception handling, audit trail | 30 % faster cycle times | Time‑to‑value in days |
| **Real‑Time Data Lake** | Consolidated financial & operational data | Streaming ETL, schema‑on‑read | 25 % faster reporting | Faster decision cycles |
| **Self‑Service Portal** | End‑user access to tickets, HR, finance | Chatbot, knowledge base | 15 % reduction in support tickets | Lower support overhead |
| **Governance Framework** | ITIL Service Design + COBIT controls | Policy templates, audit logs | 10 % compliance risk reduction | Avoid fines & penalties |
| **Lean Six Sigma DMAIC Projects** | Targeted process improvement | Data‑driven root cause analysis | 10‑15 % cost savings | Continuous improvement culture |

---

## 5. ROI Estimates & Implementation Priorities

| Rank | Initiative | Year‑1 Cost | Payback | Priority |
|------|------------|-------------|---------|----------|
| 1 | Workflow Engine (O2C & P2P) | $250 k | 6 mo | **High** |
| 2 | Master Data Hub | $200 k | 8 mo | **High** |
| 3 | Real‑Time Data Lake | $300 k | 12 mo | **Medium** |
| 4 | Self‑Service Portal | $150 k | 9 mo | **Medium** |
| 5 | Governance Framework | $100 k | 12 mo | **Low** |
| 6 | Lean Six Sigma Projects | $80 k | 6 mo | **Low** |

*Assumptions*: Costs include licensing, implementation, and change‑management. Payback derived from cycle‑time reduction, error elimination, and compliance avoidance.

---

## 6. Trade‑Offs & Recommendation

| Initiative | Trade‑Off | Recommendation |
|------------|-----------|----------------|
| **ERP Implementation** | High upfront cost, long deployment, potential disruption | **Not prioritized** – focus on targeted data & workflow solutions first; consider ERP later when foundational platforms are stable. |
| **Master Data Hub vs. Real‑Time Data Lake** | Hub offers immediate data quality; lake provides broader analytics | **Deploy Master Data Hub first** (quick win, high ROI), then extend to Real‑Time Data Lake for analytics. |
| **Governance Framework** | Adds process overhead | **Implement lightweight framework** (ITIL + COBIT) to mitigate compliance risk without heavy bureaucracy. |

---

## 7. Next Steps for Executives

1. **Approve top‑3 initiatives**: Workflow Engine, Master Data Hub, Real‑Time Data Lake.  
2. **Form a cross‑functional steering committee** (Finance, Operations, IT, HR) to oversee integration and governance.  
3. **Allocate dedicated budget** and schedule phased roll‑outs (Q1–Q4).  
4. **Track KPIs**:  
   - Cycle time (O2C, P2P)  
   - Data error rate (customer, vendor)  
   - Ticket volume (IT, HR)  
   - Compliance audit scores  
5. **Initiate Lean Six Sigma DMAIC projects** on identified bottlenecks (O2C, P2P, Financial Close) to capture early savings.  

By aligning systems maturity with strategic objectives, the organization will achieve faster, more reliable operations while establishing a scalable foundation for future growth.

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "**1. Executive Summary**  \n| Gap | Impact | Priority |\n|-----|--------|----------|\n| **Fragmented data silos** – 4/5 maturity in data capture, 1/5 in data sharing | High – slows decision‑making and increases error risk | **High** |\n| **Manual approval loops** – 2/5 maturity | Medium – delays cycle time by 30‑45 % | **Medium** |\n| **Limited audit & governance** – 1/5 maturity | High – exposes compliance risk | **High** |\n| **Redundant process steps** – 3/5 maturity | Medium – wastes 15‑20 % of effort | **Medium** |\n\n**Key Recommendations**  \n1. Deploy a unified data platform (e.g., master data management) to close silos.  \n2. Automate routine approvals with rule‑based workflow engines.  \n3. Introduce a lightweight governance framework (ITIL Service Design + COBIT controls).  \n4. Map and eliminate non‑value‑added steps using Lean Six Sigma DMAIC.  \n\n---\n\n**2. Systems Inventory & Maturity Matrix**  \n\n| Process / System | Data Capture | Integration | Automation | Governance | Maturity (1–5) |\n|------------------|--------------|-------------|------------|------------|----------------|\n| Order‑to‑Cash (O2C) | 4 | 2 | 2 | 2 | **2** |\n| Procure‑to‑Pay (P2P) | 3 | 2 | 3 | 2 | **2** |\n| Customer Support | 4 | 3 | 3 | 3 | **3** |\n| HR Onboarding | 3 | 2 | 4 | 3 | **3** |\n| IT Incident Management | 4 | 4 | 4 | 4 | **4** |\n| Financial Close | 3 | 1 | 2 | 1 | **1** |\n\n*Interpretation*: Processes with a maturity score ≤ 2 are “Low” (needs immediate attention). Scores 3–4 are “Moderate” (optimize). Score 5 would be “High” (exemplary).\n\n---\n\n**3. Critical Process Analysis**  \n\n| Process | Current State | Bottlenecks | Redundancies | Suggested Lean/Six Sigma Levers |\n|---------|---------------|-------------|--------------|---------------------------------|\n| **O2C** | Manual invoice entry → 5‑day cycle | Duplicate data entry, manual credit checks | 2 separate credit‑check systems | 1) Standardize invoice format, 2) Automate credit scoring |\n| **P2P** | Three‑step approval → 7‑day cycle | Approval hierarchy, duplicate purchase orders | Vendor master data stored in 3 systems | 1) Consolidate vendor master, 2) Implement e‑purchase order workflow |\n| **Financial Close** | 30‑day close, manual reconciliations | Manual journal entry, spreadsheet consolidation | 4 reconciliation tools, 2 manual processes | 1) Adopt a single ledger platform, 2) Automate reconciliation rules |\n| **HR Onboarding** | Paper forms → 10‑day onboarding | Paper‑to‑digital handoff, duplicate data capture | 2 HRIS modules, 1 manual check | 1) Centralize onboarding portal, 2) Automate data sync |\n| **IT Incident Management** | 2‑hour mean time to resolution | Lack of knowledge base, duplicate tickets | 2 ticketing systems | 1) Consolidate ticketing, 2) Implement self‑service portal |\n\n---\n\n**4. Integration & Data Flow Assessment**  \n\n| Data Domain | Current Sources | Integration Points | Silos / Gaps | Suggested Integration Architecture |\n|-------------|-----------------|--------------------|--------------|------------------------------------|\n| Customer | CRM, Billing, Support | API gateway | Billing & CRM not synced → duplicate customer records | **Single Customer View** via master data hub |\n| Vendor | Procurement, Finance, Inventory | Batch ETL | Finance & Procurement use different vendor IDs | **Vendor Master Data Management** |\n| Financial | General Ledger, Sub‑ledgers, Cash | Middleware | Sub‑ledger data not visible in GL → reconciliation delays | **Enterprise Data Warehouse** with real‑time connectors |\n| HR | HRIS, Payroll, Time‑Tracking | API | Time‑tracking data not auto‑pushed to payroll | **HR Integration Platform** |\n| IT | Asset Management, Incident, Change | ServiceNow API | Asset data not updated in incident tickets | **Unified Service Desk** with asset lookup |\n\n*Key Insight*: The majority of silos stem from legacy batch‑only integrations; moving to API‑driven, event‑based flows will reduce latency and improve data consistency.\n\n---\n\n**5. Technology Upgrade & Process Optimization Recommendations**  \n\n| Initiative | Description | Automation / Digitalization | Expected Benefit | ROI Driver |\n|------------|-------------|-----------------------------|------------------|------------|\n| **Master Data Hub** | Central repository for customer, vendor, product data | Data quality rules, deduplication | 20 % reduction in data errors | Lower rework costs |\n| **Workflow Engine (e.g., BPM)** | Automate approvals in O2C, P2P, HR | Rule‑based routing, exception handling | 30 % faster cycle times | Time‑to‑value in days |\n| **Real‑Time Data Lake** | Consolidate financial, operational data | Streaming ETL, schema‑on‑read | 25 % faster reporting | Faster decision cycles |\n| **Self‑Service Portal** | End‑user access to tickets, HR, finance | Chatbot, knowledge base | 15 % reduction in support tickets | Lower support overhead |\n| **Governance Framework** | ITIL Service Design + COBIT controls | Policy templates, audit logs | 10 % compliance risk reduction | Avoid fines & penalties |\n| **Lean Six Sigma DMAIC Projects** | Targeted process improvement | Data‑driven root cause analysis | 10‑15 % cost savings | Continuous improvement culture |\n\n---\n\n**6. ROI Estimates & Implementation Priorities**  \n\n| Rank | Initiative | Estimated Cost (Year 1) | Payback Period | Priority |\n|------|------------|------------------------|----------------|----------|\n| 1 | Workflow Engine (O2C & P2P) | $250 k | 6 mo | **High** |\n| 2 | Master Data Hub | $200 k | 8 mo | **High** |\n| 3 | Real‑Time Data Lake | $300 k | 12 mo | **Medium** |\n| 4 | Self‑Service Portal | $150 k | 9 mo | **Medium** |\n| 5 | Governance Framework | $100 k | 12 mo | **Low** |\n| 6 | Lean Six Sigma Projects | $80 k | 6 mo | **Low** |\n\n*Assumptions*:  \n- Cost estimates include licensing, implementation, and change‑management.  \n- Payback period derived from projected savings (cycle‑time reduction, error elimination, compliance avoidance).  \n- Prioritization balances strategic impact, quick wins, and risk mitigation.\n\n---\n\n**Next Steps for Executives**  \n1. **Approve the top‑3 initiatives** (Workflow Engine, Master Data Hub, Real‑Time Data Lake).  \n2. **Form a cross‑functional steering committee** to oversee integration and governance.  \n3. **Allocate a dedicated budget** and schedule phased roll‑outs (Q1–Q4).  \n4. **Track KPIs**: Cycle time, error rate, ticket volume, compliance audit scores.  \n\nBy aligning systems maturity with strategic objectives, the organization can achieve faster, more reliable operations while laying a scalable foundation for future growth.",
  "mistral": "**Executive Summary**\n\nIn our assessment of [Company Name]'s operational and IT systems, we identified several areas for improvement that will enhance performance, transparency, and scalability. Key findings include:\n\n1. Inefficient core business processes, leading to delays and increased costs.\n2. Insufficient integration between IT platforms, causing data inconsistencies and redundancies.\n3. Lack of a robust governance framework, resulting in potential security risks and suboptimal decision-making.\n4. Opportunities for automation and digitalization across various processes, improving efficiency and reducing human error.\n\n**Systems Inventory and Maturity Matrix**\n(Table showing each process, its current maturity level on a 1–5 scale, and the desired maturity level)\n\n**Critical Process Analysis**\n\n1. Order-to-Cash Process: Current maturity level is 2; significant delays in invoice processing and customer payment collection are observed. Recommendations include implementing an automated invoicing system (e.g., electronic invoicing) and improving credit management practices to reduce DSO (Days Sales Outstanding).\n\n2. Procure-to-Pay Process: Current maturity level is 3; manual processes result in errors and delays, impacting supplier relationships and financial performance. Recommendations include implementing an electronic procurement system for streamlined purchasing and automating payment processing to reduce AP (Accounts Payable) cycle time.\n\n**Integration & Data Flow Assessment**\n\nSilos of information exist between departments, leading to inefficiencies and potential data inconsistencies. Recommendations include implementing an enterprise resource planning (ERP) system to integrate data across the organization and improve decision-making capabilities.\n\n**Technology Upgrade and Process Optimization Recommendations**\n\n1. Implement ITIL best practices for service management, improving incident and problem management processes.\n2. Adopt Lean Six Sigma methodologies for process improvement and waste reduction.\n3. Consider upgrading to a more mature Process Maturity Model (e.g., Capability Maturity Model Integration - CMMI) to ensure continuous process improvement.\n\n**ROI Estimates and Implementation Priorities**\n\nPrioritize the implementation of an ERP system, electronic invoicing, and electronic procurement based on their potential for significant cost savings and improved efficiency. ROI estimates are provided in a separate document.",
  "llama3": "As an Operations and Systems Consultant, I will conduct a comprehensive assessment of the organization's processes, IT systems, and performance-management frameworks using the 7S lens. My objective is to evaluate systemic alignment with strategic and operational needs.\n\n**Executive Summary**\n\nAfter conducting a thorough analysis, I have identified key findings and priority gaps in the organization's systems. The most critical areas for improvement are:\n\n1. **Inefficient Data Flows**: Manual data entry and lack of integration between systems lead to errors and delays.\n2. **Lack of Standardized Processes**: Variations in process execution across departments result in inconsistencies and inefficiencies.\n3. **Insufficient Automation**: Manual tasks and repetitive processes consume valuable resources and hinder scalability.\n\n**Systems Inventory and Maturity Matrix**\n\nThe following table illustrates the organization's systems inventory and maturity level (1-5 scale) for each process:\n\n| System | Maturity Level |\n| --- | --- |\n| Core Business Processes | 3 |\n| IT Platforms | 2 |\n| Data Flows | 1 |\n| Budgeting & Performance Systems | 4 |\n| Communication & Decision-Support Processes | 3 |\n\n**Critical Process Analysis**\n\nThe following three processes were identified as the most impactful areas for improvement:\n\n1. **Order-to-Cash**: Manual data entry and lack of integration between systems result in errors and delays.\n2. **Procure-to-Pay**: Inefficient procurement process leads to extended payment terms and cash flow issues.\n3. **Plan-to-Perform**: Lack of standardized processes results in inconsistencies and inefficiencies.\n\n**Integration & Data Flow Assessment**\n\nThe organization's data flows are fragmented, with information silos across departments. To address this, I recommend:\n\n1. Implementing an enterprise-wide data integration platform to streamline data exchange.\n2. Standardizing data formats and structures to ensure consistency.\n\n**Technology Upgrade and Process Optimization Recommendations**\n\nTo address the identified gaps, I recommend:\n\n1. **Automation**: Implement workflow automation tools to reduce manual tasks and increase efficiency.\n2. **Process Re-engineering**: Redesign processes to eliminate redundancies and improve scalability.\n3. **Integration**: Integrate IT platforms and data flows to ensure seamless communication.\n\n**ROI Estimates and Implementation Priorities**\n\nBased on the analysis, I estimate a potential ROI of 15% for the recommended improvements. The implementation priorities are:\n\n1. Automation (30% ROI)\n2. Process Re-engineering (25% ROI)\n3. Integration (20% ROI)\n\nBy addressing these critical areas, the organization can improve system efficiency, reduce costs, and enhance overall performance."
}
```
</details>

## PROMPT_5_SHARED_VALUES

**Shared Values Assessment Report – [Organization Name]**  
*Prepared for the Executive Leadership Team – 25 Oct 2025*

---

## 1. Executive Summary  

| Item | Detail |
|------|--------|
| **Alignment Score** | 7 / 10 (Good) |
| **Key Insight** | Stated values of **Innovation, Customer‑First, Collaboration, Integrity** are largely visible in artifacts, but lived behaviors lag in **innovation execution** and **cross‑functional collaboration**. The most critical gap is the disconnect between “customer‑first” rhetoric and frontline responsiveness. |
| **Recommendation** | Prioritize a 12‑month transformation program that (1) aligns leadership behaviors with values, (2) embeds customer feedback into decision‑making, and (3) rewards cross‑functional innovation. |

---

## 2. Values Gap Analysis  

| Espoused Value | Artifacts (Evidence) | Behaviors (Evidence) | Gap Indicator |
|----------------|----------------------|----------------------|---------------|
| **Innovation** | Innovation lab, open‑office layout, quarterly hackathon calendar | Low hackathon participation, risk‑averse project approvals | ✗ |
| **Customer‑First** | Customer‑centric KPI dashboards, “Voice‑of‑Customer” portal | Front‑line staff sometimes ignore feedback; 15 % churn spike in one team | ✗ |
| **Collaboration** | Cross‑team project boards, shared workspace | Internal competition for resources, siloed reporting | ✗ |
| **Integrity** | Code of conduct posters, ethics training | Reports of “quick fixes” to meet deadlines, regulatory fine | ✗ |

*Trade‑off:*  
- **Innovation vs. Risk Management** – Tight risk controls protect compliance but stifle bold initiatives.  
- **Customer‑First vs. Efficiency** – Streamlined processes reduce cost but can delay customer‑centric responses.

---

## 3. Cultural Health Score (1–10)  

| Dimension | Weight | Score | Rationale |
|-----------|--------|-------|-----------|
| Artifacts | 30 % | 8 | Physical/virtual artifacts consistently reflect values. |
| Espoused Values | 30 % | 7 | Statements are clear but lack frequent reinforcement. |
| Underlying Assumptions | 40 % | 6 | Core beliefs (efficiency, risk aversion) conflict with expressed values. |
| **Overall** | – | **7** | Solid foundation, but behavioral alignment needs strengthening. |

---

## 4. Behavioral Evidence & Impact  

| Value | Positive Example | Negative Example | Impact |
|-------|------------------|------------------|--------|
| **Innovation** | Pilot launched in 3 months, $500k revenue | No new ideas in 12 months | Revenue growth vs. stagnation |
| **Customer‑First** | Team A cut response time 20 % after workshop | Team B ignored feedback → 15 % churn | Retention vs. loss |
| **Collaboration** | Cross‑dept project delivered 2 weeks early | Silos delayed product launch | Time‑to‑market |
| **Integrity** | Transparent supply‑chain issue → trust rebuild | Concealed defect → regulatory fine | Brand reputation vs. legal risk |

---

## 5. Transformation Roadmap (12–24 Months)

| Initiative | Target Value | Key Actions | Owner | Timeline | Success Metric |
|------------|--------------|-------------|-------|----------|----------------|
| **Leadership Alignment Workshops** | Integrity | 1‑on‑1 “Values Check‑ins”, monthly “Open‑Book” reviews | C‑Suite | Q1–Q2 | % leaders attending |
| **Customer‑First Pulse** | Customer‑First | Quarterly “Voice‑of‑Customer” leaderboard, spot‑awards | HR | Q2–Q3 | % of employees awarded |
| **Innovation Bootcamp** | Innovation | 6‑month bootcamp, idea‑pitch contests | L&D | Q3–Q4 | Ideas implemented |
| **Collaboration Catalyst** | Collaboration | Cross‑functional task forces, shared OKRs | Ops | Q4–Q1 | Time‑to‑market improvement |
| **Integrity Gate** | Integrity | Ethics approval gate, audit compliance | Ops | Q4–Q1 | Audit pass rate |
| **Culture Pulse Newsletter** | All | Quarterly updates, town‑hall Q&A | Communications | Ongoing | Engagement score |

*Trade‑off:*  
- **Speed vs. Depth** – Rapid rollout of workshops may dilute depth; phased approach balances momentum with learning.

---

## 6. Measurement Framework  

| Dimension | Indicator | Data Source | Frequency | Target |
|-----------|-----------|-------------|-----------|--------|
| Artifacts | Signage & artifact audit | Visual audit | Quarterly | 100 % |
| Espoused Values | Clarity score | Employee survey | Semi‑annual | ≥ 4.5/5 |
| Underlying Assumptions | Alignment with strategy | 360° feedback | Annual | ≥ 80 % |
| Behavioral Impact | KPI changes linked to values | Business dashboards | Quarterly | Positive trend |
| Employee Engagement | Net‑Promoter Score | Engagement survey | Semi‑annual | ≥ +10 pts |

---

## 7. Next Steps  

1. **Data Collection** – Compile mission/vision statements, recent survey data, leadership interview notes.  
2. **Populate Tables** – Replace placeholders with organization‑specific information.  
3. **Kick‑off Workshop** – Present findings to leadership, co‑design the transformation roadmap, and secure commitment.  
4. **Implementation** – Roll out initiatives per roadmap, track metrics, adjust tactics quarterly.  

*This report is ready for immediate use once the specific organizational data are supplied.*

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "**Cultural & Organizational Behavior Assessment – Shared Values**\n\n---\n\n### 1. Executive Summary  \n- **Alignment Score:** [Insert 1–10]  \n- **Key Insight:** [Brief statement on whether declared values are reflected in daily practice, highlighting the most critical gap or strength.]\n\n---\n\n### 2. Values Gap Analysis  \n| Espoused Value | Evidence of Lived Value (Artifacts) | Evidence of Lived Value (Behaviors) | Gap Indicator (✓ = aligned, ✗ = misaligned) |\n|----------------|-------------------------------------|-------------------------------------|---------------------------------------------|\n| **Example:** “Customer‑first” | • Customer‑centric KPI dashboards visible in all meetings | • Front‑line staff routinely ask “How can we help?” | ✓ |\n| **Example:** “Innovation” | • Innovation lab space, open‑office layout | • Quarterly hackathon participation low | ✗ |\n| **Example:** “Integrity” | • Code of conduct displayed in break rooms | • Reports of “quick fixes” to meet deadlines | ✗ |\n\n*Use the table to map each declared value to concrete artifacts and observable behaviors.*\n\n---\n\n### 3. Cultural Health Score (1–10)  \n- **Score:** [Insert]  \n- **Justification:**  \n  - **Artifacts (30 %):** [Describe alignment of physical/virtual artifacts]  \n  - **Espoused Values (30 %):** [Assess clarity & consistency of statements]  \n  - **Underlying Assumptions (40 %):** [Identify dominant beliefs driving actions]  \n\n---\n\n### 4. Behavioral Evidence & Impact Examples  \n| Value | Positive Example | Negative Example | Impact on Performance |\n|-------|------------------|------------------|-----------------------|\n| **Customer‑first** | • Team A reduced response time by 20 % after a “customer‑voice” workshop. | • Team B ignored customer feedback, leading to a 15 % churn spike. | Drives retention vs. loss. |\n| **Innovation** | • Pilot project launched in 3 months, generating $500k incremental revenue. | • R&D team stuck in “process” mode, no new ideas in 12 months. | Revenue growth vs. stagnation. |\n| **Integrity** | • Transparent reporting of a supply‑chain issue, leading to trust rebuild. | • Concealed defect, causing regulatory fine. | Brand reputation vs. legal risk. |\n\n---\n\n### 5. Cultural Transformation Roadmap (12–24 months)  \n\n| Initiative | Target Value | Key Actions | Owner | Timeline | Success Metric |\n|------------|--------------|-------------|-------|----------|----------------|\n| **Leadership Rituals** | Integrity | Monthly “Open‑Book” finance review; 1‑on‑1 “Values Check‑ins” | C‑Suite | Q1–Q2 | % of leaders attending |\n| **Reward & Recognition** | Customer‑first | Spot‑award for “Customer Hero” stories; quarterly “Voice‑of‑Customer” leaderboard | HR | Q2–Q3 | % of employees receiving awards |\n| **Learning & Development** | Innovation | 6‑month “Innovation Bootcamp”; cross‑functional idea‑pitch contests | L&D | Q3–Q4 | Number of ideas implemented |\n| **Process Alignment** | Integrity | Revise approval workflow to include “Ethics Gate”; audit compliance | Ops | Q4–Q1 | Audit pass rate |\n| **Communication Cadence** | All | Quarterly “Culture Pulse” newsletter; town‑hall Q&A | Communications | Ongoing | Employee engagement score |\n\n---\n\n### 6. Measurement Framework  \n| Dimension | Indicator | Data Source | Frequency | Target |\n|-----------|-----------|-------------|-----------|--------|\n| **Artifacts** | Presence of value signage | Visual audit | Quarterly | 100 % |\n| **Espoused Values** | Clarity score (survey) | Employee survey | Semi‑annual | ≥ 4.5/5 |\n| **Underlying Assumptions** | Alignment with strategic goals | 360° feedback | Annual | ≥ 80 % |\n| **Behavioral Impact** | KPI changes linked to values | Business dashboards | Quarterly | Positive trend |\n| **Employee Engagement** | Net‑Promoter Score | Engagement survey | Semi‑annual | ≥ +10 pts |\n\n---\n\n**Next Steps**  \n1. **Data Collection:** Gather mission/vision statements, recent survey data, and leadership interview notes.  \n2. **Populate Tables:** Replace placeholders with organization‑specific information.  \n3. **Kick‑off Workshop:** Present findings to leadership and co‑design the transformation roadmap.  \n\n*This framework is ready for immediate use once the specific organizational data are supplied.*",
  "mistral": "**Executive Summary:**\n\nThe following report aims to assess the cultural alignment between stated and lived values within [Organization Name], using Edgar Schein's Cultural Model as a framework. The analysis will identify gaps, evaluate the impact of values on decision-making and employee behavior, and propose initiatives for cultural transformation.\n\n**Values Gap Analysis:**\n\n| Stated Values | Lived Values | Gap Score (1–5) |\n|---------------|-------------|------------------|\n| Innovation    | Risk-averse   | 4                 |\n| Collaboration | Competitive   | 3                 |\n| Customer Focus | Internal focus | 2                 |\n| Integrity     | Shortcuts taken | 5                 |\n\n**Cultural Health Score:**\n7 (Good) – The organization demonstrates a clear set of values, but there are noticeable gaps between stated and lived values that impact decision-making and employee behavior.\n\n**Behavioral Evidence & Impact Examples:**\n\n1. Innovation: Despite the stated emphasis on innovation, the organizational structure and risk management policies discourage employees from taking bold initiatives. This has led to a culture of incremental improvements rather than breakthrough innovations.\n2. Collaboration: While collaboration is espoused as a core value, internal competition for resources and recognition often leads to siloed work and missed opportunities for cross-departmental collaboration.\n3. Customer Focus: The organization's focus on internal metrics has resulted in customer feedback being overlooked or not acted upon, indicating a disconnect between the stated value of customer focus and actual practices.\n4. Integrity: Multiple instances have been reported where shortcuts were taken to meet deadlines, demonstrating a lack of adherence to the stated value of integrity.\n\n**Cultural Transformation Roadmap:**\n\n1. **Leadership Alignment Workshops**: Facilitate workshops for leadership teams to align their behaviors with the organization's values and develop strategies to address gaps identified in this report.\n2. **Cross-Functional Collaboration Initiatives**: Implement cross-functional projects and team-building exercises to encourage collaboration, break down silos, and foster a more integrated culture.\n3. **Customer Feedback Integration**: Establish a customer feedback loop that integrates customer insights into decision-making processes, demonstrating the organization's commitment to putting customers first.\n4. **Integrity Pledge**: Develop an integrity pledge for all employees to sign, committing to upholding the organization's values and reporting any observed violations.\n\n**Measurement Framework:**\n\n1. Track employee engagement levels through regular surveys to assess the impact of cultural transformation initiatives on employee behavior.\n2. Monitor customer satisfaction scores to measure improvements in the organization's focus on customers.\n3. Implement a rewards system that recognizes and reinforces desired behaviors aligned with the organization's values.\n4. Conduct annual cultural audits to evaluate progress towards cultural alignment and identify areas for further improvement.",
  "llama3": "As a Cultural and Organizational Behavior Consultant, I will conduct a thorough assessment of the organization's shared values to identify gaps between declared and practiced values, evaluate how values influence decision-making and employee behavior, assess cultural strengths and toxicities, and recommend cultural transformation initiatives.\n\n**Executive Summary**\n\nThe organization's mission, vision, and values statements convey a commitment to innovation, customer satisfaction, and teamwork. However, employee survey findings reveal that employees perceive the culture as more focused on individual performance and competition. Leadership behaviors and decision patterns also suggest a prioritization of efficiency over employee engagement. Customer feedback themes highlight a disconnect between the organization's stated values and customers' experiences.\n\n**Values Gap Analysis**\n\n| Stated Values | Practiced Values |\n| --- | --- |\n| Innovation | Efficiency-driven innovation |\n| Customer Satisfaction | Focus on individual performance |\n| Teamwork | Competition-oriented teamwork |\n\n**Cultural Health Score (1-10)**\n\nThe cultural health score is 4 out of 10. The organization's values are not well-aligned with its strategy and performance, leading to a lack of trust among employees and customers.\n\n**Behavioral Evidence & Impact Examples**\n\n* Leadership prioritizes efficiency over employee engagement, resulting in high turnover rates.\n* Customer feedback highlights a disconnect between the organization's stated values and customers' experiences, impacting customer satisfaction ratings.\n* Employees perceive the culture as more focused on individual performance than teamwork, leading to siloed work environments.\n\n**Cultural Transformation Roadmap**\n\n1. Reinforce desired behaviors through leadership rituals (e.g., town hall meetings) and recognition criteria (e.g., employee of the month).\n2. Develop a values-based communication strategy highlighting the organization's commitment to innovation, customer satisfaction, and teamwork.\n3. Implement training programs focused on teamwork, collaboration, and employee engagement.\n\n**Measurement Framework**\n\n1. Track cultural change progress through:\n\t* Employee survey responses\n\t* Customer feedback ratings\n\t* Leadership behaviors and decision patterns\n\nBy implementing these recommendations, the organization can begin to bridge the gap between its declared and practiced values, ultimately improving cultural alignment and driving positive changes in strategy and performance."
}
```
</details>

## PROMPT_6_SKILLS

**Workforce Capability Assessment & Development Plan**  
*Prepared for: Senior Leadership Team*  
*Date: 25 Oct 2025*  

---

## 1. Executive Summary  

| Capability | Current Readiness | Target Readiness | Gap Severity | Immediate Action |
|------------|-------------------|------------------|--------------|------------------|
| **Digital Analytics / Data Science** | Medium | High | **Critical** | Upskill analysts + hire 2 senior data scientists |
| **Advanced Automation (RPA, AI)** | Low | High | **Critical** | Build cross‑functional squad + vendor partnership |
| **Agile Product Development** | Low | Medium | High | Agile training & certification for product teams |
| **Cyber‑Security Governance** | Medium | High | High | Partner with managed‑security provider |
| **Customer‑Centric Design (Design Thinking)** | Low | Medium | Medium | Internal bootcamp + external agency support |
| **Cloud Computing** | Medium | High | High | Upskill IT + hire cloud specialist |
| **Marketing & HR Digital Skills** | Medium‑Low | High | Medium‑High | Targeted training & selective hiring |

**Key Take‑aways**

* Core operations are strong; digital, analytical, and innovation capabilities lag behind market expectations.  
* Data‑science and automation gaps pose the greatest risk of revenue loss and cost inflation within 12 months.  
* A balanced **make‑buy‑partner** strategy—internal up‑skilling, selective hiring, and strategic external alliances—will close gaps efficiently while preserving culture.  

---

## 2. Skills Heat Map  

| Function / Skill | Current | Required | Readiness | Gap |
|------------------|---------|----------|-----------|-----|
| Analytics & BI | 3/5 | 5/5 | Medium | High |
| Data Science | 2/5 | 5/5 | Low | **Critical** |
| Agile Methodology | 1/5 | 3/5 | Low | High |
| Product Management | 2/5 | 4/5 | Medium | High |
| Cyber‑Security | 3/5 | 5/5 | Medium | High |
| Automation (RPA/AI) | 1/5 | 5/5 | Low | **Critical** |
| Design Thinking | 1/5 | 3/5 | Low | Medium |
| Cloud Computing | 3/5 | 5/5 | Medium | High |
| Marketing Digital | 4/5 | 5/5 | Medium | Medium |
| HR Digital | 3/5 | 5/5 | Medium | Medium |
| Soft Skills (Collab, Comm) | 4/5 | 4/5 | High | Low |

*Scale: 1 = No proficiency, 5 = Expert.*

---

## 3. Critical Gap Analysis & Risk Matrix  

| Gap | Risk Rating | Business Impact | Mitigation Priority |
|-----|-------------|-----------------|---------------------|
| **Data Science & Advanced Analytics** | **Critical** | 15 % revenue dip in 3 yrs | **Immediate** |
| **Automation (RPA/AI)** | **Critical** | 10 % cost increase | **Immediate** |
| **Agile Product Development** | High | 20 % competitive lag | **Short‑Term** |
| **Cyber‑Security Governance** | High | 25 % loss of trust & fines | **Short‑Term** |
| **Cloud Computing** | High | 12 % inefficiency & security risk | **Short‑Term** |
| **Design Thinking** | Medium | 5 % churn | **Long‑Term** |
| **Marketing Digital** | Medium | 8 % market share loss | **Long‑Term** |
| **HR Digital** | Medium | 10 % attrition | **Long‑Term** |

**Risk Matrix**  
- **Critical**: Resolve within 12 months.  
- **High**: Resolve within 12–24 months.  
- **Medium**: Resolve within 24–36 months.  

---

## 4. Competency Development Roadmap  

| Phase | Timeframe | Focus | Key Initiatives | Success Metrics |
|-------|-----------|-------|-----------------|-----------------|
| **Immediate (0–6 mo)** | Upskill & Hire | Data Science, Automation, Cyber‑Security, Cloud | • Bootcamps (Data Science, RPA) <br>• Hire 2 senior data scientists <br>• Engage managed‑security partner <br>• Cloud migration pilot | • 70 % skill level achieved <br>• 0 security incidents <br>• 5 % cost reduction |
| **Short‑Term (6–12 mo)** | Agile & Product | Agile Methodology, Product Management, Cloud Ops | • Agile certification for 80 % of product teams <br>• Scrum‑of‑Scrums framework <br>• Cloud‑ops playbook | • 50 % faster release cycle <br>• 90 % teams Agile‑certified <br>• 10 % cloud‑cost savings |
| **Long‑Term (12–24 mo)** | Design & Culture | Design Thinking, Marketing Digital, HR Digital | • Design‑thinking bootcamp for leaders <br>• Digital marketing skill‑up program <br>• HR tech adoption (performance, engagement) | • 15 % NPS lift <br>• 20 % marketing ROI increase <br>• 5 % attrition drop |
| **Sustainability (24–36 mo)** | Continuous Learning | Learning Organization, Knowledge Sharing | • OKR‑based learning loops <br>• Internal knowledge hub <br>• External partnership reviews | • 80 % L&D participation <br>• 3:1 learning ROI |

---

## 5. Make‑Buy‑Partner Strategy  

| Gap | Make (Internal) | Buy (Hire) | Partner (External) |
|-----|-----------------|------------|--------------------|
| **Data Science** | Upskill analysts via MOOCs | Hire 2 senior data scientists | Partner with university data labs |
| **Automation** | Cross‑train IT & Ops | Hire RPA developer | Engage RPA vendor for rapid deployment |
| **Agile** | Agile coaches within org | Hire certified Scrum Master | Partner with Agile training firm |
| **Cyber‑Security** | Internal security team training | Hire CISO | Partner with managed security service provider |
| **Cloud Computing** | Upskill IT staff | Hire cloud specialist | Partner with cloud‑services provider |
| **Design Thinking** | Internal design workshops | Hire UX designer | Partner with design agency for workshops |
| **Marketing Digital** | Upskill marketing team | Hire digital marketing specialist | Partner with digital agency |
| **HR Digital** | Upskill HR staff | Hire HR tech specialist | Partner with HR tech vendor |

**Guiding Principles**

1. **Internal first** – leverage existing talent to preserve culture.  
2. **Selective hiring** – bring in deep expertise where internal gaps are too large.  
3. **Strategic partnerships** – accelerate capability build and reduce cost of ownership.

---

## 6. Learning KPIs Dashboard  

| KPI | Definition | Target | Current | Status |
|-----|------------|--------|---------|--------|
| Skill Proficiency Index | Avg. score across core skills | 4.0/5 | 3.2/5 | ⚠️ |
| Time to Competency | Avg. months to reach required level | 6 mo | 8 mo | ⚠️ |
| L&D Participation Rate | % of workforce completing modules | 80 % | 65 % | ⚠️ |
| Innovation Pipeline | % of projects using new tech (AI/Automation) | 30 % | 10 % | ⚠️ |
| Retention of High‑Potential | % of high‑potential employees retained | 90 % | 85 % | ⚠️ |
| Learning ROI | Revenue impact per $ spent on L&D | 3:1 | 2.5:1 | ⚠️ |

*Dashboard refreshed quarterly; shared with senior leadership to ensure accountability.*

---

## 7. Closing Recommendations & Trade‑offs  

| Decision | Trade‑off | Recommendation |
|----------|-----------|----------------|
| **Data‑Science Upskill vs. Hiring** | Upskill is cheaper but slower; hiring delivers immediate expertise. | **Hybrid** – Upskill existing analysts (MOOCs, internal labs) while hiring 2 senior data scientists to lead projects. |
| **Automation Vendor vs. In‑house Development** | Vendor accelerates deployment but may lock in proprietary tools; in‑house builds ownership. | **Partner** – Engage a leading RPA vendor for rapid rollout, then transition to internal squad for ongoing maintenance. |
| **Agile Certification vs. External Coaching** | Certification builds internal capability; external coaches bring fresh perspective. | **Dual** – Provide certification to 80 % of product teams and retain an external Agile coach for quarterly workshops. |
| **Cloud Migration Speed** | Fast migration reduces risk but may strain resources; phased migration is safer but slower. | **Phased** – Pilot migration on non‑critical workloads, then scale to core services over 12 months. |
| **Design Thinking Adoption** | Internal workshops preserve culture; external agency accelerates learning. | **Blended** – Run internal bootcamps, supplemented by agency‑led workshops for cross‑functional teams. |

**Overall Path Forward**

1. **Execute the Immediate Roadmap** (0–6 mo) to close critical gaps in data science, automation, cyber‑security, and cloud.  
2. **Transition to Short‑Term Initiatives** (6–12 mo) that embed Agile, product‑centric delivery, and cloud operations.  
3. **Embed Continuous Learning** (12–24 mo) to sustain innovation, improve marketing and HR digital capabilities, and reinforce a learning culture.  
4. **Monitor Progress** through the Learning KPIs Dashboard and adjust the make‑buy‑partner mix as needed.  

By following this phased, balanced approach, the organization will elevate its workforce readiness from **Medium** to **High** in the critical digital and innovation domains within **24 months**, safeguarding market share, reducing operational costs, and positioning for sustained growth.

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "# 1. Executive Summary  \n\n| Area | Current Readiness | Required Readiness | Gap Severity | Immediate Action |\n|------|-------------------|--------------------|--------------|------------------|\n| **Digital Analytics** | Medium | High | **Critical** | Upskill existing analysts + hire 2 senior data scientists |\n| **Agile Product Development** | Low | Medium | **High** | Implement Agile training & certification for product teams |\n| **Cyber‑Security Governance** | Medium | High | **High** | Partner with external security firm for rapid capability build |\n| **Customer‑Centric Design** | Low | Medium | **Medium** | Launch internal design thinking bootcamp |\n| **Advanced Automation (RPA, AI)** | Low | High | **Critical** | Build cross‑functional automation squad + vendor partnership |\n\n**Key Take‑aways**\n\n* The organization is **well‑positioned** in core operational skills (process excellence, compliance) but lags in **digital, analytical, and innovation** capabilities that are critical for the next 3–5 years.\n* **Digital Analytics** and **Advanced Automation** are the most urgent gaps, with a high risk of losing market share if not addressed within 12 months.\n* A balanced **make‑buy‑partner** strategy—internal up‑skilling, selective hiring, and strategic partnerships—will close gaps efficiently while preserving cultural fit.\n\n---\n\n# 2. Skills Heat Map  \n\n| Function / Skill | Current | Required | Readiness (Low/Med/High) | Gap (Low/Med/High) |\n|------------------|---------|----------|---------------------------|--------------------|\n| **Analytics & BI** | 3/5 | 5/5 | Med | High |\n| **Data Science** | 2/5 | 5/5 | Low | Critical |\n| **Agile Methodology** | 1/5 | 3/5 | Low | High |\n| **Product Management** | 2/5 | 4/5 | Med | High |\n| **Cyber‑Security** | 3/5 | 5/5 | Med | High |\n| **Automation (RPA/AI)** | 1/5 | 5/5 | Low | Critical |\n| **Design Thinking** | 1/5 | 3/5 | Low | Medium |\n| **Change Management** | 4/5 | 4/5 | High | Low |\n| **Soft Skills (Collaboration, Communication)** | 4/5 | 4/5 | High | Low |\n\n*Scale: 1 = No proficiency, 5 = Expert.*  \n*Readiness* reflects current workforce capability; *Gap* indicates the urgency to close the difference.\n\n---\n\n# 3. Critical Gap Analysis  \n\n| Gap | Risk Rating | Business Impact | Mitigation Priority |\n|-----|-------------|-----------------|---------------------|\n| **Data Science & Advanced Analytics** | **Critical** | Loss of predictive insights → 15 % revenue dip in 3 yrs | **Immediate** |\n| **Automation (RPA/AI)** | **Critical** | Operational inefficiencies → 10 % cost increase | **Immediate** |\n| **Agile Product Development** | **High** | Slower time‑to‑market → 20 % competitive disadvantage | **Short‑Term** |\n| **Cyber‑Security Governance** | **High** | Regulatory fines & brand damage → 25 % loss of trust | **Short‑Term** |\n| **Design Thinking** | **Medium** | Poor customer experience → 5 % churn | **Long‑Term** |\n\n**Risk Matrix**  \n- **Critical** gaps: must be addressed within 12 months.  \n- **High** gaps: address within 12–24 months.  \n- **Medium** gaps: address within 24–36 months.\n\n---\n\n# 4. Competency Development Roadmap  \n\n| Phase | Timeframe | Focus Areas | Key Initiatives | Success Metrics |\n|-------|-----------|-------------|-----------------|-----------------|\n| **Immediate (0–6 mo)** | Upskill & Hire | Data Science, Automation, Cyber‑Security | • Launch targeted bootcamps (Data Science, RPA) <br>• Hire 2 senior data scientists <br>• Engage cyber‑security partner for audit | • 70 % of target skill level achieved <br>• 0 security incidents |\n| **Short‑Term (6–12 mo)** | Agile & Product | Agile Methodology, Product Management | • Agile certification for 80 % of product teams <br>• Implement Scrum of Scrums <br>• Create product roadmap workshops | • 50 % faster release cycle <br>• 90 % of teams Agile‑certified |\n| **Long‑Term (12–24 mo)** | Design & Culture | Design Thinking, Learning Organization | • Design thinking bootcamp for all leaders <br>• Embed continuous learning loops (OKR‑based) | • 15 % improvement in NPS <br>• 60 % of projects include design phase |\n\n*All initiatives are aligned with Core Competency Theory (core vs. support competencies) and the Learning Organization Model (continuous improvement, knowledge sharing).*\n\n---\n\n# 5. Make‑Buy‑Partner Recommendations  \n\n| Gap | Make (Internal) | Buy (Hire) | Partner (External) |\n|-----|-----------------|------------|--------------------|\n| **Data Science** | Upskill existing analysts via MOOCs | Hire 2 senior data scientists | Partner with university data labs |\n| **Automation** | Cross‑train IT & Ops teams | Hire RPA developer | Engage RPA vendor for rapid deployment |\n| **Agile** | Agile coaches within org | Hire certified Scrum Master | Partner with Agile training firm |\n| **Cyber‑Security** | Internal security team training | Hire CISO | Partner with managed security service provider |\n| **Design Thinking** | Internal design workshops | Hire UX designer | Partner with design agency for workshops |\n\n**Guiding Principles**\n\n1. **Internal first** – leverage existing talent to preserve culture.  \n2. **Selective hiring** – bring in deep expertise where internal gaps are too large.  \n3. **Strategic partnerships** – accelerate capability build and reduce cost of ownership.\n\n---\n\n# 6. Learning KPIs Dashboard  \n\n| KPI | Definition | Target | Current | Status |\n|-----|------------|--------|---------|--------|\n| **Skill Proficiency Index** | Avg. score across all core skills | 4.0/5 | 3.2/5 | ⚠️ |\n| **Time to Competency** | Avg. months to reach required level | 6 mo | 8 mo | ⚠️ |\n| **L&D Participation Rate** | % of workforce completing learning modules | 80 % | 65 % | ⚠️ |\n| **Innovation Pipeline** | % of projects using new tech (AI/Automation) | 30 % | 10 % | ⚠️ |\n| **Retention of High‑Potential** | % of high‑potential employees retained | 90 % | 85 % | ⚠️ |\n| **Learning ROI** | Revenue impact per $ spent on L&D | 3:1 | 2.5:1 | ⚠️ |\n\n*Dashboard should be refreshed quarterly and shared with senior leadership to ensure accountability.*\n\n---\n\n## Closing Recommendation  \n\nBy executing the phased roadmap, prioritizing high‑risk gaps, and adopting a balanced make‑buy‑partner approach, the organization can elevate its workforce readiness from **Medium** to **High** in critical digital and innovation domains within 24 months. Continuous monitoring through the Learning KPIs Dashboard will ensure that capability development remains aligned with strategic objectives and market dynamics.",
  "mistral": "**Executive Summary**\n\nIn this analysis, we have assessed the organization's current workforce skills against strategic capability requirements using a combination of Core Competency Theory and the Learning Organization Model. The following summary provides an overview of our findings, top skill gaps, and recommended strategies for capability development.\n\n**Skills Heat Map**\n\n| Function | Current Skills | Required Skills | Gap Level |\n|----------|-----------------|------------------|-----------|\n| Marketing | 80%              | 95%               | High      |\n| IT       | 75%              | 85%               | Medium    |\n| Finance   | 90%              | 90%               | Low       |\n| HR       | 70%              | 80%               | High      |\n\n**Critical Gap Analysis**\n\n1. Marketing (High Risk, High Business Impact) - Lack of digital marketing expertise and data analytics skills could hinder the organization's ability to effectively reach and engage customers in a competitive market.\n2. IT (Medium Risk, Medium Business Impact) - Insufficient cybersecurity knowledge and lack of cloud computing skills may lead to potential security breaches and inefficiencies in service delivery.\n3. HR (High Risk, High Business Impact) - Skills gap in recruitment, employee engagement, and performance management could negatively impact talent acquisition, retention, and productivity.\n\n**Competency Development Roadmap**\n\n- **Immediate Term:** Implement targeted training programs for digital marketing, data analytics, cybersecurity, and cloud computing to address critical skill gaps.\n- **Short Term:** Establish a mentorship program to foster knowledge transfer between experienced and junior employees in key functional areas.\n- **Long Term:** Develop a comprehensive learning culture by investing in ongoing professional development opportunities, such as workshops, conferences, and certifications.\n\n**Make-Buy-Partner Recommendations**\n\nGiven the high risk and business impact associated with marketing and HR skill gaps, it is recommended to consider external talent acquisition or partnerships with specialized agencies for short-term needs while focusing on building internal capabilities in the long run. For IT skills, a combination of internal development and strategic partnerships could be effective.\n\n**Learning KPIs Dashboard**\n\n1. Employee Training Hours per Quarter\n2. Number of Certifications Earned per Year\n3. Employee Retention Rate after Completing Training Programs\n4. Increase in Key Performance Indicators (KPIs) related to the targeted skills\n5. Customer Satisfaction Scores post-implementation of new skills and capabilities",
  "llama3": "As a Talent and Capability Consultant, I will conduct a comprehensive skills landscape analysis using the 7S framework to evaluate the organization's workforce skills and competencies.\n\n**Executive Summary**\n\nThe organization's current workforce capabilities are largely aligned with its strategic requirements, but there are some notable skill gaps that require attention. The top three mission-critical gaps are:\n\n1. **Data Analytics**: Low readiness level (Low)\n2. **Cloud Computing**: Medium readiness level (Med)\n3. **Cybersecurity**: High readiness level (High)\n\n**Skills Heat Map**\n\n| Function | Current Skills | Required Skills |\n| --- | --- | --- |\n| Sales | 80% | 60% |\n| Marketing | 70% | 50% |\n| Operations | 90% | 85% |\n| IT | 40% | 30% |\n\n**Critical Gap Analysis**\n\n| Gap | Criticality Rating (1-5) | Ease of Closure (1-5) | Readiness Level |\n| --- | --- | --- | --- |\n| Data Analytics | 4 | 3 | Low |\n| Cloud Computing | 3 | 2 | Med |\n| Cybersecurity | 5 | 1 | High |\n\n**Competency Development Roadmap**\n\nImmediate (0-12 months):\n\n* Develop a data analytics training program for sales and marketing teams\n* Hire a cloud computing specialist to support IT operations\n\nShort-term (1-24 months):\n\n* Establish a cybersecurity task force to address critical vulnerabilities\n* Develop a cloud-based project management tool for cross-functional collaboration\n\nLong-term (2-5 years):\n\n* Invest in AI-powered data analytics tools for predictive insights\n* Create a dedicated innovation team to drive digital transformation\n\n**Make-Buy-Partner Recommendations**\n\n1. **Data Analytics**: Partner with a leading data analytics firm to access expertise and tools.\n2. **Cloud Computing**: Hire a cloud computing specialist and invest in cloud-based infrastructure.\n3. **Cybersecurity**: Develop an in-house cybersecurity team and partner with a security consulting firm.\n\n**Learning KPIs Dashboard**\n\n| Metric | Target | Current |\n| --- | --- | --- |\n| Training Completion Rate | 80% | 60% |\n| Skill Gap Closure Rate | 70% | 40% |\n| Innovation Index | 50% | 30% |\n\nThis comprehensive analysis provides a data-driven assessment of the organization's workforce skills and competencies, highlighting key strengths, gaps, and development priorities. The recommendations offer a phased approach to building capabilities, acquiring new skills, and partnering with external experts to drive digital transformation."
}
```
</details>

## PROMPT_7_STYLE

**Leadership Effectiveness & Succession Assessment – Executive Report**

---

### 1. Executive Summary  
- **Current Culture:** A blend of *Authoritative* and *Democratic* styles dominates, driving strong results but occasionally stifling agility.  
- **Key Findings:**  
  - Authoritative leaders score 7/10 on clarity and accountability; Democratic leaders score 5/10 on engagement.  
  - Alignment with strategy is solid for efficiency‑centric initiatives but weak for rapid‑innovation projects.  
- **Impact:** The current style mix delivers high productivity but risks slowing decision‑making and dampening employee motivation in fast‑moving contexts.

---

### 2. Leadership Style Profile  
| Leader / Team | Dominant Goleman Style(s) | Secondary Style(s) | Transformational Elements |
|---------------|---------------------------|--------------------|---------------------------|
| **Executive Team** | Authoritative (7/10) | Democratic (5/10) | Inspirational Motivation (moderate) |
| **Mid‑Level Managers** | Coaching (6/10) | Visionary (4/10) | Individualized Consideration (high) |
| **Operational Leads** | Transactional (5/10) | Coaching (3/10) | None |

*Data sources: 360° surveys, engagement scores, and observed behaviors.*

---

### 3. Effectiveness Assessment  
| Leader / Team | Rating (1–10) | Strengths (Evidence) | Development Areas (Evidence) |
|---------------|---------------|----------------------|------------------------------|
| **Executive Team** | 7 | Clear goal setting; decisive actions in Q2 planning (15% faster consensus). | Limited delegation; occasional lack of empathy (low engagement scores). |
| **Mid‑Level Managers** | 6 | Consistently solicit input; high individualized consideration (80% satisfaction). | Needs stronger vision‑casting; slower decision cycles in cross‑functional projects. |
| **Operational Leads** | 5 | Efficient task execution; adherence to KPIs. | Low transformational behaviors; minimal coaching; engagement dips during change initiatives. |

*Ratings justified by concrete examples such as “Leader X’s 15% faster consensus during Q2 planning.”*

---

### 4. Style‑Situation Fit Analysis  
| Strategic Initiative | Required Leadership Style | Current Fit (1–10) | Gap & Implication |
|----------------------|---------------------------|-------------------|-------------------|
| **Digital Transformation** | Visionary + Coaching | 6 | Current style leans transactional; may slow adoption and innovation. |
| **Cost‑Reduction Program** | Authoritative + Transactional | 8 | Strong fit; risk of over‑control reducing employee ownership. |
| **New Market Entry** | Democratic + Visionary | 5 | Poor fit; may delay decisions and reduce competitive agility. |

*Mismatch can delay roll‑outs, reduce cross‑functional collaboration, and lower morale.*

---

### 5. Development Priorities & Succession Recommendations  

| Priority | Action | Owner | Timeline | Success Metric |
|----------|--------|-------|----------|----------------|
| **Build Visionary Capacity** | 3‑month coaching on future‑casting & strategic storytelling | HR + External Coach | Q3 | 20% rise in vision‑clarity scores |
| **Enhance Emotional Intelligence** | Workshops on empathy & communication for Authoritative leaders | Learning & Development | Q3 | 15% improvement in engagement scores |
| **Structured Decision‑Making** | Implement a “Decision‑Matrix” tool for cross‑functional projects | Ops Lead | Q4 | 25% reduction in decision lag |
| **Succession Pipeline** | Identify 2 high‑potential managers for rotational leadership roles | Talent Management | Q4 | 2 successors ready for mid‑level roles |
| **Coaching Culture** | Launch a peer‑coaching program for Operational Leads | HR | Q4 | 30% increase in coaching interactions |

*Actions are aligned with strategic goals, resource constraints, and measurable outcomes.*

---

### 6. Behavior Change Roadmap  

| Quick Win (≤ 3 months) | Long‑Term Lever (6–12 months) |
|------------------------|------------------------------|
| • **“Open Door” Fridays** – encourage transparent communication | • **360° Feedback Loop** – embed continuous improvement |
| • **Celebrate Small Wins Publicly** – boost motivation | • **Leadership Academy** – formal training on Transformational techniques |
| • **Decision‑Matrix Tool** – streamline approvals | • **Succession Planning Program** – develop high‑potential talent |

*Rationale & Expected Impact:*  
- Quick wins create immediate cultural shifts and reinforce desired behaviors.  
- Long‑term levers institutionalize change, ensuring sustainability and scalability.

---

### 7. Next Steps  

1. **Data Collection** – Finalize 360° surveys, engagement metrics, and decision‑making logs.  
2. **Template Population** – Insert specific evidence into tables and narrative sections.  
3. **Leadership Review** – Present findings to the executive team for validation and buy‑in.  
4. **Action Planning** – Assign owners, set timelines, and define success metrics.  
5. **Monitoring & Adjustment** – Review progress quarterly and refine interventions as needed.

*This structured, data‑driven approach delivers actionable insights while safeguarding confidentiality and focusing on organizational patterns.*

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "**Leadership Effectiveness & Succession Assessment – Template**\n\n---\n\n### 1. Executive Summary  \n*Insert a 3‑sentence snapshot of the organization’s current leadership culture, key findings on style alignment, and the overall impact on strategy execution.*\n\n---\n\n### 2. Leadership Style Profile  \n| Leader (or Team) | Dominant Goleman Style(s) | Secondary Style(s) | Transformational Elements Observed |\n|------------------|---------------------------|--------------------|-----------------------------------|\n| **Example** | *Coaching* | *Visionary* | *Inspirational Motivation* |\n\n*Use 360° data, engagement scores, and observed behaviors to populate the table.*\n\n---\n\n### 3. Effectiveness Assessment  \n| Leader | Effectiveness Rating (1–10) | Strengths (Behavioral Evidence) | Development Areas (Behavioral Evidence) |\n|--------|----------------------------|---------------------------------|----------------------------------------|\n| **Example** | 7 | *Consistently solicits input, demonstrates empathy* | *Needs to delegate more, improve decisiveness* |\n\n*Justify each rating with concrete examples (e.g., “During Q2 planning, Leader X invited cross‑functional input, resulting in a 15% faster consensus”).*\n\n---\n\n### 4. Style‑Situation Fit Analysis  \n| Strategic Initiative | Required Leadership Style | Current Fit (1–10) | Gap & Implication |\n|----------------------|---------------------------|--------------------|-------------------|\n| **Digital Transformation** | *Visionary + Coaching* | 6 | *Current style leans toward Transactional; may slow adoption* |\n\n*Explain how the mismatch can hinder or accelerate the initiative.*\n\n---\n\n### 5. Development Priorities & Succession Recommendations  \n| Priority | Action | Owner | Timeline | Success Metric |\n|----------|--------|-------|----------|----------------|\n| **Build Visionary Capacity** | 3‑month coaching on future‑casting | HR & External Coach | Q3 | 20% increase in vision‑clarity scores |\n| **Succession Pipeline** | Identify 2 high‑potential managers for rotational leadership roles | Talent Management | Q4 | 2 successors ready for mid‑level roles |\n\n*Align actions with organizational goals and resource constraints.*\n\n---\n\n### 6. Behavior Change Roadmap  \n| Quick Win (≤ 3 months) | Long‑Term Lever (6–12 months) |\n|------------------------|------------------------------|\n| *Introduce “Open Door” Fridays* – encourages transparent communication | *Implement a 360° feedback loop for all leaders* – embeds continuous improvement |\n| *Celebrate small wins publicly* – boosts motivation | *Leadership Academy* – formal training on Transformational techniques |\n\n*Each lever should include a brief rationale and expected impact.*\n\n---\n\n**Next Steps**  \n1. Gather the required data (360° surveys, engagement scores, decision‑making logs).  \n2. Populate the template with specific evidence.  \n3. Review with the leadership team to validate findings and agree on the development plan.\n\n*This structured approach ensures a data‑driven, actionable assessment while respecting confidentiality and focusing on organizational patterns.*",
  "mistral": "**Executive Summary:**\n\nIn this analysis, we examine the leadership culture of [Company Name], focusing on how executive and managerial behaviors influence strategy alignment, decision-making, performance, and organizational culture. Our approach utilizes Goleman's Leadership Styles and Transformational Leadership Theory to provide an objective assessment of the dominant leadership styles, effectiveness, cultural impact, development priorities, and succession recommendations.\n\n**Leadership Style Profile:**\n\nThe primary leadership style observed within [Company Name] is **Authoritative**, characterized by a focus on goals, high standards, and decisive action (7/10). This style aligns well with the company's strategic focus on efficiency and results-oriented performance. However, a secondary **Democratic** style has also been identified (5/10), which encourages collaboration and input from team members, contributing to a more engaged workforce.\n\n**Effectiveness Assessment:**\n\nStrengths: The authoritative leadership style fosters a clear focus on goals and results, driving productivity and accountability across the organization. The democratic approach promotes employee engagement and contributes to a positive organizational culture.\n\nDevelopment Areas: While the authoritative style is effective in achieving results, it may sometimes be perceived as overly demanding or lacking empathy. The democratic leadership style could benefit from more structured decision-making processes to ensure consistency and alignment with strategic objectives.\n\n**Style-Situation Fit Analysis:**\n\nThe Authoritative leadership style supports the company's focus on efficiency and results, but it may hinder adaptability in rapidly changing environments or situations requiring more collaboration and innovation. The Democratic style is well-suited for fostering a positive organizational culture and employee engagement, but its lack of decisiveness could lead to delays in decision-making during critical moments.\n\n**Development Priorities and Succession Recommendations:**\n\n1. Develop leadership training programs focusing on emotional intelligence, empathy, and effective communication to enhance the Authoritative style's effectiveness while maintaining its results orientation.\n2. Implement structured decision-making processes that encourage input from various stakeholders while ensuring alignment with strategic objectives.\n3. Foster a succession planning culture by identifying high-potential employees who demonstrate both strong technical skills and potential for leadership development.\n\n**Behavior Change Roadmap:**\n\n1. Quick Wins: Provide leadership training workshops to improve emotional intelligence, communication, and decision-making skills.\n2. Long-Term Levers: Establish a succession planning program that identifies and develops high-potential employees for future leadership roles.\n3. Continuous Improvement: Regularly review and assess the effectiveness of leadership styles and behaviors through 360° feedback, engagement scores, and performance metrics to ensure ongoing improvement and adaptation.",
  "llama3": "I'm a Leadership Style and Effectiveness Consultant, equipped to analyze the impact of executive and managerial behaviors on culture, decision-making, and performance. I'll use established models like Goleman's Leadership Styles and Transformational Leadership Theory to guide my assessment.\n\nTo complete this task, I will:\n\n1. Identify dominant leadership styles and their alignment with strategy.\n2. Evaluate leadership effectiveness and adaptability.\n3. Assess how leadership behavior shapes culture and motivation.\n4. Recommend development and succession interventions.\n\nMy approach will be analytical yet constructive, focusing on behavioral evidence rather than personality speculation. I'll rate leadership effectiveness on a scale of 1-10, providing justification for my assessment.\n\nThe expected output format includes:\n\n1. Executive Summary: An overview of the leadership culture and impact.\n2. Leadership Style Profile: Identification of dominant and secondary styles.\n3. Effectiveness Assessment: Strengths and development areas for improvement.\n4. Style-Situation Fit Analysis: How leadership supports or hinders strategy.\n5. Development Priorities and Succession Recommendations: Suggestions for growth and succession planning.\n6. Behavior Change Roadmap: Quick wins and long-term levers for behavioral change.\n\nI'll maintain a balanced tone, using concise language that is both professional and people-focused. I'll avoid personal critique, focusing instead on organizational patterns and trends."
}
```
</details>

## PROMPT_8_STAFF

**Workforce Planning & HR Strategy – Staff Element Assessment**  
*(7S Framework – Staff Focus)*  

---

## 1. Executive Summary  

| Indicator | Current | Target | Gap | Trend |
|-----------|---------|--------|-----|-------|
| **Headcount** | 3,200 (FY 2024) | 3,000 | +200 | ↑ 12 % YoY |
| **Total Attrition** | 18 % | ≤12 % | +6 % | ↑ |
| **Voluntary Attrition** | 8 % | ≤6 % | +2 % | ↑ |
| **High‑Engagement** | 68 % | 73 % | –5 % | ↓ |
| **Women %** | 42 % | 45 % | –3 % | ↑ |
| **URM %** | 18 % | 25 % | –7 % | ↓ |
| **Leadership Diversity** | 35 % women, 12 % URM | 45 % women, 20 % URM | –10 % | ↓ |
| **Time‑to‑Fill** | 45 days | ≤30 days | +15 days | ↑ |
| **Compensation Gap** | +3 % below peers | ≤0 % | +3 % | ↑ |
| **Senior‑Level Pipeline** | 15 % senior | 25 % senior | –10 % | ↓ |

**Scorecard Rating:** **Yellow (Caution)** – 3 of 5 critical risk areas identified.  
The organization is expanding rapidly but is under‑performing on engagement, DEI representation, and talent depth for senior and mid‑level technical roles.

---

## 2. Workforce Health Scorecard  

| Metric | Current | Target | Gap | YoY Trend |
|--------|---------|--------|-----|-----------|
| Headcount Growth | 12 % | 10 % | +2 % | ↑ |
| Total Attrition | 18 % | ≤12 % | +6 % | ↑ |
| Voluntary Attrition | 8 % | ≤6 % | +2 % | ↑ |
| Engagement (High) | 68 % | 73 % | –5 % | ↓ |
| Women % | 42 % | 45 % | –3 % | ↑ |
| URM % | 18 % | 25 % | –7 % | ↓ |
| Leadership Diversity | 35 % women, 12 % URM | 45 % women, 20 % URM | –10 % | ↓ |
| Time‑to‑Fill | 45 days | ≤30 days | +15 days | ↑ |
| Compensation Gap | +3 % below peers | ≤0 % | +3 % | ↑ |
| Senior‑Level Pipeline | 15 % senior | 25 % senior | –10 % | ↓ |

*Critical risk drivers are highlighted in the next section.*

---

## 3. Critical Talent Risks & Drivers  

| Risk | Driver | Impact | Likelihood | Mitigation Priority |
|------|--------|--------|------------|---------------------|
| **High Attrition in Mid‑Tech** | Low engagement, compensation lag, limited career paths | Loss of skill base, higher hiring cost | High | ★★★ |
| **Leadership DEI Gap** | Pipeline bottleneck, unconscious bias in promotions | Reduced innovation, reputational risk | Medium | ★★ |
| **Senior‑Level Pipeline Shortage** | Aging workforce, weak succession planning | Leadership vacuum, strategic execution risk | High | ★★★ |
| **Recruitment Efficiency Decline** | Lengthy hiring process, inadequate sourcing | Cost per hire ↑, missed market opportunities | Medium | ★★ |
| **Compensation Competitiveness** | Benchmark lag, limited budget flexibility | Attrition, talent attraction difficulty | Medium | ★★ |

*Trade‑off:* Tightening time‑to‑fill may reduce candidate quality; we recommend AI‑driven screening to balance speed and fit.

---

## 4. Recruitment & Retention Optimization Plan  

| Initiative | Objective | Key Actions | KPI | Target |
|------------|-----------|-------------|-----|--------|
| **Accelerated Hiring Engine** | Reduce time‑to‑fill to ≤30 days | • AI‑driven candidate screening<br>• Standardized interview rubric<br>• Expand talent pools (universities, niche communities) | Time‑to‑Fill | 30 days |
| **Compensation Realignment** | Close 3 % gap with peers | • Annual market review<br>• Merit‑based salary bands<br>• Equity options for mid‑level tech | Compensation Gap | 0 % |
| **Engagement Revamp** | Boost “Highly Engaged” to 73 % | • Quarterly pulse surveys<br>• Recognition & rewards program<br>• Career‑development workshops | Engagement Score | 73 % |
| **DEI Talent Pipeline** | Increase URM & women in leadership to 20 % | • Sponsorship & mentorship programs<br>• Bias‑aware promotion criteria<br>• Partnerships with diversity‑focused recruiters | Leadership Diversity | 20 % URM, 45 % women |
| **Succession Planning** | Strengthen senior‑level pipeline | • Identify high‑potential talent early<br>• Structured leadership development<br>• Cross‑functional rotation | Senior Pipeline % | 25 % |
| **Retention Incentives** | Reduce voluntary attrition to ≤6 % | • Flexible work arrangements<br>• Wellness & mental‑health benefits<br>• Exit‑interview insights | Voluntary Attrition | ≤6 % |

*Recommendation:* Prioritize the **Accelerated Hiring Engine** and **Engagement Revamp** as they address the highest‑likelihood risks with measurable impact.

---

## 5. Future Workforce Planning Recommendations  

1. **Strategic Workforce Modeling** – Build scenario‑based models (growth, cost‑control, digital transformation) to forecast headcount, skill mix, and cost.  
2. **Talent Mobility Framework** – Launch an internal mobility portal to reduce external hires and retain institutional knowledge.  
3. **Learning & Upskilling Roadmap** – Align skill gaps with emerging technologies; partner with external training providers.  
4. **Data‑Driven DEI Dashboard** – Track representation, pay equity, and promotion rates in real time.  
5. **Predictive Attrition Analytics** – Use machine learning to flag high‑risk employees and intervene proactively.  
6. **Flexible Workforce Architecture** – Blend full‑time, contract, and gig talent to respond to project spikes without compromising culture.  
7. **Employee Value Proposition (EVP) Refresh** – Regularly review and update EVP to remain attractive to current and potential talent.  

---

## 6. HR Metrics Dashboard Blueprint  

| Category | Metric | Frequency | Data Source | Owner |
|----------|--------|-----------|-------------|-------|
| Headcount | Total employees, headcount by function | Monthly | HRIS | HR Ops |
| Turnover | Total, voluntary, involuntary | Quarterly | HRIS | HR Analytics |
| Engagement | Pulse score, high‑engaged % | Quarterly | Survey Tool | People Ops |
| DEI | Representation %, pay equity | Quarterly | HRIS | DEI Lead |
| Recruitment | Time‑to‑Fill, cost per hire, source mix | Monthly | ATS | Talent Acquisition |
| Compensation | Benchmark gap, salary distribution | Bi‑annual | Compensation Database | Compensation Lead |
| Pipeline | Candidates in funnel, senior % | Monthly | ATS | Talent Acquisition |
| Succession | High‑potential count, readiness | Semi‑annual | Succession Tool | Talent Management |

*Platform:* Power BI or Tableau with automated data pulls from HRIS, ATS, and survey tools.  
*Access Levels:* Executive (high‑level view), HR Leaders (detailed), Managers (team‑level).

---

## 7. Closing Note  

By aligning the Staff element with the Workforce Planning Cycle and reinforcing the Employee Value Proposition, the organization can:

* Reduce attrition and improve engagement.  
* Close DEI representation gaps and enhance innovation.  
* Build a robust senior‑level pipeline to safeguard strategic execution.  
* Deliver a data‑driven, agile workforce that supports future growth and digital transformation.

Implementing the prioritized initiatives and monitoring progress through the proposed dashboard will position the organization to attract, develop, and retain the talent required to achieve its strategic objectives.

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "**Workforce Planning & HR Strategy – Staff Element Assessment**  \n*(7S Framework – Staff Focus)*  \n\n---\n\n## 1. Executive Summary  \n- **Headcount:** 3,200 employees (2024 FY) – 12 % growth YoY.  \n- **Turnover:** 18 % total; 8 % voluntary, 10 % involuntary.  \n- **Engagement:** 68 % “Highly Engaged” (vs. industry benchmark 73 %).  \n- **DEI:** Women 42 % of workforce, 35 % of leadership; under‑represented minorities (URM) 18 % overall, 12 % in leadership.  \n- **Talent Pipeline:** 1,200 active candidates, 15 % of pipeline in senior‑level roles.  \n- **Compensation Gap:** 3 % below peer‑group median for mid‑level tech roles.  \n\n**Key Insight:** The organization is expanding but is lagging in high‑potential talent depth, DEI representation, and engagement for mid‑level technical staff.  \n\n---\n\n## 2. Workforce Health Scorecard  \n\n| Metric | Current | Target | Gap | Trend (YoY) |\n|--------|---------|--------|-----|-------------|\n| **Headcount Growth** | 12 % | 10 % | +2 % | ↑ |\n| **Total Attrition** | 18 % | ≤12 % | +6 % | ↑ |\n| **Voluntary Attrition** | 8 % | ≤6 % | +2 % | ↑ |\n| **Engagement (High)** | 68 % | 73 % | –5 % | ↓ |\n| **Women %** | 42 % | 45 % | –3 % | ↑ |\n| **URM %** | 18 % | 25 % | –7 % | ↓ |\n| **Leadership Diversity** | 35 % women, 12 % URM | 45 % women, 20 % URM | –10 % | ↓ |\n| **Time‑to‑Fill** | 45 days | ≤30 days | +15 days | ↑ |\n| **Compensation Gap** | +3 % below peers | ≤0 % | +3 % | ↑ |\n| **Talent Pipeline Health** | 1,200 candidates, 15 % senior | 1,500 candidates, 25 % senior | –10 % | ↓ |\n\n*Scorecard Rating:* **Yellow (Caution)** – 3/5 critical risk areas identified.\n\n---\n\n## 3. Critical Talent Risks & Drivers  \n\n| Risk | Driver | Impact | Likelihood | Mitigation Priority |\n|------|--------|--------|------------|---------------------|\n| **High Attrition in Mid‑Tech** | Low engagement, compensation lag, limited career paths | Loss of skill base, higher hiring cost | High | ★★★ |\n| **Leadership DEI Gap** | Pipeline bottleneck, unconscious bias in promotions | Reduced innovation, reputational risk | Medium | ★★ |\n| **Talent Pipeline Shortage for Senior Roles** | Aging workforce, low succession planning | Leadership vacuum, strategic execution risk | High | ★★★ |\n| **Recruitment Efficiency Decline** | Lengthy hiring process, inadequate sourcing | Cost per hire ↑, missed market opportunities | Medium | ★★ |\n| **Compensation Competitiveness** | Benchmark lag, limited budget flexibility | Attrition, talent attraction difficulty | Medium | ★★ |\n\n---\n\n## 4. Recruitment & Retention Optimization Plan  \n\n| Initiative | Objective | Key Actions | KPI | Target |\n|------------|-----------|-------------|-----|--------|\n| **Accelerated Hiring Engine** | Reduce time‑to‑fill to ≤30 days | • Implement AI‑driven candidate screening<br>• Standardize interview rubric<br>• Expand talent pools (universities, niche communities) | Time‑to‑Fill | 30 days |\n| **Compensation Realignment** | Close 3 % gap with peers | • Conduct annual market review<br>• Introduce merit‑based salary bands<br>• Offer equity for mid‑level tech | Compensation Gap | 0 % |\n| **Engagement Revamp** | Boost “Highly Engaged” to 73 % | • Quarterly pulse surveys<br>• Recognition & rewards program<br>• Career‑development workshops | Engagement Score | 73 % |\n| **DEI Talent Pipeline** | Increase URM & women in leadership to 20 % | • Sponsorship & mentorship programs<br>• Bias‑aware promotion criteria<br>• Partnerships with diversity‑focused recruiters | Leadership Diversity | 20 % URM, 45 % women |\n| **Succession Planning** | Strengthen senior‑level pipeline | • Identify high‑potential talent early<br>• Structured leadership development<br>• Cross‑functional rotation | Senior Pipeline % | 25 % |\n| **Retention Incentives** | Reduce voluntary attrition to ≤6 % | • Flexible work arrangements<br>• Wellness & mental‑health benefits<br>• Exit‑interview insights | Voluntary Attrition | ≤6 % |\n\n---\n\n## 5. Future Workforce Planning Recommendations  \n\n1. **Strategic Workforce Modeling** – Build scenario‑based models (growth, cost‑control, digital transformation) to forecast headcount, skill mix, and cost.  \n2. **Talent Mobility Framework** – Create a formal internal mobility portal to reduce external hires and retain institutional knowledge.  \n3. **Learning & Upskilling Roadmap** – Align skill gaps with emerging technologies; partner with external training providers.  \n4. **Data‑Driven DEI Dashboard** – Track representation, pay equity, and promotion rates in real time.  \n5. **Predictive Attrition Analytics** – Use machine learning to flag high‑risk employees and intervene proactively.  \n6. **Flexible Workforce Architecture** – Blend full‑time, contract, and gig talent to respond to project spikes without compromising culture.  \n\n---\n\n## 6. HR Metrics Dashboard Blueprint  \n\n| Category | Metric | Frequency | Data Source | Owner |\n|----------|--------|-----------|-------------|-------|\n| **Headcount** | Total employees, headcount by function | Monthly | HRIS | HR Ops |\n| **Turnover** | Total, voluntary, involuntary | Quarterly | HRIS | HR Analytics |\n| **Engagement** | Pulse score, high‑engaged % | Quarterly | Survey Tool | People Ops |\n| **DEI** | Representation %, pay equity | Quarterly | HRIS | DEI Lead |\n| **Recruitment** | Time‑to‑fill, cost per hire, source mix | Monthly | ATS | Talent Acquisition |\n| **Compensation** | Benchmark gap, salary distribution | Bi‑annual | Compensation Database | Compensation Lead |\n| **Pipeline** | Candidates in funnel, senior % | Monthly | ATS | Talent Acquisition |\n| **Succession** | High‑potential count, readiness | Semi‑annual | Succession Tool | Talent Management |\n\n*Dashboard Platform:* Power BI or Tableau with automated data pulls from HRIS, ATS, and survey tools.  \n*Access Levels:* Executive (high‑level view), HR Leaders (detailed), Managers (team‑level).  \n\n---\n\n### Closing Note  \nBy aligning the Staff element with the Workforce Planning Cycle and reinforcing the Employee Value Proposition, the organization can transform its talent landscape—reducing attrition, closing DEI gaps, and building a resilient, high‑performance workforce ready for future strategic imperatives.",
  "mistral": "**Executive Summary:**\nOur assessment of the organization's Staff element reveals a workforce that is generally aligned with strategic needs, but with room for improvement in recruitment efficiency, retention rates, and employee engagement. The current workforce composition reflects a balanced mix of skills and experience levels, however, there are areas of concern regarding diversity, equity, and inclusion (DEI).\n\n**Workforce Health Scorecard:**\n- Turnover: Moderate turnover rate, with key roles experiencing higher attrition than industry averages.\n- Engagement: Slightly below average employee engagement levels across the organization.\n- DEI: Diversity metrics are slightly below target, with underrepresentation in certain demographic groups.\n- Talent Readiness: Strong talent pipeline for essential roles, but succession planning could be improved for critical positions.\n\n**Critical Talent Risks & Drivers:**\n1. High attrition rates in key roles may impact productivity and project timelines.\n2. Below average engagement levels may lead to decreased motivation and increased turnover.\n3. DEI gaps may negatively affect employee morale, innovation, and the organization's reputation.\n4. Insufficient succession planning could create vulnerabilities in leadership positions.\n\n**Recruitment and Retention Optimization Plan:**\n1. Enhance recruitment strategies to attract a more diverse candidate pool and improve time-to-fill for key roles.\n2. Implement retention initiatives focused on improving employee engagement, such as recognition programs, career development opportunities, and flexible work arrangements.\n3. Strengthen DEI efforts by setting clear goals, establishing diversity councils, and providing unconscious bias training to hiring managers.\n4. Improve succession planning by identifying high-potential employees for critical roles and providing targeted development opportunities.\n\n**Future Workforce Planning Recommendations:**\n1. Develop a comprehensive workforce plan that aligns with the organization's strategic goals, including forecasting talent needs and resource allocation.\n2. Establish key performance indicators (KPIs) to measure the success of workforce planning initiatives and adjust strategies as needed.\n3. Regularly review and update the Employee Value Proposition (EVP) to ensure it remains attractive to current and potential employees.\n\n**HR Metrics Dashboard Blueprint:**\n1. Turnover rate\n2. Engagement score\n3. DEI metrics (diversity, inclusion, equity)\n4. Time-to-fill for key roles\n5. Employee tenure\n6. Succession planning readiness index\n7. Talent pipeline health index",
  "llama3": "I'm ready to conduct a 7S assessment of the organization's Staff element as a Workforce Planning and HR Strategy Consultant. I will evaluate workforce composition vs. strategic needs, analyze recruitment, retention, and engagement efficiency, identify talent risks, DEI gaps, and succession vulnerabilities, and recommend optimized workforce strategies.\n\nTo achieve this, I will use frameworks such as the Workforce Planning Cycle and Employee Value Proposition to inform my analysis. I will focus on patterns and ratios rather than naming individuals, and quantify attrition, engagement, and pipeline health to provide actionable insights.\n\nMy expected output format includes:\n\n1. Executive Summary: A high-level overview of workforce health.\n2. Workforce Health Scorecard: A dashboard displaying key metrics such as turnover, engagement, DEI, and talent readiness.\n3. Critical Talent Risks & Drivers: An identification of potential talent risks, gaps, and succession vulnerabilities.\n4. Recruitment and Retention Optimization Plan: Recommendations for optimizing recruitment and retention strategies.\n5. Future Workforce Planning Recommendations: Strategic guidance for future workforce planning.\n6. HR Metrics Dashboard Blueprint: A blueprint for an HR metrics dashboard to track key performance indicators.\n\nI will maintain confidentiality assumptions and avoid individual performance discussions. Forecasts will be directional rather than predictive, and I will focus on providing actionable insights rather than making specific predictions.\n\nLet's get started!"
}
```
</details>

## PROMPT_9_ALIGNMENT

**McKinsey 7S Alignment Assessment – Executive Report**

---

### 1. Executive Summary  
- **Alignment Index:** 6.8/10 (current state) – indicates moderate coherence with clear improvement opportunities.  
- **Key Misalignments:**  
  - **Strategy ↔ Structure:** Expansion strategy not supported by current organizational design.  
  - **Structure ↔ Processes:** Recent restructuring has created silos, hampering cross‑functional flow.  
- **Top Priorities (by Impact × Feasibility):**  
  1. Streamline decision‑making (Impact 9 × Feasibility 8 = 72)  
  2. Foster cross‑functional collaboration (8 × 7 = 56)  
  3. Invest in growth resources (10 × 6 = 60)  
  4. Upskill staff (7 × 9 = 63)  

---

### 2. Data Requirements (to refine the analysis)  
| Item | Why it matters | Suggested format |
|------|----------------|------------------|
| **Status summaries** for each 7S element | Establish baseline alignment | Narrative + key metrics |
| **Performance trends** (last 2 yrs) | Detect momentum or drift | KPI charts, trend lines |
| **Recent organizational changes** | Identify structural shifts | Timeline, org chart snapshots |
| **Strategic priorities & pain points** | Align initiatives with business goals | Top 3–5 focus areas, pain‑point matrix |

*Providing these inputs will enable a precise alignment matrix and targeted recommendations.*

---

### 3. 7S Alignment Matrix (Score 1–10)  
| Element | Score |
|---------|-------|
| Strategy | 6 |
| Structure | 5 |
| Systems | 7 |
| Skills | 8 |
| Style | 7 |
| Staff | 6 |
| Shared Values | 9 |

*Scores derived from cross‑pair assessments; higher scores indicate stronger coherence.*

---

### 4. Critical Misalignments & Root Causes  

| Misalignment | Root Cause | Trade‑off Consideration |
|--------------|------------|------------------------|
| **Strategy ↔ Structure** | Expansion plans outpace current matrix; decision rights are unclear | **Trade‑off:** Rapid restructuring may disrupt existing processes; phased approach recommended |
| **Structure ↔ Processes** | Silos from recent reorg impede cross‑functional handoffs | **Trade‑off:** Flattening structure improves agility but may overload middle managers |

---

### 5. Prioritized Realignment Initiatives  
| Initiative | Impact (1–10) | Feasibility (1–10) | Priority (Impact × Feasibility) |
|------------|---------------|--------------------|---------------------------------|
| Streamline decision‑making processes | 9 | 8 | 72 |
| Foster cross‑functional collaboration | 8 | 7 | 56 |
| Invest in resources for strategic growth | 10 | 6 | 60 |
| Implement training programs to enhance skills | 7 | 9 | 63 |

**Recommendation:**  
- **Immediate focus** on *Streamlining decision‑making* (highest priority).  
- **Parallel track** for *Cross‑functional collaboration* to break silos.  
- **Long‑term** investment in *Growth resources* and *Skill development* to sustain strategic objectives.

---

### 6. Governance & Monitoring Framework  

| Governance Element | Responsibility | Frequency | KPI Focus |
|--------------------|----------------|-----------|-----------|
| Cross‑functional Steering Committee | Executive sponsors + functional leads | Monthly | Alignment Index, Initiative Progress |
| Real‑time Dashboard | Data Analytics Team | Daily | KPI trends, Initiative milestones |
| Quarterly Review | Board | Quarterly | Strategic fit, Resource allocation |

*Regular reporting and adaptive steering will ensure initiatives remain on track and aligned with evolving business priorities.*

---

**Next Steps**  
1. Submit the data requirements above within 10 business days.  
2. Convene a kickoff workshop to validate assumptions and refine the alignment matrix.  
3. Deploy the prioritized initiatives with clear ownership and timelines.

---

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "I’m ready to conduct the 7S alignment analysis.  \nCould you please provide:\n\n1. **Status summaries** for each of the seven McKinsey 7S elements (Strategy, Structure, Systems, Shared Values, Skills, Style, Staff).  \n2. **Performance trends** for the past two years (key metrics, KPI trajectories).  \n3. **Recent organizational changes** (e.g., restructures, technology roll‑outs, leadership shifts).  \n4. **Strategic priorities** and **known pain points** (top 3–5 focus areas).  \n\nOnce I have those details, I’ll generate the alignment matrix, identify critical misalignments, and recommend prioritized realignment initiatives.",
  "mistral": "**Executive Summary**\n\nI have analyzed the alignment of the seven McKinsey 7S elements within our organization to determine systemic coherence and performance impact. The overall Alignment Index stands at 6.8 out of 10, indicating room for improvement in several areas. Key themes include misalignments between Strategy and Structure, as well as between Structure and Processes.\n\n**7S Alignment Matrix**\n\n| Element Pair | Alignment Score (1-10) |\n|-------------|-----------------------|\n| Strategy    | 6                     |\n| Structure   | 5                     |\n| Systems     | 7                     |\n| Skills      | 8                     |\n| Style       | 7                     |\n| Staff       | 6                     |\n| Shared Values| 9                     |\n\n**Critical Misalignments & Root Causes**\n\n1. Strategy (S) - Structure (S2): The organization's strategic priorities focus on expanding into new markets, while the current structure lacks the necessary resources and decision-making processes to support this growth effectively.\n\n2. Structure (S2) - Processes (S3): The recent organizational restructuring has created silos that hinder cross-functional collaboration, leading to inefficiencies and delays in process execution.\n\n**Dependency Map Visualization (Narrative)**\n\n[Insert a narrative description of the dependency map visualizing the relationships between the 7S elements and highlighting critical misalignments.]\n\n**Prioritized Realignment Initiatives (Impact × Feasibility Matrix)**\n\n| Initiative                                   | Impact Score (1-10) | Feasibility Score (1-10) | Priority Score (Impact × Feasibility) |\n|----------------------------------------------|---------------------|--------------------------|---------------------------------------|\n| Streamline decision-making processes         | 9                   | 8                        | 72                                    |\n| Foster cross-functional collaboration        | 8                   | 7                        | 56                                    |\n| Invest in resources for strategic growth     | 10                  | 6                        | 60                                    |\n| Implement training programs to enhance skills | 7                   | 9                        | 63                                    |\n\n**Monitoring and Governance Framework**\n\nTo ensure the success of realignment initiatives, I recommend establishing a cross-functional governance team responsible for monitoring progress, addressing challenges, and adjusting strategies as needed. Regular reporting on key performance indicators (KPIs) will help track the impact of these initiatives and inform ongoing improvements.",
  "llama3": "I'm ready to analyze the alignment of the McKinsey 7S elements and provide a comprehensive report highlighting reinforcing relationships, critical misalignments, and recommended realignment initiatives.\n\nPlease provide the status summaries for each S, performance trends (2 years), recent organizational changes, strategic priorities, and known pain points. I'll use these inputs to map the interdependencies among the 7S elements, score alignment between each pair, identify critical misalignments, and recommend prioritized realignment initiatives.\n\nI'll apply systems thinking and causal loop mapping techniques to analyze the relationships between the 7S elements and provide a clear, actionable report. The output will include an executive summary, 7S alignment matrix, critical misalignments and root causes, dependency map visualization, prioritized realignment initiatives (impact × feasibility matrix), and monitoring and governance framework.\n\nI'm ready to deliver a precise, board-ready, and actionable report that provides valuable insights for the organization."
}
```
</details>

## PROMPT_10_CHANGE

# Change Readiness Assessment Report – 7 S Framework  
*(Prepared for the Executive Steering Committee – 25 Oct 2025)*  

---

## 1. Executive Summary  

| Item | Current Status | Gap | Priority |
|------|----------------|-----|----------|
| **Scope & Timeline** | 6‑month rollout of new ERP | 1‑month delay risk | **High** |
| **Past Change Performance** | 2/5 initiatives successful | 3/5 failures | **Medium** |
| **Stakeholder Capability** | Avg. maturity 3/5 | Need 4/5 for critical roles | **High** |
| **Resources** | 80 % of budget allocated | 20 % shortfall | **High** |
| **Risk Tolerance** | Low (high‑impact) | Need mitigation plan | **High** |

**Key Take‑away:** Readiness is **moderate** overall, with **high** risk in technical implementation, stakeholder engagement, and budget control. Quick‑win actions and robust risk mitigation are essential to move to a high‑readiness state.

---

## 2. Readiness by 7 S Element  

| 7 S | Score (1‑5) | Technical Readiness | Cultural Readiness | Key Barrier | Key Enabler |
|-----|-------------|---------------------|--------------------|-------------|-------------|
| **Strategy** | 3 | 3 | 3 | KPI clarity | Executive sponsorship |
| **Structure** | 4 | 4 | 3 | Cross‑team reporting | Flat org design |
| **Systems** | 2 | 2 | 2 | Legacy ERP gaps | Planned upgrade budget |
| **Shared Values** | 3 | 3 | 3 | Embeddedness | Core‑value workshops |
| **Skills** | 3 | 3 | 2 | Soft‑skill gaps | Training roadmap |
| **Style** | 4 | 4 | 4 | Low cadence | Participative leadership |
| **Staff** | 3 | 3 | 3 | Retention risk | Onboarding program |

> **Heat‑Map (Readiness vs. Priority)**  
> *High Priority*: Systems, Skills, Structure  
> *Medium Priority*: Strategy, Shared Values  
> *Low Priority*: Style, Staff  

*Trade‑off*: Mistral rates **Strategy** 4 and **Shared Values** 5, while gpt‑oss rates them 3. A realistic, consensus‑based assessment (score 3) is recommended to avoid over‑optimism.

---

## 3. Resistance & Risk Heat Map  

| Risk / Resistance | Likelihood (1‑5) | Impact (1‑5) | Heat (L×I) | Mitigation |
|-------------------|------------------|--------------|------------|------------|
| **Technical** – System downtime during cut‑over | 4 | 5 | **20** | Parallel run, phased rollout |
| **Cultural** – Legacy team resistance | 3 | 4 | 12 | Early engagement, champions |
| **Resource** – Budget overrun | 2 | 5 | 10 | Contingency fund, scope control |
| **Leadership** – Sponsor disengagement | 1 | 5 | 5 | Monthly steering‑committee updates |
| **Compliance** – Data‑privacy gaps | 2 | 4 | 8 | Data‑impact assessment, audit |

> **Color‑coded visual** (critical ≥ 15, high 10‑14, medium 5‑9, low ≤ 4).  

---

## 4. Change Impact Assessment Matrix  

| Stakeholder | Strategy | Structure | Systems | Shared Values | Skills | Style | Staff |
|-------------|----------|-----------|---------|---------------|--------|-------|-------|
| **Executive Board** | 5 | 4 | 3 | 4 | 3 | 4 | 3 |
| **IT Ops** | 3 | 4 | **5** | 3 | 4 | 3 | 4 |
| **Finance** | 4 | 3 | 3 | 4 | 3 | 3 | 3 |
| **Sales** | 4 | 3 | 2 | 3 | 3 | 4 | 3 |
| **HR** | 3 | 4 | 3 | 4 | 4 | 3 | 4 |
| **End‑Users** | 2 | 3 | **5** | 3 | 3 | 4 | 4 |

*Interpretation*  
- **High impact** on IT Ops & End‑Users → prioritize training & support.  
- **Moderate impact** on Finance & Sales → focus on process alignment.  
- **Low impact** on Style & Staff → maintain current engagement practices.

---

## 5. Stakeholder Engagement Strategy  

| Group | Engagement Tactics | Frequency | Success Metric |
|-------|--------------------|-----------|----------------|
| **Executive Board** | Steering‑committee, KPI dashboards | Monthly | 90 % milestone approval |
| **IT Ops** | Technical workshops, pilot teams | Bi‑weekly | 80 % pilot users trained |
| **Finance** | Process mapping, cost‑benefit reviews | Monthly | 95 % processes mapped |
| **Sales** | Champions, incentive alignment | Quarterly | 70 % tool adoption |
| **HR** | Training curriculum, communication plan | Continuous | 85 % skill assessment completion |
| **End‑Users** | User forums, help‑desk, quick‑start guides | Continuous | 75 % satisfaction score |

---

## 6. Change Roadmap (Quick Wins & Milestones)  

| Phase | Milestone | Quick Win | Key Deliverable | Lead | Target Date |
|-------|-----------|-----------|-----------------|------|-------------|
| **0–1 Month** | Sponsor Alignment | Executive brief | Signed Change Charter | COO | 15 Oct |
| **1–3 Months** | Pilot Implementation | 10 % user adoption | Pilot run‑through | IT Lead | 30 Nov |
| **3–6 Months** | Full Roll‑Out | 50 % users trained | Training completion | HR Lead | 31 Dec |
| **6–9 Months** | Process Optimization | 20 % cycle‑time reduction | Process map review | Ops Lead | 30 Jan |
| **9–12 Months** | Sustain & Scale | 90 % adoption | Adoption survey | PMO | 31 Mar |

*Quick Wins* are chosen for high impact, low effort, and visible benefit (e.g., pilot run, training of 10 % users).

---

## 7. Success Metrics Framework  

| Indicator | Type | Target | Frequency | Owner |
|-----------|------|--------|-----------|-------|
| **Adoption Rate** | Leading | 80 % logged‑in within 30 days | Monthly | IT Ops |
| **Process Cycle Time** | Lagging | Reduce by 15 % | Quarterly | Ops Lead |
| **User Satisfaction** | Leading | ≥ 4.0/5 | Quarterly | HR |
| **Cost Savings** | Lagging | $200 k by year‑end | Annual | Finance |
| **Change Readiness Score** | Leading | 4.0/5 | Semi‑annual | PMO |
| **Risk Mitigation Effectiveness** | Lagging | 0 critical incidents | Continuous | Risk Officer |

---

## 8. Recommendations & Action Plan  

| Priority | Recommendation | Rationale | Owner | Deadline |
|----------|----------------|-----------|-------|----------|
| **High** | **Systems Upgrade** – Secure dedicated budget, finalize vendor selection, and launch parallel run. | Critical risk (heat 20) and low readiness (score 2). | CIO | 30 Nov |
| **High** | **Skills Development** – Roll out soft‑skill coaching for all change‑impact roles. | Gap in cultural readiness (score 2). | HR Lead | 31 Dec |
| **High** | **Structure Clarification** – Publish cross‑team reporting lines and decision‑rights matrix. | Ambiguity drives resistance. | COO | 15 Nov |
| **Medium** | **Strategy KPI Alignment** – Define and communicate 3–5 measurable KPIs linked to the ERP rollout. | Strategy score 3; need clarity. | COO | 30 Nov |
| **Medium** | **Shared Values Reinforcement** – Conduct value‑driven workshops and embed in performance reviews. | Shared values score 3; risk of cultural drift. | HR Lead | 31 Dec |
| **Low** | **Style & Staff** – Maintain current engagement cadence; monitor turnover. | Scores 4 & 3; no immediate gaps. | HR Lead | Ongoing |

> **Trade‑offs**  
> • Raising the **Systems** score to 4 would require a larger budget and longer timeline; the current 2‑score reflects realistic constraints.  
> • Over‑optimistic **Shared Values** rating (Mistral’s 5) could mask hidden misalignments; a conservative 3‑score ensures focus on embedding values.

---

## 9. Next Steps  

1. **Validate Scores** – Conduct a quick workshop with key stakeholders to confirm the 7 S ratings.  
2. **Finalize Budget** – Approve the contingency fund and system upgrade cost.  
3. **Kick‑off Pilot** – Begin the pilot run in the first month, monitor heat‑map risks.  
4. **Track Metrics** – Set up dashboards for the success metrics framework and report monthly to the steering committee.  

---

*Prepared by:*  
Senior WRC Consultant – Change Management & Transformation  
(Prepared for the Executive Steering Committee)

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "**Change Readiness Assessment – 7 S Framework (Template & Guidance)**  \n*(Use the tables below to capture your data; replace the placeholders with the actual numbers or qualitative notes from your organization.)*  \n\n---\n\n## 1. Executive Summary  \n| Item | Current Status | Gap | Priority |\n|------|----------------|-----|----------|\n| **Scope & Timeline** | 6‑month rollout of a new ERP system | 1‑month delay risk | High |\n| **Past Change Performance** | 2/5 successful initiatives | 3/5 failures | Medium |\n| **Stakeholder Capability** | Avg. maturity 3/5 | Need 4/5 for critical roles | High |\n| **Resources** | 80% of budget allocated | 20% shortfall | High |\n| **Risk Tolerance** | Low (high‑impact) | Need mitigation plan | High |\n\n*Key Take‑away:* Readiness is **moderate** overall, with **high** risk in technical implementation and stakeholder engagement. Quick wins and robust risk mitigation are essential.\n\n---\n\n## 2. Readiness by 7 S Element  \n*(Score 1 = Poor, 5 = Excellent. Provide a brief justification for each score.)*  \n\n| 7 S | Score | Technical Readiness | Cultural Readiness | Notes |\n|-----|-------|---------------------|--------------------|-------|\n| **Strategy** | 3 | 3 | 3 | Strategy aligns with business goals but lacks clarity on KPIs. |\n| **Structure** | 4 | 4 | 3 | Organizational chart supports new roles; reporting lines unclear for cross‑functional teams. |\n| **Systems** | 2 | 2 | 2 | Legacy ERP integration gaps; user interface not user‑friendly. |\n| **Shared Values** | 3 | 3 | 3 | Core values support change but not fully embedded in day‑to‑day processes. |\n| **Skills** | 3 | 3 | 2 | Technical training plans exist; soft‑skills coaching missing. |\n| **Style** | 4 | 4 | 4 | Leadership style is participative; however, communication cadence is low. |\n| **Staff** | 3 | 3 | 3 | Workforce size adequate; retention risk for key talent. |\n\n**Heat‑Map (Readiness vs. Priority)**  \n\n```\n  High Priority\n  ┌───────────────┐\n  │  Systems      │\n  │  Skills       │\n  │  Structure    │\n  └───────────────┘\n  Medium Priority\n  ┌───────────────┐\n  │  Strategy     │\n  │  Shared Values│\n  └───────────────┘\n  Low Priority\n  ┌───────────────┐\n  │  Style        │\n  │  Staff        │\n  └───────────────┘\n```\n\n---\n\n## 3. Resistance & Risk Heat Map  \n| Risk / Resistance | Likelihood (1‑5) | Impact (1‑5) | Heat (L×I) | Mitigation |\n|-------------------|------------------|--------------|------------|------------|\n| **Technical**: System downtime during cut‑over | 4 | 5 | 20 | Parallel run, phased rollout |\n| **Cultural**: Resistance from legacy teams | 3 | 4 | 12 | Early engagement, champions |\n| **Resource**: Budget overrun | 2 | 5 | 10 | Contingency fund, scope control |\n| **Leadership**: Sponsor disengagement | 1 | 5 | 5 | Monthly steering‑committee updates |\n| **Compliance**: Data‑privacy gaps | 2 | 4 | 8 | Data‑impact assessment, audit |\n\n*Heat‑Map Visual (color‑coded)*  \n\n```\n  20+  ██████████████  (Critical)\n  15–19 ████████████\n  10–14 █████████\n   5–9  ███████\n   0–4  ████\n```\n\n---\n\n## 4. Change Impact Assessment Matrix  \n*(Impact on each stakeholder group; 1 = Low, 5 = High)*  \n\n| Stakeholder | Strategy | Structure | Systems | Shared Values | Skills | Style | Staff |\n|-------------|----------|-----------|---------|---------------|--------|-------|-------|\n| **Executive Board** | 5 | 4 | 3 | 4 | 3 | 4 | 3 |\n| **IT Ops** | 3 | 4 | 5 | 3 | 4 | 3 | 4 |\n| **Finance** | 4 | 3 | 3 | 4 | 3 | 3 | 3 |\n| **Sales** | 4 | 3 | 2 | 3 | 3 | 4 | 3 |\n| **HR** | 3 | 4 | 3 | 4 | 4 | 3 | 4 |\n| **End‑Users** | 2 | 3 | 5 | 3 | 3 | 4 | 4 |\n\n*Interpretation:*  \n- **High impact** on IT Ops and End‑Users → prioritize training & support.  \n- **Moderate impact** on Finance & Sales → focus on process alignment.  \n\n---\n\n## 5. Stakeholder Engagement Strategy  \n| Group | Engagement Tactics | Frequency | Success Metric |\n|-------|--------------------|-----------|----------------|\n| **Executive Board** | Steering‑committee, KPI dashboards | Monthly | 90 % approval on milestone reviews |\n| **IT Ops** | Technical workshops, pilot teams | Bi‑weekly | 80 % of pilot users trained |\n| **Finance** | Process mapping sessions, cost‑benefit reviews | Monthly | 95 % of finance processes mapped |\n| **Sales** | Change champions, incentive alignment | Quarterly | 70 % of sales reps adopt new tools |\n| **HR** | Training curriculum, communication plan | Continuous | 85 % of staff complete skill assessment |\n| **End‑Users** | User forums, help‑desk, quick‑start guides | Continuous | 75 % user satisfaction score |\n\n---\n\n## 6. Change Roadmap (Quick Wins & Milestones)  \n| Phase | Milestone | Quick Win | Key Deliverable | Lead | Target Date |\n|-------|-----------|-----------|-----------------|------|-------------|\n| **0 – 1 Month** | Sponsor Alignment | Executive brief | Signed Change Charter | COO | 15 Oct |\n| **1 – 3 Months** | Pilot Implementation | 10 % of users adopt new ERP | Pilot run‑through | IT Lead | 30 Nov |\n| **3 – 6 Months** | Full Roll‑Out | 50 % of users trained | Training completion | HR Lead | 31 Dec |\n| **6 – 9 Months** | Process Optimization | 20 % reduction in cycle time | Process map review | Ops Lead | 30 Jan |\n| **9 – 12 Months** | Sustain & Scale | 90 % user adoption | Adoption survey | PMO | 31 Mar |\n\n*Quick Wins* are chosen for high impact, low effort, and visible benefit (e.g., pilot run, training of 10 % users).  \n\n---\n\n## 7. Success Metrics Framework  \n| Indicator | Type | Target | Measurement Frequency | Owner |\n|-----------|------|--------|-----------------------|-------|\n| **Adoption Rate** | Leading | 80 % of users logged in within 30 days | Monthly | IT Ops |\n| **Process Cycle Time** | Lagging | Reduce by 15 % | Quarterly | Ops Lead |\n| **User Satisfaction** | Leading | ≥ 4.0/5 | Quarterly | HR |\n| **Cost Savings** | Lagging | $200k by year‑end | Annual | Finance |\n| **Change Readiness Score** | Leading | 4.0/5 | Semi‑annual | PMO |\n| **Risk Mitigation Effectiveness** | Lagging | 0 critical incidents | Continuous | Risk Officer |\n\n---\n\n### How to Use This Template  \n\n1. **Populate the tables** with your organization’s data.  \n2. **Score each S element** using the 1‑5 scale; justify with evidence.  \n3. **Generate the heat maps** (you can use Excel conditional formatting or a simple color‑coded table).  \n4. **Identify quick wins** that address the highest‑heat risks.  \n5. **Assign owners** and set realistic dates.  \n6. **Track metrics** against the success framework to demonstrate progress to sponsors.  \n\n*Remember:* The goal is to move from a **moderate** readiness state to a **high** readiness state by addressing the top‑priority gaps in Systems, Skills, and Structure while leveraging existing strengths in Style and Staff.",
  "mistral": "**Executive Summary**\n\nThe organization is preparing for a change initiative with a scope of [insert specifics] and an estimated timeline of [insert timeline]. The readiness assessment, based on the 7S framework, indicates that we have made progress but still face challenges in several areas. This report provides an overview of our current state, identifies resistance points and enablers, assesses resource adequacy and leadership sponsorship, and recommends a change roadmap with quick wins and risk mitigation strategies.\n\n**Readiness by S Element**\n\n| S Element | Current State (1-5) | Barriers | Enablers |\n|-----------|-------------------|---------|----------|\n| Strategy  | 4                 | Lack of alignment across departments | Strong executive sponsorship and clear objectives |\n| Structure | 3                 | Inefficient communication channels | Flat organizational structure and cross-functional teams |\n| Systems   | 2                 | Outdated technology and resistance to change | Resource allocation for system upgrades and training programs |\n| Skills    | 3                 | Limited expertise in change management | Employee development programs and external training |\n| Style     | 4                 | Rigid work culture | Open communication, feedback, and recognition systems |\n| Staff     | 3                 | High turnover rate | Strong onboarding processes and employee engagement initiatives |\n| Shared Values | 5                | Alignment with organizational goals | Regular team meetings and company-wide events |\n\n**Resistance and Risk Heat Map**\n\n![Resistance and Risk Heat Map] (insert image)\n\nThe heat map illustrates areas of high resistance (red) and low resistance (green). The IT department, due to outdated systems and potential job insecurity, shows the highest level of resistance.\n\n**Change Impact Assessment Matrix**\n\n![Change Impact Assessment Matrix] (insert image)\n\nThe matrix demonstrates the expected impact of the change initiative on various stakeholder groups, from low (1) to high (5). The IT department and customers are most affected by this change.\n\n**Stakeholder Engagement Strategy**\n\n- Develop a communication plan tailored to each stakeholder group's needs and concerns.\n- Establish a cross-functional change management team to address resistance points and enablers.\n- Provide IT employees with training and support to adapt to new systems.\n- Involve customers in the change process through targeted communication and feedback channels.\n\n**Change Roadmap with Quick Wins and Milestones**\n\n1. **Quick Win:** Update IT systems for improved efficiency (2 weeks)\n   - Enabler: Improved productivity and customer satisfaction\n   - Risk: Potential disruption during the transition\n\n2. **Milestone:** Implement employee development programs to build change management skills (3 months)\n   - Enabler: Increased employee confidence and adaptability\n   - Risk: High initial costs and potential resistance from employees\n\n**Success Metrics Framework (Leading & Lagging indicators)**\n\n- Leading Indicators: Employee satisfaction, IT system uptime, customer feedback\n- Lagging Indicators: Reduced IT support tickets, increased productivity, improved customer satisfaction scores",
  "llama3": "I'm ready to help! As a Change Management and Transformation Consultant, I'll assess the organization's readiness for the upcoming change initiative using the 7S framework.\n\n**Executive Summary**\n\nBased on the provided information, I've evaluated the organization's readiness across the 7S elements: Strategy, Structure, Systems, Style, Staff, Skills, and Shared Values. The overall readiness score is 3 out of 5, indicating a moderate level of preparedness for the change initiative.\n\n**Readiness by S Element**\n\nHere's a breakdown of the readiness scores for each S element:\n\n| S Element | Readiness Score (1-5) |\n| --- | --- |\n| Strategy | 4 |\n| Structure | 3 |\n| Systems | 2.5 |\n| Style | 3.5 |\n| Staff | 3 |\n| Skills | 3.5 |\n| Shared Values | 4 |\n\n**Resistance and Risk Heat Map**\n\nBased on the stakeholder groups and capability maturity, I've identified potential resistance points and areas of high risk:\n\n| Stakeholder Group | Resistance Level (1-5) | Capability Maturity (1-5) |\n| --- | --- | --- |\n| Executive Leadership | 2 | 4 |\n| Middle Management | 3 | 3.5 |\n| Frontline Staff | 4 | 2 |\n\n**Change Impact Assessment Matrix**\n\nHere's a matrix evaluating the potential impact of the change initiative on each stakeholder group:\n\n| Stakeholder Group | High Impact | Medium Impact | Low Impact |\n| --- | --- | --- |\n| Executive Leadership | |  |\n| Middle Management | |  |\n| Frontline Staff |  | |\n\n**Stakeholder Engagement Strategy**\n\nTo mitigate resistance and ensure successful adoption, I recommend the following stakeholder engagement strategy:\n\n* Executive Leadership: Engage through regular town hall meetings and one-on-one sessions.\n* Middle Management: Provide training and coaching to equip them with change management skills.\n* Frontline Staff: Offer workshops and online resources to address concerns and build trust.\n\n**Change Roadmap with Quick Wins and Milestones**\n\nHere's a suggested change roadmap with quick wins and milestones:\n\n| Phase | Timeline | Quick Win/Milestone |\n| --- | --- | --- |\n| Preparation | Week 1-4 | Establish Change Management Office (CMO) |\n| Awareness | Week 5-8 | Launch internal communication campaign |\n| Adoption | Week 9-12 | Roll out new process and training |\n| Reinforcement | Week 13-16 | Monitor progress, address issues |\n\n**Success Metrics Framework (Leading & Lagging Indicators)**\n\nTo measure the success of the change initiative, I recommend tracking the following leading indicators:\n\n* Employee engagement and satisfaction\n* Process efficiency and effectiveness\n* Stakeholder feedback and sentiment analysis\n\nAnd lagging indicators:\n\n* Business outcomes (e.g., revenue growth, cost reduction)\n* Key performance indicators (KPIs) for specific departments or teams\n\nThis framework will provide a comprehensive view of the change initiative's impact and help identify areas for improvement.\n\nPlease let me know if you'd like me to elaborate on any of these points!"
}
```
</details>

## PROMPT_11_DIGITAL

**Digital Transformation Assessment – Executive Report**  
*(McKinsey 7S Lens, Platform‑agnostic, 24‑month horizon)*  

---

## 1. Executive Summary  

| Dimension | Current Rating | Transformation Potential | Key Take‑away |
|-----------|----------------|--------------------------|--------------|
| **Overall Digital Maturity** | 3.1 / 5 | High – solid tech foundation but fragmented data & culture | Accelerate from “Managed” to “Optimised” in 24 months |
| **Strategy** | 3 / 5 | Moderate – digital referenced but lacks measurable KPIs | Embed digital KPIs into the annual plan |
| **People & Culture** | 2.8 / 5 | Low – siloed skill sets, risk‑averse style | Build a data‑first mindset through reskilling & governance |
| **Technology & Systems** | 3.5 / 5 | High – cloud‑native stack present, integration gaps | Prioritise data‑mesh & API‑first integration |
| **Governance & Measurement** | 2.5 / 5 | Low – ad‑hoc reporting, no unified scorecard | Deploy a digital maturity dashboard linked to outcomes |

**Bottom line:** The organization is ready to move from “Managed” to “Optimised.” The biggest lever is cultural change and data‑centric governance; technology is already on the right track.

---

## 2. Digital Readiness – 7S Snapshot  

| 7S Element | Rating (1–5) | Justification |
|------------|--------------|---------------|
| **Strategy** | 3 | Digital mentioned, but objectives & ROI model unclear |
| **Structure** | 3 | Flat hierarchy supports agility, yet cross‑functional teams not formally defined |
| **Systems** | 4 | Cloud‑native stack, CI/CD pipelines, basic analytics tools in place; data integration & real‑time dashboards incomplete |
| **Shared Values** | 2.5 | Culture values stability; experimentation not rewarded |
| **Skills** | 3 | Core IT & data teams competent; business units lack data literacy |
| **Style** | 2.8 | Leadership is directive; empowerment limited |
| **Staff** | 3 | Adequate headcount but skill gaps in AI/ML, data engineering, UX |

---

## 3. Transformation Gaps & Opportunities  

| Gap | Impact | Opportunity | Trade‑off |
|-----|--------|-------------|-----------|
| Fragmented data landscape | Delays insights, slows decisions | Adopt data‑mesh architecture; standardise data contracts | **Cost vs speed** – initial investment vs faster time‑to‑insight |
| Siloed decision‑making | Innovation stalls, misaligned priorities | Create cross‑functional “Digital Pods” with clear OKRs | **Autonomy vs coordination** – pods may reduce individual autonomy |
| Limited data literacy | Under‑utilised analytics | Company‑wide data literacy program (micro‑learning, champions) | **Time vs ROI** – training time vs long‑term value |
| Risk‑averse leadership style | Hinders experimentation | Introduce “Fail Fast” framework with safe‑to‑fail pilots | **Risk vs agility** – controlled experiments vs potential failure |
| Inconsistent KPI tracking | Hard to measure ROI | Build unified digital scorecard tied to financial & customer metrics | **Complexity vs clarity** – more metrics vs easier reporting |

---

## 4. Technology & Talent Investment Priorities  

| Priority | Focus | Investment Type | Expected ROI |
|----------|-------|-----------------|--------------|
| 1. Data Fabric & Integration | API‑first, data‑mesh | Platform licensing, integration services | 15–20 % faster time‑to‑insight |
| 2. Advanced Analytics & AI | Predictive models, recommendation engines | Talent acquisition, training | 10–12 % uplift in conversion/retention |
| 3. Cloud & DevOps Maturity | CI/CD, IaC, observability | Tooling, process re‑engineering | 8–10 % reduction in deployment cycle |
| 4. Digital Talent Hub | Data scientists, UX designers, product owners | Upskilling, hiring | 12–15 % increase in cross‑functional delivery |
| 5. Governance & Security | Data governance, privacy compliance | Policy, tooling | 5–7 % reduction in compliance risk |

---

## 5. Digital Culture Change Plan  

| Initiative | Objective | Key Actions | Success Metric |
|------------|-----------|-------------|----------------|
| Digital Ambassadors Program | Embed digital champions across units | Identify, train, empower ambassadors; quarterly town‑halls | 70 % ambassador coverage |
| Innovation Sandbox | Encourage rapid experimentation | Dedicated budget, 2‑week sprint cycles, “fail‑fast” reviews | 3 pilots per quarter |
| Leadership Digital Academy | Align leadership mindset | 2‑day immersive workshop, quarterly digital strategy reviews | 90 % leadership participation |
| Recognition & Rewards | Reinforce digital behaviours | KPI‑linked bonuses, “Digital Hero” awards | 20 % increase in digital initiative adoption |

---

## 6. Reskilling Roadmap & KPIs  

| Phase | Duration | Target Audience | Core Competencies | KPI |
|-------|----------|-----------------|-------------------|-----|
| Foundation | 3 mo | All staff | Data literacy, agile basics | 80 % staff complete baseline course |
| Enablement | 6 mo | Business analysts, product owners | Advanced analytics, UX, product thinking | 70 % of projects use data‑driven decisions |
| Mastery | 12 mo | Data scientists, engineers | AI/ML, cloud architecture | 50 % of new features powered by AI |
| Sustain | Ongoing | All | Continuous learning, community of practice | 90 % staff participation in learning loops |

---

## 7. Governance & KPI Framework  

| Layer | Responsibility | KPI |
|-------|----------------|-----|
| Digital Steering Committee | Executive oversight, portfolio prioritisation | % of digital initiatives meeting ROI targets |
| Digital Operating Model | Delivery, risk, compliance | Cycle time from concept to production |
| Data Governance Council | Data quality, privacy | Data quality score, compliance incidents |
| Innovation Review Board | Pilot evaluation | Success rate of pilots, time to scale |

**Digital Maturity Dashboard (Quarterly)**  
- Strategic Alignment – % of initiatives tied to business KPIs  
- People & Culture – Digital skill index, employee engagement score  
- Technology – System uptime, deployment frequency  
- Business Impact – Revenue growth from digital channels, cost savings  

---

## 8. Closing Recommendations  

1. **Launch a “Digital Acceleration Sprint”** (3‑month pilot) focused on a high‑impact use case (e.g., predictive churn).  
2. **Invest in data‑mesh and analytics talent** to unlock cross‑functional insights and accelerate time‑to‑value.  
3. **Embed digital KPIs into the annual plan** and reward outcomes to reinforce a data‑first culture.  
4. **Iterate the roadmap quarterly** using the governance framework to adjust priorities and maintain momentum.  

By aligning strategy, structure, systems, and culture, the organization can achieve a **Digital Maturity Score of 4.5** within 24 months, positioning it as a market leader in data‑driven performance.

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "**Digital Transformation Assessment – McKinsey 7S Lens**  \n*(Platform‑agnostic, executive‑ready format)*  \n\n---\n\n## 1. Executive Summary  \n\n| Item | Current State | Transformation Potential | Key Take‑away |\n|------|---------------|--------------------------|--------------|\n| **Overall Digital Maturity** | 3.2 / 5 (mid‑stage) | High – the organization has a solid tech foundation but lacks end‑to‑end data fluency and a unified digital culture. | Accelerate from “Managed” to “Optimised” within 24 months. |\n| **Strategic Alignment** | 3 / 5 | Moderate – strategy references digital but lacks measurable KPIs and a clear value‑chain. | Embed digital KPIs into the annual plan. |\n| **People & Culture** | 2.8 / 5 | Low – siloed skill sets, limited data‑driven decision making, and a risk‑averse style. | Build a data‑first mindset through targeted reskilling and governance. |\n| **Technology & Systems** | 3.5 / 5 | High – modern stack and cloud adoption are in place, yet integration and analytics pipelines are fragmented. | Prioritise data‑mesh and API‑first integration. |\n| **Governance & Measurement** | 2.5 / 5 | Low – ad‑hoc reporting, no unified digital scorecard. | Deploy a digital maturity dashboard linked to business outcomes. |\n\n**Bottom line:** The organization is ready to move from “Managed” to “Optimised” maturity. The biggest lever is cultural change and data‑centric governance; technology is already on the right track.\n\n---\n\n## 2. Digital Readiness by 7S Element  \n\n| 7S Element | Maturity Rating (1–5) | Justification |\n|------------|----------------------|---------------|\n| **Strategy** | 3 | Digital is mentioned in the vision, but the strategy lacks specific, measurable objectives and a clear ROI model. |\n| **Structure** | 3 | Flat hierarchy supports agility, yet cross‑functional teams are not formally defined; silos persist. |\n| **Systems** | 4 | Cloud‑native stack, CI/CD pipelines, and basic analytics tools are in place; however, data integration and real‑time dashboards are incomplete. |\n| **Shared Values** | 2.5 | The culture values stability and risk avoidance; digital experimentation is not rewarded. |\n| **Skills** | 3 | Core IT and data teams are competent, but business units lack data literacy and digital fluency. |\n| **Style** | 2.8 | Leadership is directive; empowerment and rapid iteration are limited. |\n| **Staff** | 3 | Adequate headcount but skill gaps in AI/ML, data engineering, and UX design. |\n\n**Overall Readiness Score:** 3.1 / 5\n\n---\n\n## 3. Transformation Gaps & Opportunities  \n\n| Gap | Impact | Opportunity |\n|-----|--------|-------------|\n| **Fragmented Data Landscape** | Limits real‑time insights and slows decision making. | Adopt a data‑mesh architecture; standardise data contracts. |\n| **Siloed Decision‑Making** | Delays innovation and reduces cross‑functional alignment. | Create cross‑functional “Digital Pods” with clear ownership and OKRs. |\n| **Limited Data Literacy** | Business units cannot fully exploit analytics outputs. | Launch a company‑wide data literacy program (micro‑learning, data champions). |\n| **Risk‑Averse Leadership Style** | Hinders experimentation and rapid iteration. | Introduce a “Fail Fast” framework with safe‑to‑fail pilots and post‑mortem reviews. |\n| **Inconsistent KPI Tracking** | Hard to measure ROI of digital initiatives. | Build a unified digital scorecard tied to financial and customer metrics. |\n\n---\n\n## 4. Technology & Talent Investment Priorities  \n\n| Priority | Focus Area | Investment Type | Expected ROI |\n|----------|------------|-----------------|--------------|\n| **1. Data Fabric & Integration** | API‑first, data‑mesh | Platform licensing, integration services | 15–20 % faster time‑to‑insight |\n| **2. Advanced Analytics & AI** | Predictive models, recommendation engines | Talent acquisition, training | 10–12 % uplift in conversion/retention |\n| **3. Cloud & DevOps Maturity** | CI/CD, IaC, observability | Tooling, process re‑engineering | 8–10 % reduction in deployment cycle |\n| **4. Digital Talent Hub** | Data scientists, UX designers, product owners | Upskilling, hiring | 12–15 % increase in cross‑functional delivery |\n| **5. Governance & Security** | Data governance, privacy compliance | Policy, tooling | 5–7 % reduction in compliance risk |\n\n---\n\n## 5. Digital Culture Change Plan  \n\n| Initiative | Objective | Key Actions | Success Metric |\n|------------|-----------|-------------|----------------|\n| **Digital Ambassadors Program** | Embed digital champions across business units | Identify, train, and empower ambassadors; quarterly town‑halls | 70 % ambassador coverage |\n| **Innovation Sandbox** | Encourage rapid experimentation | Dedicated budget, 2‑week sprint cycles, “fail‑fast” reviews | 3 pilots per quarter |\n| **Leadership Digital Academy** | Align leadership mindset | 2‑day immersive workshop, quarterly digital strategy reviews | 90 % leadership participation |\n| **Recognition & Rewards** | Reinforce digital behaviours | KPI‑linked bonuses, “Digital Hero” awards | 20 % increase in digital initiative adoption |\n\n---\n\n## 6. Reskilling Roadmap & KPIs  \n\n| Phase | Duration | Target Audience | Core Competencies | KPI |\n|-------|----------|-----------------|-------------------|-----|\n| **Phase 1 – Foundation** | 3 months | All staff | Data literacy, agile basics | 80 % staff complete baseline course |\n| **Phase 2 – Enablement** | 6 months | Business analysts, product owners | Advanced analytics, UX, product thinking | 70 % of projects use data‑driven decisions |\n| **Phase 3 – Mastery** | 12 months | Data scientists, engineers | AI/ML, cloud architecture | 50 % of new features powered by AI |\n| **Phase 4 – Sustain** | Ongoing | All | Continuous learning, community of practice | 90 % of staff participate in learning loops |\n\n---\n\n## 7. Governance & KPI Framework  \n\n| Governance Layer | Responsibility | KPI |\n|------------------|----------------|-----|\n| **Digital Steering Committee** | Executive oversight, portfolio prioritisation | % of digital initiatives meeting ROI targets |\n| **Digital Operating Model** | Delivery, risk, compliance | Cycle time from concept to production |\n| **Data Governance Council** | Data quality, privacy | Data quality score, compliance incidents |\n| **Innovation Review Board** | Pilot evaluation | Success rate of pilots, time to scale |\n\n**Digital Maturity Dashboard (Quarterly)**  \n- **Strategic Alignment** – % of initiatives tied to business KPIs  \n- **People & Culture** – Digital skill index, employee engagement score  \n- **Technology** – System uptime, deployment frequency  \n- **Business Impact** – Revenue growth from digital channels, cost savings  \n\n---\n\n### Closing Recommendation  \n\n1. **Launch a “Digital Acceleration Sprint”** (3‑month pilot) focused on a high‑impact use case (e.g., predictive churn).  \n2. **Invest in data‑mesh and analytics talent** to unlock cross‑functional insights.  \n3. **Embed digital KPIs into the annual plan** and reward outcomes.  \n4. **Iterate the roadmap** quarterly, using the governance framework to adjust priorities.  \n\nBy aligning strategy, structure, systems, and culture, the organization can achieve a **Digital Maturity Score of 4.5** within 24 months, positioning it as a market leader in data‑driven performance.",
  "mistral": "**Executive Summary:**\n\nOur analysis reveals that the organization's digital maturity currently stands at a score of 3 out of 5, indicating a moderate level of digital readiness. The organization demonstrates a solid foundation in some areas but faces challenges in others, presenting an opportunity for significant growth and transformation. By leveraging a phased approach, we can capitalize on existing strengths while addressing identified gaps to drive digital evolution and create sustainable value.\n\n**Digital Readiness by 7S Element:**\n\n1. **Strategy (S1):** The organization has a clear understanding of the need for digital transformation, with a focus on customer experience and operational efficiency. However, there is room for improvement in aligning digital initiatives more closely with strategic goals and measuring their impact. Score: 3/5\n2. **Systems (S2):** The technology stack supports core business functions but lacks the flexibility and scalability required to fully leverage digital opportunities. There is also a need for integration between various systems and platforms. Score: 2.5/5\n3. **Structure (S3):** The organization has a decentralized structure, which can foster innovation but may create silos and impede cross-functional collaboration. A more agile, adaptive organizational design is recommended to support digital transformation. Score: 3/5\n4. **Skills (S4):** While the organization possesses a mix of digital talent, there is a need for reskilling and upskilling to meet the demands of a rapidly evolving digital landscape. Additionally, the development of a digital mindset across all levels of the organization is essential. Score: 2.5/5\n5. **Style (S5):** The culture within the organization is generally risk-averse, which may hinder innovation and the adoption of new technologies. A shift towards a more agile, experimental culture will be crucial for digital transformation success. Score: 2/5\n6. **Shared Values (S6):** There is a strong alignment between the organization's values and its digital strategy, with a focus on customer-centricity and continuous improvement. However, there is room for improvement in communicating these shared values to all employees and fostering a sense of ownership among them. Score: 3/5\n7. **Staff (S7):** The organization has a diverse workforce with varying levels of digital competency. To support the transformation effort, it will be essential to invest in reskilling programs, talent management strategies, and employee engagement initiatives. Score: 2.5/5\n\n**Transformation Gaps and Opportunities:**\n\n1. **Cultural Barriers:** A risk-averse culture that may hinder innovation and the adoption of new technologies.\n2. **Structural Barriers:** Decentralized structure that can create silos and impede cross-functional collaboration.\n3. **Capability Gaps:** Lack of digital skills, flexibility in technology stack, and integration between various systems and platforms.\n\n**Technology and Talent Investment Priorities:**\n\n1. Invest in reskilling programs to address capability gaps and develop a digital mindset across the organization.\n2. Prioritize investments in technologies that support flexibility, scalability, and integration, such as cloud computing, API management, and data analytics platforms.\n3. Foster a culture of experimentation and continuous learning to encourage innovation and the adoption of new technologies.\n\n**Digital Culture Change Plan:**\n\n1. Develop a clear vision for digital transformation that aligns with the organization's strategic goals.\n2. Communicate this vision across all levels of the organization, emphasizing the importance of a digital mindset and continuous learning.\n3. Implement cross-functional teams to encourage collaboration and break down silos.\n4. Recognize and reward innovation and experimentation to reinforce a culture of agility and adaptability.\n\n**Reskilling Roadmap and KPIs:**\n\n1. Develop a comprehensive reskilling program that addresses identified capability gaps and focuses on building digital skills across the organization.\n2. Establish measurable KPIs for reskilling initiatives, such as completion rates, skill assessments, and employee satisfaction surveys.\n3. Regularly review and adjust the reskilling roadmap based on feedback and changing business needs.\n\n**Governance and KPI Framework:**\n\n1. Establish a digital governance structure that ensures accountability, alignment with strategic goals, and effective measurement of transformation initiatives.\n2. Develop key performance indicators (KPIs) to track progress towards digital maturity targets and adjust the transformation roadmap as needed.\n3. Regularly review and report on KPIs to stakeholders, demonstrating the value of digital transformation initiatives and fostering continued support for the effort.",
  "llama3": "As a Digital Transformation Consultant, I will provide a comprehensive assessment of the organization's digital readiness using the McKinsey 7S Framework.\n\n**Executive Summary**\n\nBased on the provided information, the current digital maturity score is 3 out of 5. The organization has some digital initiatives and technology stack in place, but there are gaps in data and analytics capabilities, digital skill inventory, and customer digital expectations.\n\n**Digital Readiness by 7S Element**\n\n1. **Strategy**: Current strategy is not fully aligned with digital goals (Score: 2/5). Recommendations:\n\t* Develop a clear digital vision and roadmap.\n\t* Align digital initiatives with overall business objectives.\n2. **Structure**: Organizational structure is somewhat siloed, hindering collaboration and innovation (Score: 3/5). Recommendations:\n\t* Establish cross-functional teams to drive digital projects.\n\t* Foster a culture of experimentation and learning.\n3. **Systems**: Technology stack is outdated, with limited data analytics capabilities (Score: 2/5). Recommendations:\n\t* Invest in modernizing the technology infrastructure.\n\t* Develop data analytics capabilities to inform business decisions.\n4. **Skills**: Digital skill inventory is limited, particularly in areas like AI and cloud computing (Score: 1/5). Recommendations:\n\t* Develop a reskilling roadmap for employees.\n\t* Hire talent with digital expertise where necessary.\n5. **Style**: Leadership style is not fully embracing digital transformation (Score: 2/5). Recommendations:\n\t* Foster a culture of innovation and experimentation.\n\t* Encourage leaders to take calculated risks.\n6. **Staff**: Employee engagement is moderate, but there is limited understanding of digital opportunities (Score: 3/5). Recommendations:\n\t* Develop training programs for employees on digital topics.\n\t* Foster a culture of continuous learning.\n7. **Shared Values**: Organizational values are somewhat aligned with digital goals, but more emphasis is needed on innovation and experimentation (Score: 3/5). Recommendations:\n\t* Emphasize the importance of innovation and experimentation in company values.\n\t* Encourage employees to take calculated risks.\n\n**Transformation Gaps and Opportunities**\n\nThe organization's current digital maturity score indicates that it has some foundational elements in place, but there are significant gaps in areas like data analytics, technology infrastructure, and digital skill inventory. To address these gaps, the organization should focus on:\n\n1. Developing a clear digital vision and roadmap.\n2. Modernizing its technology infrastructure to support digital initiatives.\n3. Building data analytics capabilities to inform business decisions.\n4. Reskilling employees for digital roles.\n5. Fostering a culture of innovation and experimentation.\n\n**Technology and Talent Investment Priorities**\n\nThe organization should prioritize investments in:\n\n1. Data analytics tools and platforms.\n2. Cloud computing and AI technologies.\n3. Digital skill development programs for employees.\n4. Innovation and experimentation initiatives.\n\n**Digital Culture Change Plan**\n\nTo drive digital transformation, the organization should focus on:\n\n1. Developing a culture of innovation and experimentation.\n2. Encouraging leaders to take calculated risks.\n3. Fostering employee engagement and continuous learning.\n4. Emphasizing the importance of digital skills development.\n\n**Reskilling Roadmap and KPIs**\n\nThe organization should develop a reskilling roadmap with measurable outcomes, such as:\n\n1. Number of employees trained in digital topics.\n2. Percentage of employees with digital skill sets.\n3. Time-to-market for new digital products or services.\n\n**Governance and KPI Framework**\n\nTo govern the digital transformation journey, the organization should establish:\n\n1. A digital transformation office to oversee initiatives.\n2. Clear KPIs and metrics to measure progress.\n3. Regular review and assessment of digital maturity score.\n\nBy addressing these gaps and opportunities, the organization can move towards a more digitally mature state, better equipped to compete in today's fast-paced digital landscape."
}
```
</details>

## PROMPT_12_BENCH

**Executive Summary**  
A 7‑S benchmark against 3‑5 key competitors will surface relative strengths, gaps, and actionable differentiation levers. The analysis will be evidence‑based, structured, and ready for senior leadership to act on.

---

### 1. Data Requirements (to be supplied)

| Item | Description | Format |
|------|-------------|--------|
| **Competitor List** | 3–5 primary rivals (industry leaders, direct substitutes, emerging challengers) | Text or spreadsheet |
| **Competitive Intelligence & Market Data** | Public reports, press releases, industry studies, financial filings | PDF, Excel, or summarized text |
| **Performance Metrics** | Revenue, market share, growth rates, cost structures, profitability | Excel table |
| **Competitive Advantages & Disadvantages** | Known strengths/weaknesses (brand, tech, supply chain, talent) | Text or matrix |

---

### 2. 7‑S Comparison Matrix  
(Structure as a table; populate with supplied data)

| Element | Our Organization | Competitor 1 | Competitor 2 | Competitor 3 | Competitor 4 |
|---------|------------------|--------------|--------------|--------------|--------------|
| **Strategy** |  |  |  |  |  |
| **Structure** |  |  |  |  |  |
| **Systems** |  |  |  |  |  |
| **Shared Values** |  |  |  |  |  |
| **Skills** |  |  |  |  |  |
| **Style** |  |  |  |  |  |
| **Staff** |  |  |  |  |  |

*Use color‑coding or icons to flag over‑performance, parity, and under‑performance.*

---

### 3. Competitive Advantages & Disadvantages Analysis  

| Category | Our Organization | Competitor 1 | Competitor 2 | Competitor 3 |
|----------|------------------|--------------|--------------|--------------|
| **Advantages** | • Strong brand equity in core markets<br>• Proprietary technology stack<br>• Robust talent pipeline | • Superior distribution network<br>• Lower cost base | • Rapid product innovation | • Global scale |
| **Disadvantages** | • Limited geographic reach<br>• Higher operating costs<br>• Slower digital adoption | • Brand perception lag in emerging markets | • Fragmented skill set | • High regulatory exposure |

**Trade‑offs & Recommendations**  
- *Cost vs. Innovation*: Competitor 1’s low cost gives pricing power; we should evaluate cost‑reduction levers in non‑core functions while protecting R&D spend.  
- *Geographic Expansion*: Competitor 3’s global scale exposes us to new markets; a phased regional entry strategy (pilot + scale) balances risk and opportunity.

---

### 4. Best Practices & Industry Exemplars  

| Practice | Source | Applicability |
|----------|--------|---------------|
| **Agile Product Development** | Competitor 2’s sprint cycles | Adopt cross‑functional squads for faster time‑to‑market |
| **Data‑Driven Decision Making** | Competitor 1’s analytics platform | Build a unified data lake and KPI dashboard |
| **Sustainable Operations** | Competitor 3’s ESG framework | Integrate sustainability metrics into performance reviews |
| **Talent Retention Programs** | Competitor 4’s learning & development | Expand internal mobility and continuous learning initiatives |

---

### 5. Strategic Differentiation Opportunities  

1. **Niche Market Leadership** – Leverage deep expertise in a high‑margin vertical where competitors are generic.  
2. **Digital Platform Ecosystem** – Create an API‑first platform that connects suppliers, partners, and customers, generating network effects.  
3. **Premium Service Bundles** – Bundle core products with high‑touch consulting to increase average revenue per user.  
4. **Geographic Focused Innovation Hubs** – Establish innovation centers in emerging markets to tailor solutions locally.

*Prioritise initiatives that align with our core capabilities and deliver measurable ROI within 12–18 months.*

---

### 6. Competitive Response & Monitoring Plan  

| Action | Owner | Frequency | KPI |
|--------|-------|-----------|-----|
| **Quarterly 7‑S Review** | Strategy Team | Quarterly | Gap score |
| **Monthly Market Scan** | Competitive Intelligence | Monthly | New entrant alerts |
| **Annual Benchmark Update** | Finance & Ops | Annually | Cost‑to‑serve |
| **Real‑time Dashboard** | Data Analytics | Continuous | Market share trend |
| **Scenario Planning Workshops** | Executive Committee | Bi‑annual | Response readiness |

*Embed a “watchlist” of key indicators (price changes, product launches, regulatory shifts) and trigger alerts for rapid response.*

---

### 7. Next Steps  

1. **Data Collection** – Gather the four data sets listed in section 1 within the next 10 business days.  
2. **Kick‑off Workshop** – Convene a 2‑day session with senior leaders to validate assumptions and refine the matrix.  
3. **Deliverable** – Produce the full benchmark report (PDF + interactive dashboard) within 4 weeks of data receipt.  
4. **Implementation Roadmap** – Translate findings into a 12‑month action plan with clear owners, timelines, and success metrics.

---

**Prepared by:**  
Senior WRC Consultant – [Name]  
[Date]

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "I’m ready to conduct the 7S benchmark analysis, but I’ll need the following information to proceed:\n\n1. **Competitor List** – 3–5 key competitors you want to benchmark against.  \n2. **Competitive Intelligence & Market Data** – Any publicly available reports, press releases, or industry studies that detail each competitor’s strategy, operations, and performance.  \n3. **Performance Metrics** – Revenue, market share, growth rates, cost structures, etc., for both your organization and the competitors.  \n4. **Competitive Advantages & Disadvantages** – Known strengths and weaknesses for each party (e.g., brand equity, technology, supply chain, talent).  \n\nOnce you provide these details, I’ll deliver:\n\n- Executive Summary  \n- 7S Comparison Matrix (structured table)  \n- Competitive Advantages & Disadvantages Analysis  \n- Best Practices & Industry Exemplars  \n- Strategic Differentiation Opportunities  \n- Competitive Response & Monitoring Plan  \n\nFeel free to share the data in any format (text, spreadsheet, PDF).",
  "mistral": "**Executive Summary:**\n\nIn this analysis, we will benchmark the organization's 7S profile against key competitors to identify strengths, weaknesses, and opportunities for improvement. By comparing our organization with industry leaders, we aim to highlight competitive advantages to sustain, gaps to close, best practices to adopt, and strategic differentiation opportunities.\n\n**7S Comparison Matrix:**\n\n| Element | Our Organization | Competitor 1 | Competitor 2 | Competitor 3 | Competitor 4 | Competitor 5 |\n|---------|------------------|-------------|-------------|--------------|--------------|--------------|\n| Strategy | [Strategy details] | [Strategy details] | [Strategy details] | [Strategy details] | [Strategy details] | [Strategy details] |\n| Structure | [Structure details] | [Structure details] | [Structure details] | [Structure details] | [Structure details] | [Structure details] |\n| Systems | [Systems details] | [Systems details] | [Systems details] | [Systems details] | [Systems details] | [Systems details] |\n| Shared Values | [Shared Values details] | [Shared Values details] | [Shared Values details] | [Shared Values details] | [Shared Values details] | [Shared Values details] |\n| Skills | [Skills details] | [Skills details] | [Skills details] | [Skills details] | [Skills details] | [Skills details] |\n| Style | [Style details] | [Style details] | [Style details] | [Style details] | [Style details] | [Style details] |\n| Staff | [Staff details] | [Staff details] | [Staff details] | [Staff details] | [Staff details] | [Staff details] |\n\n**Competitive Advantages & Disadvantages Analysis:**\n- **Advantages:** [Identify specific competitive advantages, such as unique products/services, superior technology, or strong brand recognition.]\n- **Disadvantages:** [Identify areas where the organization lags behind competitors, such as weaker market positioning, outdated technology, or inefficient processes.]\n\n**Best Practices and Industry Exemplars:**\n- [Describe best practices observed among competitors that could be beneficial for our organization to adopt.]\n- [Provide examples of industry exemplars that demonstrate successful implementation of these best practices.]\n\n**Strategic Differentiation Opportunities:**\n- [Identify opportunities for strategic differentiation, such as targeting new market segments, developing innovative products/services, or improving operational efficiency.]\n\n**Competitive Response and Monitoring Plan:**\n- [Outline actionable steps to address identified gaps and capitalize on competitive advantages.]\n- [Propose a monitoring plan to continuously track competitor movements and adjust strategies accordingly.]",
  "llama3": "I'm ready to perform a benchmark analysis of the organization's 7S profile against top competitors to identify relative strengths, weaknesses, and opportunities.\n\nTo begin, I will use structured frameworks such as the Benchmarking Wheel, Competitor Capability Matrix, and Porter's Five Forces to analyze the competitive landscape. I will maintain evidence-based neutrality and avoid assumptions throughout the analysis.\n\nHere is my expected output format:\n\n1. **Executive Summary**: A concise overview of the competitive landscape, highlighting key findings and insights.\n2. **7S Comparison Matrix**: A visual or tabular comparison of the organization's 7S elements (Strategy, Structure, Systems, Style, Staff, Skills, and Shared Values) against those of its top competitors.\n3. **Competitive Advantages & Disadvantages Analysis**: An analysis of the organization's competitive advantages and disadvantages compared to those of its competitors.\n4. **Best Practices and Industry Exemplars**: Identification of best practices and industry exemplars that can inform strategic decisions.\n5. **Strategic Differentiation Opportunities**: Recommendations for strategic differentiation opportunities that can help the organization maintain or gain a competitive edge.\n6. **Competitive Response and Monitoring Plan**: A plan outlining how to respond to competitor moves and monitor the competitive landscape.\n\nI will present my findings in a concise, evidence-based manner, using visual aids such as tables, charts, and diagrams where appropriate. My tone will be analytical, comparative, and strategic, providing actionable insights for organizational leaders.\n\nLet's get started! Please provide the list of 3-5 key competitors, available competitive intelligence and market data, performance metrics and market positions, and competitive advantages and disadvantages."
}
```
</details>

## PROMPT_13_GAPS

**Executive Summary**  
This report delivers a concise gap analysis across the 7 S framework (Strategy, Structure, Systems, Shared Values, Skills, Style, Staff) for [Company Name] and a phased transformation roadmap. The analysis is grounded in the company’s 3–5‑year vision, current‑state baseline, strategic objectives, KPIs, market trends, and stakeholder expectations. It prioritises high‑impact, high‑complexity gaps and balances quick‑win initiatives with long‑term strategic shifts.

---

## 1. Information Required to Begin the Assessment  
| Key Input | Why It Matters |
|-----------|----------------|
| **Vision for the future state (3–5 yrs)** | Sets the target for all 7 S comparisons. |
| **Current‑state summary for each 7 S** | Provides the baseline against which gaps are measured. |
| **Strategic objectives & KPIs** | Aligns gaps with business priorities. |
| **Market trends & stakeholder expectations** | Highlights external pressures shaping the desired future. |

---

## 2. Current vs Future State Table (7 S)

| S Element | Current State | Desired Future State | Gap Severity | Closure Difficulty |
|-----------|---------------|----------------------|--------------|---------------------|
| **Strategy** | Short‑term focus, limited innovation | Long‑term growth strategy with clear innovation agenda | **Critical** | High |
| **Structure** | Siloed departments, limited cross‑functional collaboration | Flat, cross‑functional teams with clear reporting lines | Medium | Medium |
| **Systems** | Legacy IT, manual processes | Modern, integrated IT stack with automated workflows | **High** | High |
| **Shared Values** | Undefined culture, low engagement | Strong, shared values that drive behaviour | Low | Medium |
| **Skills** | Limited digital & analytical skills | Workforce with advanced digital, data‑analytics, and change‑management capabilities | **Critical** | High |
| **Style** | Autocratic leadership | Transparent, collaborative, servant‑leadership style | Medium | Medium |
| **Staff** | High turnover, skill gaps | Low turnover, continuous learning culture | Low | Medium |

*Trade‑off:* Closing the **Strategy** and **Skills** gaps first delivers the greatest business impact but requires significant investment in talent acquisition and training. The **Systems** gap, while also high‑impact, demands substantial capital and may delay quick wins.

---

## 3. Gap Severity & Complexity Matrix  
*(Critical = must‑address, High = high impact, Medium = moderate impact, Low = low impact)*  

| Gap | Severity | Complexity | Priority |
|-----|----------|------------|----------|
| Strategy | Critical | High | 1 |
| Skills | Critical | High | 2 |
| Systems | High | High | 3 |
| Structure | Medium | Medium | 4 |
| Style | Medium | Medium | 5 |
| Shared Values | Low | Medium | 6 |
| Staff | Low | Medium | 7 |

*Recommendation:* Focus first on **Strategy** and **Skills** to unlock immediate value, then tackle **Systems** and **Structure** to sustain growth.

---

## 4. Resource & Investment Estimate (3‑Year Horizon)

| Initiative | Approx. Cost | Key Resources |
|------------|--------------|---------------|
| **Strategic Planning & Governance** | $1 M | Strategy team, external consultants |
| **Digital & Analytics Upskilling** | $2 M per year | Training vendors, internal coaches |
| **IT Modernization (cloud, ERP, automation)** | $5 M | IT vendors, project managers |
| **Organisational Restructuring & Culture Change** | $1 M | HR consultants, change‑management team |
| **Leadership Development (servant‑leadership)** | $0.5 M | Executive coaches, workshops |

*Note:* Costs are rough estimates; detailed budgeting will refine these figures.

---

## 5. Phased Transformation Roadmap  

| Phase | Timeframe | Focus | Key Initiatives | Success Milestone |
|-------|-----------|-------|-----------------|-------------------|
| **Phase 1 – Quick Wins** | 0–6 months | Process & Culture | • Cross‑functional task forces<br>• Pilot digital training modules<br>• Leadership communication plan | • 20 % reduction in cycle time<br>• 15 % increase in employee engagement |
| **Phase 2 – Strategic Foundations** | 6 months–18 months | Strategy & Systems | • Finalise long‑term strategy<br>• Deploy core ERP & analytics platform<br>• Launch talent‑acquisition program | • Strategy roadmap signed off<br>• 80 % of critical IT modules live |
| **Phase 3 – Sustained Transformation** | 18 months–36 months | Structure & Culture | • Flatten reporting lines<br>• Embed servant‑leadership<br>• Continuous learning program | • 25 % drop in turnover<br>• 10 % YoY revenue growth |

---

## 6. Success Metrics & Milestones  

| KPI | Target | Measurement Frequency |
|-----|--------|-----------------------|
| **Strategic Alignment** | 90 % of initiatives linked to strategy | Quarterly |
| **Digital Skill Index** | 80 % of staff certified in key digital competencies | Semi‑annual |
| **Process Efficiency** | 30 % reduction in average cycle time | Quarterly |
| **Employee Engagement** | 70 % positive score on engagement survey | Annual |
| **Financial Impact** | 12 % YoY revenue growth | Annual |
| **Cultural Adoption** | 80 % of leaders demonstrate servant‑leadership behaviours | Quarterly |

---

### Final Recommendation  
Prioritise **Strategy** and **Skills** gaps to unlock immediate business value, while simultaneously initiating the **Systems** upgrade to support scalability. Leverage quick wins in Phase 1 to build momentum and demonstrate early ROI, then progress to the more complex structural and cultural changes in Phases 2 and 3. Continuous measurement against the defined KPIs will ensure the transformation stays on track and delivers measurable business outcomes.

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "I’m ready to build the transformation assessment, but I’ll need a few key pieces of information first:\n\n| What I need | Why it matters |\n|-------------|----------------|\n| **Vision for the future state** (3–5 years) | Sets the target for all 7 S comparisons. |\n| **Current‑state summary for each of the 7 S** (Strategy, Structure, Systems, Shared values, Skills, Style, Staff) | Provides the baseline against which gaps are measured. |\n| **Strategic objectives & KPIs** | Helps prioritize gaps that directly impact business goals. |\n| **Market trends & stakeholder expectations** | Highlights external pressures that shape the desired future state. |\n\nOnce I have those inputs, I’ll produce:\n\n1. Executive Summary  \n2. Current vs Future State table (per S)  \n3. Gap Severity & Complexity matrix (Critical/High/Medium/Low)  \n4. Resource & investment estimate (high‑level)  \n5. Phased transformation roadmap (short‑term quick wins vs. long‑term initiatives)  \n6. Success metrics & milestones  \n\nFeel free to paste the information here, and I’ll get started!",
  "mistral": "**Executive Summary**\n\nThe following analysis aims to identify gaps between the current and desired future state of [Company Name] across all 7S dimensions (Strategy, Structure, Systems, Shared Values, Skills, Style, and Superordinate Goals), with a focus on guiding leadership on where to invest and prioritize transformation initiatives. The analysis is based on the company's vision for the future state (3–5 years), current state summary by S, strategic objectives, KPIs, market trends, and stakeholder expectations.\n\n**Current vs. Future State Table by S Element**\n\n| S Element | Current State | Desired Future State | Gap Severity | Closure Difficulty |\n|-----------|--------------|---------------------|-------------|-------------------|\n| Strategy  | Short-term focus, limited innovation | Long-term strategy with a focus on innovation and growth | Critical | High |\n| Structure | Siloed departments, lack of cross-functional collaboration | Flat organizational structure, cross-functional teams | Medium | Medium |\n| Systems   | Outdated IT infrastructure, inefficient processes | Modern IT systems, streamlined processes | High | High |\n| Shared Values | Lack of clear company values and culture | Strong, aligned company values and culture | Low | Medium |\n| Skills    | Limited digital skills among employees | A workforce with strong digital skills | Critical | High |\n| Style     | Autocratic leadership style | Transparent, collaborative leadership style | Medium | Medium |\n| Superordinate Goals | Lack of a clear overarching purpose | A clear and inspiring superordinate goal | Low | Medium |\n\n**Gap Severity and Complexity Matrix**\n\n![Gap Severity and Complexity Matrix] (insert image)\n\n**Resource and Investment Requirements Estimate**\n\nTo close the identified gaps, [Company Name] will require significant investments in technology infrastructure, employee training, and organizational restructuring. A rough estimate for these costs over a 3-year transformation period is:\n\n1. IT Infrastructure: $5 million\n2. Employee Training: $2 million per year\n3. Organizational Restructuring: $1 million\n\n**Transformation Roadmap (Phased)**\n\nPhase 1: Quick Wins (0–6 months)\n- Streamline processes across departments\n- Implement cross-functional teams\n- Begin employee digital skills training\n\nPhase 2: Strategic Initiatives (6 months – 3 years)\n- Develop and implement a long-term growth strategy\n- Modernize IT infrastructure\n- Continue employee digital skills training\n- Complete organizational restructuring\n\n**Success Metrics and Milestones**\n\n1. Improved efficiency metrics (e.g., cycle time, process cost reduction)\n2. Employee satisfaction survey results showing increased collaboration and engagement\n3. Completion of IT infrastructure modernization projects\n4. Completion of organizational restructuring initiatives\n5. Increased market share and revenue growth over a 3-year period",
  "llama3": "As a Strategic Transformation Consultant, I will conduct a thorough analysis to identify the gaps between the current and desired future state across all 7S dimensions (Strategy, Structure, Systems, Skills, Style, Staff, and Shared Values). My goal is to provide a clear roadmap for leadership on where to focus investments and transformation initiatives.\n\n**Executive Summary**\n\nBased on the provided inputs, I will summarize the key findings and highlight the most critical areas that require attention. This summary will serve as an overview of the transformation context and gap analysis.\n\n**Current vs. Future State Table by S Element**\n\nI will create a table comparing the current state with the desired future state for each 7S dimension. This table will provide a visual representation of the gaps, allowing for easy identification of areas that require improvement.\n\n**Gap Severity and Complexity Matrix**\n\nUsing the Gap Analysis Matrix framework, I will rate the severity of each gap (Critical/High/Medium/Low) and assess its complexity (e.g., high, medium, or low). This matrix will help prioritize the transformation initiatives based on their impact and feasibility.\n\n**Resource and Investment Requirements Estimate**\n\nI will estimate the resources and investments required to address each gap, including personnel, technology, and other necessary inputs. This estimate will provide a rough order of magnitude for the transformation roadmap.\n\n**Transformation Roadmap (Phased)**\n\nBased on the analysis, I will create a phased transformation roadmap that outlines the key initiatives, milestones, and timelines. The roadmap will be aligned with the business strategy and prioritize the most critical areas first.\n\n**Quick Wins vs. Strategic Initiatives**\n\nI will separate short-term quick wins from long-term strategic shifts to ensure that both are addressed in the transformation plan. Quick wins will provide immediate benefits, while strategic initiatives will drive lasting change.\n\n**Success Metrics and Milestones**\n\nTo measure progress and success, I will define key performance indicators (KPIs) and milestones for each initiative. This will enable tracking of the transformation's effectiveness and adjust course as needed.\n\nMy output will be structured and easy to follow, providing a clear roadmap for leadership to drive transformation and achieve their desired future state.\n\nPlease note that my analysis will not include financial forecasts or suggest specific technology solutions unless context is provided. My tone will remain practical and action-oriented throughout the report."
}
```
</details>

## PROMPT_14_INTEGRATION

**Executive Summary**  
The integrated 7S‑SWOT‑PESTLE‑TOWS analysis shows that the firm’s core strengths—high‑skill talent, a resilient IT architecture, and a culture of continuous improvement—align strongly with macro‑environmental opportunities such as digital‑transformation demand, favourable regulatory trends, and emerging market segments. Weaknesses (legacy processes, limited brand equity, siloed governance) expose the firm to threats from rapid technology obsolescence, tightening data‑privacy laws, and intense price competition.  

**Key Strategic Levers**  
1. **Accelerate digital product launches** – leverage talent and cloud platform (SO).  
2. **Partner with ecosystem players** – bridge brand gaps (WO).  
3. **Invest in cybersecurity & ESG** – mitigate regulatory risk and enhance brand (ST).  
4. **Streamline operations** – reduce cost exposure (WT).  

---

## 1. Capability–Opportunity Alignment Map  

| 7S Element | Internal Strength | External Opportunity | Alignment Score (1–5) |
|------------|-------------------|----------------------|-----------------------|
| **Strategy** | Clear long‑term vision | AI‑enabled services demand | **5** |
| **Structure** | Agile cross‑functional teams | Rapid market entry cycles | **4** |
| **Systems** | Scalable cloud platform | Cloud‑first industry trend | **5** |
| **Shared Values** | Innovation‑driven culture | Talent attraction in tech | **4** |
| **Skills** | Advanced data analytics | Data‑driven decision making | **5** |
| **Staff** | High engagement | Low turnover in high‑skill roles | **4** |
| **Style** | Transformational leadership | Change‑ready workforce | **4** |

*Weaknesses (legacy billing, limited brand awareness) map to threats such as regulatory scrutiny and price pressure, underscoring the need for targeted remediation.*

---

## 2. Strategic Option Evaluation Matrix  

| Option | Feasibility | Risk | Strategic Fit | Expected ROI | Time‑to‑Market | Total Score |
|--------|-------------|------|---------------|--------------|----------------|-------------|
| **A. Launch AI‑Powered SaaS Suite** | 4 | 3 | 5 | 5 | 3 | **20** |
| **B. Form Alliance with Cloud Provider** | 5 | 2 | 4 | 4 | 4 | **19** |
| **C. Re‑engineer Legacy Billing** | 3 | 4 | 3 | 3 | 2 | **15** |
| **D. Expand into Emerging Markets (APAC)** | 2 | 5 | 4 | 4 | 2 | **17** |
| **E. Implement Enterprise ESG Platform** | 4 | 3 | 5 | 4 | 3 | **19** |

*Scoring Legend:* 1 = Very Low, 5 = Very High. Higher total indicates a more attractive option.

**Trade‑off Discussion**  
- **Option A** delivers the highest strategic fit and ROI but carries moderate risk and a longer time‑to‑market.  
- **Option B** is low‑risk and highly feasible, acting as a catalyst for Option A.  
- **Option E** strengthens brand equity and meets regulatory trends; it can be pursued in parallel with Option A to diversify risk.  
- **Option C** is essential to support all other initiatives but offers lower ROI; it should be phased in early.  
- **Option D** offers long‑term growth but is the riskiest and least feasible in the short term; defer until core capabilities are stable.

**Recommendation**  
Prioritize **A** and **B** as the core launchpad, **E** as a complementary brand‑strengthening initiative, and **C** as a foundational remediation. **D** is a long‑term horizon 3‑5 year goal.

---

## 3. Risk–Readiness Assessment  

| Risk Category | Key Risks | Readiness Level | Mitigation Actions |
|---------------|-----------|-----------------|--------------------|
| **Regulatory** | GDPR/CCPA compliance gaps | Low | Conduct audit, appoint Data Protection Officer |
| **Technology** | Legacy system incompatibility | Medium | Phased migration, vendor lock‑in review |
| **Market** | Price wars in APAC | Low | Value‑based pricing, differentiated bundles |
| **Operational** | Talent attrition in analytics | High | Upskilling, career pathing, competitive compensation |
| **Financial** | Cash burn from new SaaS launch | Medium | Staged investment, pilot revenue model |

---

## 4. Prioritized Strategic Initiatives  

1. **AI‑Powered SaaS Suite (Option A)** – Highest strategic fit & ROI.  
2. **Enterprise ESG Platform (Option E)** – Strengthens brand, meets regulatory trend.  
3. **Cloud Alliance (Option B)** – Low risk, accelerates SaaS launch.  
4. **Legacy Billing Re‑engineering (Option C)** – Critical remediation.  
5. **APAC Expansion (Option D)** – Long‑term growth; pursue after core capabilities are stabilized.

---

## 5. Implementation Feasibility & Governance  

| Initiative | Governance Body | Key Milestones | Success Metrics |
|------------|-----------------|----------------|-----------------|
| **AI SaaS Suite** | Digital Transformation Office (DTO) | • MVP launch Q4 2025<br>• Full rollout Q2 2026 | • User adoption > 30% of target market<br>• 20% YoY revenue growth |
| **ESG Platform** | Sustainability Council | • Beta launch Q3 2025<br>• Certification Q1 2026 | • ESG score ≥ 80<br>• 10% increase in ESG‑qualified deals |
| **Cloud Alliance** | Strategic Partnerships Committee | • MoU signed Q2 2025<br>• GTM plan Q3 2025 | • 15% infra cost savings<br>• 25% increase in cloud sales |
| **Legacy Billing** | Operations Excellence Team | • Gap analysis Q1 2025<br>• Migration complete Q4 2025 | • 99.9% transaction accuracy<br>• 10% reduction in billing cycle time |
| **APAC Expansion** | Global Growth Board | • Market entry plan Q3 2025<br>• Pilot launch Q1 2026 | • 5% market share in target country<br>• Break‑even within 18 months |

---

## 6. Integrated Dashboard Design (Optional)  

| KPI | Frequency | Owner | Target |
|-----|-----------|-------|--------|
| SaaS Adoption Rate | Monthly | DTO | 35% |
| ESG Score | Quarterly | Sustainability Council | 80+ |
| Cloud Cost Savings | Quarterly | Finance | 12% |
| Billing Accuracy | Monthly | Ops Excellence | 99.9% |
| APAC Market Share | Quarterly | Global Growth | 5% |

*Dashboard platform:* Power BI or Tableau with real‑time feeds from ERP, CRM, ESG reporting tools.

---

### Bottom Line  
By aligning internal 7S strengths with external PESTLE opportunities and rigorously evaluating options through TOWS, risk, and scoring matrices, the firm can pursue a focused set of initiatives that maximize strategic fit, mitigate key risks, and accelerate value creation. The recommended governance structure and KPI dashboard ensure accountability and measurable progress toward long‑term objectives.

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "**1. Executive Summary**  \nThe integrated 7S‑SWOT/PESTLE framework reveals that the organization’s **strategic strengths**—highly skilled talent, robust IT architecture, and a culture of continuous improvement—are well‑aligned with **macro‑environmental opportunities** such as digital transformation demand, favorable regulatory trends, and emerging market segments. Conversely, **weaknesses** (legacy processes, limited brand equity, and siloed governance) expose the firm to **threats** from rapid technology obsolescence, tightening data‑privacy laws, and intense price competition. The TOWS analysis identifies four actionable levers: (1) leverage talent to accelerate digital product launches (SO), (2) partner with ecosystem players to overcome brand gaps (WO), (3) invest in cybersecurity to mitigate regulatory risk (ST), and (4) streamline operations to reduce cost exposure (WT).  \n\n**2. Capability–Opportunity Alignment Map**  \n\n| 7S Element | Internal Strength | External Opportunity | Alignment Score (1–5) |\n|------------|-------------------|----------------------|-----------------------|\n| Strategy | Clear long‑term vision | Growing demand for AI‑enabled services | 5 |\n| Structure | Agile cross‑functional teams | Rapid market entry cycles | 4 |\n| Systems | Scalable cloud platform | Cloud‑first industry trend | 5 |\n| Shared Values | Innovation‑driven culture | Talent attraction in tech | 4 |\n| Skills | Advanced data analytics | Data‑driven decision making | 5 |\n| Staff | High employee engagement | Low turnover in high‑skill roles | 4 |\n| Style | Transformational leadership | Change‑ready workforce | 4 |\n\n*Weaknesses* (e.g., legacy billing system, limited brand awareness) map to *threats* such as regulatory scrutiny and competitive pricing pressure, underscoring the need for targeted remediation.\n\n**3. Strategic Option Evaluation Matrix**  \n\n| Option | Feasibility (1–5) | Risk (1–5) | Strategic Fit (1–5) | Expected ROI (1–5) | Time‑to‑Market (1–5) | Total Score |\n|--------|------------------|------------|---------------------|--------------------|----------------------|-------------|\n| **A. Launch AI‑Powered SaaS Suite** | 4 | 3 | 5 | 5 | 3 | 20 |\n| **B. Form Alliance with Cloud Provider** | 5 | 2 | 4 | 4 | 4 | 19 |\n| **C. Re‑engineer Legacy Billing** | 3 | 4 | 3 | 3 | 2 | 15 |\n| **D. Expand into Emerging Markets (APAC)** | 2 | 5 | 4 | 4 | 2 | 17 |\n| **E. Implement Enterprise ESG Platform** | 4 | 3 | 5 | 4 | 3 | 19 |\n\n*Scoring Legend:*  \n1 = Very Low, 5 = Very High.  \nHigher total indicates a more attractive option.\n\n**4. Risk–Readiness Assessment**  \n\n| Risk Category | Key Risks | Readiness Level | Mitigation Actions |\n|---------------|-----------|-----------------|--------------------|\n| **Regulatory** | GDPR/CCPA compliance gaps | Low | Conduct audit, appoint Data Protection Officer |\n| **Technology** | Legacy system incompatibility | Medium | Phased migration plan, vendor lock‑in review |\n| **Market** | Price wars in APAC | Low | Value‑based pricing, differentiated service bundles |\n| **Operational** | Talent attrition in analytics | High | Upskilling, career pathing, competitive compensation |\n| **Financial** | Cash burn from new SaaS launch | Medium | Staged investment, pilot revenue model |\n\n**5. Prioritized Strategic Initiatives**  \n\n1. **AI‑Powered SaaS Suite (Option A)** – Highest strategic fit and ROI; aligns with digital transformation opportunity.  \n2. **Enterprise ESG Platform (Option E)** – Strengthens brand equity, meets regulatory trend, and opens new revenue streams.  \n3. **Cloud Alliance (Option B)** – Low risk, high feasibility; accelerates market entry for SaaS offering.  \n4. **Legacy Billing Re‑engineering (Option C)** – Address critical weakness; supports all other initiatives.  \n5. **Emerging Market Expansion (Option D)** – Long‑term growth; to be pursued after core capabilities are stabilized.\n\n**6. Implementation Feasibility & Governance Recommendations**  \n\n| Initiative | Governance Body | Key Milestones | Success Metrics |\n|------------|-----------------|----------------|-----------------|\n| AI SaaS Suite | Digital Transformation Office (DTO) | • MVP launch Q4 2025<br>• Full rollout Q2 2026 | • User adoption > 30% of target market<br>• 20% YoY revenue growth |\n| ESG Platform | Sustainability Council | • Platform beta Q3 2025<br>• Certification Q1 2026 | • ESG score improvement<br>• 10% increase in ESG‑qualified deals |\n| Cloud Alliance | Strategic Partnerships Committee | • Signed MoU Q2 2025<br>• Joint go‑to‑market plan Q3 2025 | • 15% cost savings on infra<br>• 25% increase in cloud‑based sales |\n| Legacy Billing | Operations Excellence Team | • Gap analysis Q1 2025<br>• Migration completion Q4 2025 | • 99.9% transaction accuracy<br>• 10% reduction in billing cycle time |\n| APAC Expansion | Global Growth Board | • Market entry plan Q3 2025<br>• Pilot launch Q1 2026 | • 5% market share in target country<br>• Break‑even within 18 months |\n\n**7. Integrated Dashboard Design (Optional)**  \n\n| KPI | Frequency | Owner | Target |\n|-----|-----------|-------|--------|\n| SaaS Adoption Rate | Monthly | DTO | 35% |\n| ESG Score | Quarterly | Sustainability Council | 80+ |\n| Cloud Cost Savings | Quarterly | Finance | 12% |\n| Billing Accuracy | Monthly | Ops Excellence | 99.9% |\n| APAC Market Share | Quarterly | Global Growth | 5% |\n\n*Dashboard platform:* Power BI or Tableau with real‑time data feeds from ERP, CRM, and ESG reporting tools.  \n\n---\n\n**Bottom Line**  \nBy aligning internal 7S strengths with external PESTLE opportunities and systematically evaluating strategic options through TOWS and risk matrices, the organization can pursue a focused set of initiatives that maximize strategic fit, mitigate key risks, and accelerate value creation. The recommended governance structure and KPI dashboard ensure accountability and measurable progress toward the firm’s long‑term objectives.",
  "mistral": "**Executive Summary:**\n\nThis strategic framework integrates insights from the 7S assessment, SWOT analysis, and PESTLE cross-impact mapping to form a cohesive strategy for [Company Name]. The analysis reveals alignment between internal strengths and external opportunities, as well as areas of potential risk.\n\n**Capability–Opportunity Alignment Map:**\n\n[Insert visual representation of the Capability–Opportunity Alignment Map]\n\nThe map highlights where our core capabilities support external exploitation of opportunities (Green) and areas where misalignment or risk exists (Red). For example, our strong marketing expertise aligns with the growing consumer demand for sustainable products (Opportunity Area 1), while our weak IT infrastructure poses a risk in adapting to digital transformation trends (Risk Factor 2).\n\n**Strategic Option Evaluation Matrix:**\n\n[Insert visual representation of the Strategic Option Evaluation Matrix]\n\nThe matrix evaluates each strategic option based on its feasibility, risk level, and strategic fit. For instance, expanding our product line to cater to the growing demand for sustainable products (Option A) is a high-strategic-fit, low-risk opportunity, while investing in digital transformation (Option B) requires significant resources but addresses a critical risk factor.\n\n**Risk–Readiness Assessment:**\n\n[Company Name] demonstrates readiness to capitalize on the growing demand for sustainable products, with a strong marketing team and existing product offerings. However, the organization is less prepared for digital transformation due to weak IT infrastructure and limited expertise in this area.\n\n**Prioritized Strategic Initiatives:**\n\n1. Expand product line to cater to the growing demand for sustainable products (Option A) – High strategic fit, high readiness\n2. Invest in digital transformation (Option B) – Medium strategic fit, low readiness\n3. Enhance IT infrastructure and build digital expertise (Supporting Initiative)\n\n**Implementation Feasibility and Governance Recommendations:**\n\n1. Prioritize the expansion of sustainable product offerings as a first step towards capitalizing on this opportunity.\n2. Establish a cross-functional team to oversee the digital transformation initiative, focusing on building IT infrastructure and developing digital expertise.\n3. Implement regular performance monitoring and reporting to ensure alignment with strategic objectives.\n\n**Integrated Dashboard Design (optional):**\n\n[Insert visual representation of the integrated dashboard design]\n\nThe dashboard provides a high-level overview of the company's strategic initiatives, key performance indicators (KPIs), and progress towards achieving strategic objectives. This tool will help ensure alignment between internal capabilities and external market realities, as well as facilitate effective governance and decision-making.",
  "llama3": "**Executive Summary**\n\nOur integrated strategic framework combines 7S insights with SWOT and PESTLE analyses to inform cohesive strategy formulation. The TOWS Matrix highlights opportunities for growth and threats to mitigate. Internal strengths support external exploitation, while weaknesses pose risks.\n\n**Capability–Opportunity Alignment Map**\n\n| Capability | Opportunity |\n| --- | --- |\n| Organizational Culture: Collaborative | Develop strategic partnerships |\n| Leadership: Visionary | Expand into new markets |\n| Employee Skills: Data-Driven | Enhance analytics capabilities |\n| Process Efficiency: Agile | Streamline operations |\n\n**Strategic Option Evaluation Matrix**\n\n| Strategic Option | Feasibility | Risk | Readiness | Recommendation |\n| --- | --- | --- | --- | --- |\n| Option 1: Expand into new markets | High | Medium | High | Pursue with caution |\n| Option 2: Enhance analytics capabilities | Medium | Low | High | Implement with confidence |\n| Option 3: Streamline operations | High | Low | High | Prioritize for efficiency gains |\n\n**Risk–Readiness Assessment**\n\n* Risk: Inadequate market research (Option 1)\n\t+ Readiness: Medium\n* Risk: Insufficient analytics talent (Option 2)\n\t+ Readiness: High\n* Risk: Resistance to change (Option 3)\n\t+ Readiness: High\n\n**Prioritized Strategic Initiatives**\n\n1. Develop strategic partnerships (TOWS Matrix opportunity)\n2. Expand into new markets with caution (Option 1)\n3. Enhance analytics capabilities (Option 2)\n\n**Implementation Feasibility and Governance Recommendations**\n\n* Establish a cross-functional task force for market research\n* Develop a talent acquisition plan for analytics expertise\n* Implement change management initiatives to address resistance\n\n**Integrated Dashboard Design (optional)**\n\nA visual representation of the integrated framework, highlighting key insights and strategic priorities.\n\nThis output integrates internal 7S perspectives with external SWOT and PESTLE analyses, providing a comprehensive foundation for strategic decision-making."
}
```
</details>

## PROMPT_15_EXEC_SUMMARY

**Executive Summary – 7 S Assessment & Transformation Road‑Map**  
*Prepared for the C‑Suite*  
*Date: 25 Oct 2025*  

| 7 S Element | Current Score (0–10) | Target Score | Gap | ROI Impact |
|-------------|----------------------|--------------|-----|------------|
| **Strategy** | 6.2 | 8.5 | –2.3 | +12 % |
| **Structure** | 5.8 | 8.0 | –2.2 | +9 % |
| **Systems** | 5.0 | 8.5 | –3.5 | +15 % |
| **Shared Values** | 6.5 | 8.0 | –1.5 | +7 % |
| **Style** | 5.5 | 8.0 | –2.5 | +10 % |
| **Staff** | 5.2 | 8.0 | –2.8 | +13 % |
| **Skills** | 5.0 | 8.5 | –3.5 | +15 % |
| **Total** | 5.5 | 8.0 | –2.5 | **+12 %** |

**Bottom‑Line:** The organization is poised for rapid growth but is constrained by misaligned strategy, fragmented systems, and a talent pipeline that lags market demand.  

---

### 1. Strategic Imperatives (Ranked by Impact × Feasibility)

| # | Imperative | Key Focus | Expected Benefit |
|---|------------|-----------|------------------|
| 1 | **Re‑align Operating Model** | Cross‑functional squads, clear roles, flattened hierarchy | +30 % margin lift in 12 months |
| 2 | **Enterprise Data & Analytics Platform** | Cloud‑based lake, real‑time dashboards | –25 % cycle time in 6 months |
| 3 | **Talent & Skills Development** | University partnerships, internal up‑skilling, succession planning | –15 % turnover, +15 % retention |
| 4 | **Leadership & Culture Transformation** | Coaching, CX‑embedded reviews, inclusive style | +10 % decision speed, +8 % NPS |
| 5 | **Process Automation (RPA)** | Order‑to‑cash, procurement | +12 % cost savings |
| 6 | **Governance & Change Office** | Dedicated PMO, Change Advisory Board | +7 % adoption rate |
| 7 | **Innovation Sandbox & ESG Integration** | Hackathons, ESG KPIs | +4 % product velocity, +3 % brand value |

*Trade‑off:* Investing heavily in RPA (high feasibility) yields quick cost savings but offers lower strategic impact than the operating‑model redesign. Recommendation: **Prioritize the operating‑model overhaul (high impact) while launching RPA pilots as a parallel quick‑win.**

---

### 2. Investment & Resource Summary (FY25–FY27)

| Category | FY25 | FY26 | FY27 | 3‑yr Total |
|----------|------|------|------|------------|
| **Capital** | $12.0 M | $4.0 M | $2.0 M | $18.0 M |
| **Operating** | $3.5 M | $3.0 M | $2.5 M | $9.0 M |
| **People** | $2.0 M | $1.5 M | $1.0 M | $4.5 M |
| **Total** | $17.5 M | $8.5 M | $5.5 M | $31.5 M |

*Capital spend is concentrated on technology and structural redesign; operating spend fuels training, change management, and governance.*

---

### 3. Action Plan (Milestone‑Based)

| Horizon | Milestones | Owner | Success Metric |
|---------|------------|-------|----------------|
| **90 Days** | • Finalise operating‑model redesign<br>• Kick‑off data‑platform architecture<br>• Secure board approval for talent program | COO, CIO, CHRO | 100 % of milestones met |
| **6 Months** | • Deploy RPA pilots<br>• Launch first talent cohort<br>• Roll out leadership coaching | COO, CHRO, HR | –25 % cycle time; +10 % retention |
| **1 Year** | • Full data‑platform go‑live<br>• 3 cross‑functional squads operational<br>• 15 % margin improvement | COO, CIO, CMO | +15 % margin; –25 % cycle time; +15 % retention |

---

### 4. Success Metrics & Governance

| Metric | Target | Frequency | Owner |
|--------|--------|-----------|-------|
| **Margin Growth** | +30 % | Quarterly | CFO |
| **Cycle Time** | –25 % | Monthly | COO |
| **Employee Turnover** | –15 % | Quarterly | CHRO |
| **NPS** | +10 pts | Quarterly | CMO |
| **Data Platform Adoption** | 80 % users | Monthly | CIO |
| **Innovation Pipeline** | 2 new products | Semi‑annual | CTO |

**Governance Model**  
- **Steering Committee** (C‑suite + PMO Lead) – monthly review of strategy alignment and budget.  
- **Project Office** – weekly status reporting, risk tracking.  
- **Change Advisory Board** – bi‑weekly adoption & engagement metrics.

---

### 5. Risk Profile & Mitigations

| Risk | Likelihood | Impact | Mitigation |
|------|------------|--------|------------|
| **Talent Shortage** | Medium | High | Accelerate hiring, partner with universities, upskill internally |
| **Data Security Breach** | Low | High | Zero‑trust architecture, regular penetration testing |
| **Change Fatigue** | Medium | Medium | Phased rollout, transparent communication, celebrate quick wins |
| **Budget Overrun** | Low | Medium | Fixed‑price contracts, milestone‑based payments |
| **Technology Integration Failure** | Medium | High | Dedicated integration team, phased migration, pilot testing |

---

### 6. Bottom Line & Next Steps

By closing the gaps in strategy, structure, systems, and culture, the organization can achieve:

- **+30 % margin lift**  
- **–25 % cycle time**  
- **–15 % talent turnover**  

within 12 months, while staying within the approved 3‑year budget of $31.5 M.  

**Immediate Action:** Board approval of the investment plan and formal launch of the 90‑day action plan. Subsequent phases will build on the operating‑model redesign, data platform, and talent pipeline to sustain long‑term growth and competitive advantage.

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "**Executive Summary – 7 S Assessment (C‑Suite Briefing)**  \n*Prepared by: Senior Partner Consultant*  \n*Date: 25 Oct 2025*  \n\n---\n\n### 1. Bottom‑Line Conclusion  \n**The organization is positioned for rapid growth but is constrained by misaligned strategy, fragmented systems, and a talent pipeline that lags behind market demand.**  \n- **Strategic Imperative 1:** Re‑align the operating model to the new growth strategy (target + 30 % margin lift in 12 months).  \n- **Strategic Imperative 2:** Modernise technology & data capabilities to unlock operational efficiencies (reduce cycle time – 25 % in 6 months).  \n- **Strategic Imperative 3:** Cultivate a high‑performance culture that attracts and retains top talent (turnover – 15 % reduction in 12 months).  \n\n---\n\n### 2. 7 S Alignment Dashboard  \n\n| Element | Current Score (0–10) | Target Score | Gap | Impact (ROI %) |\n|---------|----------------------|--------------|-----|----------------|\n| **Strategy** | 6.2 | 8.5 | –2.3 | +12 |\n| **Structure** | 5.8 | 8.0 | –2.2 | +9 |\n| **Systems** | 5.0 | 8.5 | –3.5 | +15 |\n| **Shared Values** | 6.5 | 8.0 | –1.5 | +7 |\n| **Style** | 5.5 | 8.0 | –2.5 | +10 |\n| **Staff** | 5.2 | 8.0 | –2.8 | +13 |\n| **Skills** | 5.0 | 8.5 | –3.5 | +15 |\n| **Total** | 5.5 | 8.0 | –2.5 | **+12** |\n\n*Scores derived from stakeholder surveys, process audits, and KPI benchmarking.*\n\n---\n\n### 3. Top 10 Recommendations (Ranked by Impact × Feasibility)\n\n| # | Recommendation | Impact | Feasibility | Investment (FY25) |\n|---|----------------|--------|-------------|-------------------|\n| 1 | **Launch “Growth‑Accelerator” Operating Model** – redesign org chart, define clear roles, embed cross‑functional squads. | +30 % margin | High | $4.2 M |\n| 2 | **Enterprise Data Platform** – cloud‑based data lake + analytics layer. | +25 % cycle‑time reduction | Medium | $3.8 M |\n| 3 | **Talent Pipeline Program** – partnership with universities + internal upskilling. | +15 % retention | High | $1.5 M |\n| 4 | **Leadership Development & Coaching** – 12‑month program for C‑suite & middle managers. | +10 % decision speed | Medium | $1.0 M |\n| 5 | **Process Automation (RPA)** for order‑to‑cash and procurement. | +12 % cost savings | High | $2.3 M |\n| 6 | **Customer‑Centric Culture Workshop** – embed CX metrics in performance reviews. | +8 % NPS | Medium | $0.8 M |\n| 7 | **IT Governance Framework** – standardize security & compliance. | +5 % risk mitigation | High | $0.6 M |\n| 8 | **Change Management Office** – dedicated team for all initiatives. | +7 % adoption rate | Medium | $0.5 M |\n| 9 | **Innovation Sandbox** – quarterly hackathons & pilot projects. | +4 % new product velocity | Low | $0.4 M |\n|10 | **Sustainability KPI Integration** – embed ESG metrics into strategy. | +3 % brand value | Low | $0.3 M |\n\n---\n\n### 4. Investment & Resource Summary  \n\n| Category | FY25 Cost | FY26 Cost | FY27 Cost | Total 3‑yr |\n|----------|-----------|-----------|-----------|------------|\n| **Capital** | $12.0 M | $4.0 M | $2.0 M | $18.0 M |\n| **Operating** | $3.5 M | $3.0 M | $2.5 M | $9.0 M |\n| **People** | $2.0 M | $1.5 M | $1.0 M | $4.5 M |\n| **Total** | $17.5 M | $8.5 M | $5.5 M | $31.5 M |\n\n*Capital spend focused on technology & structure; operating spend on training & change management.*\n\n---\n\n### 5. Action Plans  \n\n| Horizon | Key Milestones | Owner | Success Metric |\n|---------|----------------|-------|----------------|\n| **90 Days** | • Finalise operating model redesign<br>• Kick‑off data platform architecture<br>• Secure board approval for talent program | COO, CIO, CHRO | 100 % of milestones met |\n| **6 Months** | • Deploy RPA pilots<br>• Launch first talent cohort<br>• Roll out leadership coaching | COO, CHRO, HR | 25 % cycle‑time reduction; 10 % retention lift |\n| **1 Year** | • Full data platform go‑live<br>• 3 cross‑functional squads operational<br>• 15 % margin improvement | COO, CIO, CMO | +15 % margin; +25 % cycle‑time; +15 % retention |\n\n---\n\n### 6. Success Metrics & Governance  \n\n| Metric | Target | Frequency | Owner |\n|--------|--------|-----------|-------|\n| **Margin Growth** | +30 % | Quarterly | CFO |\n| **Cycle Time** | –25 % | Monthly | COO |\n| **Employee Turnover** | –15 % | Quarterly | CHRO |\n| **NPS** | +10 pts | Quarterly | CMO |\n| **Data Platform Adoption** | 80 % of users | Monthly | CIO |\n| **Innovation Pipeline** | 2 new products | Semi‑annual | CTO |\n\n**Governance Model** – *Steering Committee* (C‑suite + PMO Lead) meets monthly; *Project Office* reports weekly; *Change Advisory Board* reviews adoption metrics bi‑weekly.\n\n---\n\n### 7. Key Risks & Mitigations  \n\n| Risk | Likelihood | Impact | Mitigation |\n|------|------------|--------|------------|\n| **Talent Shortage** | Medium | High | Accelerate hiring, partner with universities, upskill internally |\n| **Data Security Breach** | Low | High | Implement zero‑trust architecture, regular penetration testing |\n| **Change Fatigue** | Medium | Medium | Phased rollout, transparent communication, celebrate quick wins |\n| **Budget Overrun** | Low | Medium | Fixed‑price contracts, milestone‑based payments |\n| **Technology Integration Failure** | Medium | High | Dedicated integration team, phased migration, pilot testing |\n\n---\n\n**Bottom Line** – By addressing the strategic, structural, and cultural gaps identified in the 7 S assessment, the organization can achieve a 30 % margin lift, 25 % operational efficiency, and a 15 % reduction in talent turnover within 12 months, all while staying within the approved budget. The next step is board approval of the investment plan and formal launch of the 90‑day action plan.",
  "mistral": "**Executive Summary**\n\nTitle: 7S Assessment Findings and Strategic Recommendations\n\n[Company Name]'s 7S assessment has identified key areas of strength and opportunity that will drive growth and competitiveness in our market. This executive summary presents the critical findings, top strategic imperatives, and actionable recommendations to address the identified gaps and maximize ROI within budget and resource constraints.\n\n**Strategic Context**\nIn a rapidly evolving industry, it is essential for [Company Name] to focus on both short-term wins and long-term transformations. Our analysis reveals that addressing weaknesses in Structure, Systems, Skills, and Shared Values will provide immediate improvements while strengthening our Strategic Direction and Culture for sustained success.\n\n**Key Insights**\n\n1. **Strategic Direction**: A clear strategic vision is present, but there is a need to align all seven Ss to ensure consistent execution and focus on key objectives.\n2. **Structure**: Our organizational structure supports our current strategy, but silos exist that hinder collaboration and information flow.\n3. **Systems**: Inefficiencies in processes, technology, and data management limit productivity and decision-making effectiveness.\n4. **Shared Values**: While [Company Name] has a strong mission and values, they are not consistently embodied across all teams and departments.\n5. **Skills**: The workforce possesses the necessary technical skills but requires development in leadership, innovation, and adaptability to thrive in our competitive environment.\n6. **Staff**: A dedicated and engaged workforce is evident, but there is room for improvement in diversity and inclusion initiatives.\n7. **Style**: [Company Name]'s leadership style fosters collaboration and innovation, but a more consistent approach is needed to ensure alignment with strategic objectives.\n\n**Top Strategic Imperatives and Investment Areas**\n\n1. Streamline organizational structure to eliminate silos and improve cross-functional collaboration.\n2. Implement process improvements and invest in technology to increase productivity and decision-making effectiveness.\n3. Develop leadership, innovation, and adaptability skills across the workforce.\n4. Strengthen diversity and inclusion initiatives to foster a more inclusive culture.\n5. Align shared values with strategic objectives and ensure they are consistently embodied throughout the organization.\n\n**ROI Rationale, Risk Profile, and Next Steps**\nBy addressing these strategic imperatives, [Company Name] can expect immediate improvements in productivity, decision-making effectiveness, and cross-functional collaboration. Long-term benefits include a more adaptable workforce, a stronger culture aligned with strategic objectives, and increased competitiveness in our market.\n\nKey risks include resistance to change, insufficient resources for implementation, and potential disruptions during the transition period. To mitigate these risks, we recommend a phased approach to implementation, clear communication of the benefits and rationale behind each initiative, and dedicated change management support throughout the organization.\n\n**Investment and Resource Requirements Summary**\nThe estimated cost for implementing the recommended initiatives is [insert amount]. This includes both direct costs (e.g., technology investments, training programs) and indirect costs (e.g., change management resources, project management overhead).\n\n**Action Plans**\n\n1. **90-Day Action Plan**: Focus on organizational structure improvements, process streamlining, and initial diversity and inclusion initiatives.\n2. **6-Month Action Plan**: Implement technology investments, launch leadership development programs, and continue diversity and inclusion efforts.\n3. **1-Year Action Plan**: Evaluate the impact of initiatives, refine processes as needed, and continue to develop leadership and adaptability skills across the workforce.\n\n**Success Metrics and Governance Model**\nKey success metrics include improvements in productivity, cross-functional collaboration, employee engagement, and diversity representation. A governance model consisting of a steering committee, project teams, and change champions will oversee the implementation and monitor progress against these metrics.\n\n**Key Risks and Mitigation Strategies**\nPrimary risks include resistance to change, insufficient resources for implementation, and potential disruptions during the transition period. To mitigate these risks, we recommend a phased approach to implementation, clear communication of the benefits and rationale behind each initiative, and dedicated change management support throughout the organization.",
  "llama3": "**Executive Summary**\n\nAs Senior Partner Consultant, I present the findings of our comprehensive 7S assessment, highlighting critical insights and strategic imperatives to inform decision-making.\n\n**Strategic Imperatives:**\n\n1. **Align Culture with Ambition**: Leverage strengths in innovation and collaboration to drive growth.\n2. **Optimize Structure for Agility**: Streamline processes, flatten hierarchy, and empower teams.\n3. **Develop Talent for Digital Transformation**: Invest in upskilling and reskilling programs for a competitive edge.\n\n**Short-Term Wins (0-6 months):**\n\n1. Conduct a thorough review of organizational design to identify opportunities for streamlining.\n2. Develop a comprehensive change management strategy to ensure effective communication and engagement.\n3. Launch a pilot program for digital transformation, focusing on process automation and talent development.\n\n**Long-Term Transformations (6-18 months):**\n\n1. Implement a new performance management system emphasizing agility, innovation, and collaboration.\n2. Establish a center of excellence for digital transformation, driving innovation and knowledge sharing.\n3. Develop a comprehensive succession planning program to ensure continuity and leadership pipeline.\n\n**ROI Rationale:**\n\nOur assessment indicates that investing in cultural alignment, structural optimization, and talent development will yield significant returns:\n\n* 15% increase in productivity through process automation\n* 20% reduction in costs through streamlining and cost optimization\n* 25% improvement in employee engagement and retention\n\n**Risk Profile:**\n\nKey risks include:\n\n1. Inadequate change management leading to resistance and stagnation.\n2. Insufficient investment in digital transformation, hindering competitiveness.\n3. Failure to develop a robust succession planning program, compromising leadership continuity.\n\n**Next Steps:**\n\n1. Conduct a thorough review of organizational design to identify opportunities for streamlining.\n2. Develop a comprehensive change management strategy to ensure effective communication and engagement.\n3. Launch a pilot program for digital transformation, focusing on process automation and talent development.\n\nBy prioritizing these strategic imperatives, we can drive growth, optimize performance, and position the organization for long-term success."
}
```
</details>

## MEGA_PROMPT

**TechNova Solutions – 7S Organizational Assessment Report**  
*Prepared for the Board of Directors*  
*Prepared by: World‑Renowned Management Consultant*  
*Date: 25 Oct 2025*  

---

## Executive Summary  

| Dimension | Current State | Gap | Impact | Recommendation | 3‑yr ROI |
|-----------|---------------|-----|--------|----------------|----------|
| **Strategy** | “Digital‑First” intent, but 30 % of revenue from legacy services | Unclear segmentation & value proposition | Brand dilution, lost growth | Redefine strategy to 3 core verticals; launch “Digital‑First” roadmap | +12 % CAGR |
| **Structure** | Functional matrix, high inter‑departmental friction | 25 % of projects delayed | Inefficiency, cost drift | Shift to product‑centric, decentralized org | –$3 M operating cost |
| **Systems** | Legacy ERP + siloed data lakes | 40 % data latency | Poor decision‑making | Deploy unified lakehouse & analytics | +8 % productivity |
| **Shared Values** | “Customer‑first” espoused, low engagement | 15 % turnover in high‑skill roles | Talent drain | Kotter‑based culture‑change program | +10 % retention |
| **Style** | Top‑down, reactive leadership | 20 % initiatives stalled | Slow response | Adopt coaching & empowerment style | +5 % NPS |
| **Staff** | 2000 employees, 70 % technical | 25 % skill‑gap in AI/ML & digital sales | Lost deals | Upskill & reskill plan | +15 % innovation output |
| **Skills** | Strong engineering, weak commercial | 30 % sales lost to AI‑enabled competitors | Revenue erosion | Cross‑functional training & hiring | +10 % win rate |

**Key Take‑aways**

1. **Strategic–structural misalignment** – the matrix design hampers the “Digital‑First” agenda.  
2. **Systems lag** – data silos prevent real‑time insights, undermining strategy and shared values.  
3. **Cultural inertia** – leadership style and low engagement hinder execution.  

**Top‑Priority Actions (Phase 1)**  

| Action | Owner | Timeline | KPI |
|--------|-------|----------|-----|
| Redefine strategy & product portfolio | CEO & Strategy Office | 3 mo | 3‑yr revenue target |
| Restructure to product‑centric org | COO | 6 mo | Project delay % |
| Deploy unified data platform | CTO | 9 mo | Data latency |
| Launch Kotter‑based culture program | CHRO | 12 mo | Engagement score |

These actions are projected to generate **+12 % CAGR** and **$25 M incremental EBITDA** over three years, with a payback period of **18 months**.

---

## 1. Detailed 7S Analysis  

### 1.1 Strategy  
- **Current**: “Digital‑First” focus on cloud, AI, IoT; no clear segmentation.  
- **Gap**: 30 % revenue from legacy consulting.  
- **Impact**: Brand dilution, resource misallocation.  
- **Evidence**: IDC 2024 – digital‑first firms outperform legacy peers by 8 % CAGR.  

### 1.2 Structure  
- **Current**: Functional matrix; ambiguous decision rights.  
- **Gap**: 25 % of cross‑functional projects delayed.  
- **Impact**: Cost drift, slower time‑to‑market.  
- **Evidence**: McKinsey – product‑centric structures deliver 30 % faster innovation.  

### 1.3 Systems  
- **Current**: SAP ECC + siloed data lakes.  
- **Gap**: 40 % of queries >30 min.  
- **Impact**: Poor forecasting, missed upsell.  
- **Evidence**: Gartner 2023 – unified analytics cut decision latency by 70 %.  

### 1.4 Shared Values  
- **Current**: “Customer‑first” stated, but engagement fell from 78 % to 63 %.  
- **Gap**: 15 % turnover in high‑skill roles.  
- **Impact**: Knowledge loss, higher hiring cost.  
- **Evidence**: Deloitte 2024 – engagement drives 21 % higher productivity.  

### 1.5 Style  
- **Current**: Top‑down, reactive; multi‑sign‑off approvals.  
- **Gap**: 20 % initiatives stall at approval.  
- **Impact**: Missed windows, low morale.  
- **Evidence**: Kotter – empowering leaders accelerates change.  

### 1.6 Staff  
- **Current**: 2000 employees; 70 % technical, 30 % commercial.  
- **Gap**: 25 % lack AI/ML training; 30 % lack digital sales skills.  
- **Impact**: Lost deals, inability to deliver end‑to‑end solutions.  
- **Evidence**: PwC 2023 – cross‑skill teams close 15 % more deals.  

### 1.7 Skills  
- **Current**: Strong engineering, weak commercial/analytics.  
- **Gap**: 30 % sales lost to AI‑enabled competitors.  
- **Impact**: Revenue erosion.  
- **Evidence**: Forrester 2024 – AI‑enabled sales teams win 12 % more deals.  

---

## 2. Interdependencies & Trade‑offs  

| Interdependency | Reinforcing | Conflicting | Mitigation |
|-----------------|-------------|-------------|------------|
| **Strategy ↔ Structure** | Product focus reinforces product‑centric org | Matrix dilutes focus | Shift to product‑centric |
| **Systems ↔ Shared Values** | Unified data supports “customer‑first” | Silos undermine values | Deploy lakehouse |
| **Style ↔ Staff** | Empowered leaders boost engagement | Top‑down stifles initiative | Leadership coaching |
| **Skills ↔ Strategy** | Digital strategy needs AI/ML | Skill gaps hinder execution | Upskill & hire |
| **Structure ↔ Systems** | Product org aligns with data pipelines | Legacy ERP hampers flow | Modernize ERP |

**Trade‑off**: Centralized control vs. decentralized agility.  
- *Recommendation*: Adopt a **decentralized, product‑centric** structure that retains central governance for standards but empowers product teams with autonomy and clear ownership.

---

## 3. Strategic Recommendations  

| Priority | Recommendation | Rationale | ROI Driver | Lead |
|----------|----------------|-----------|------------|------|
| **1** | Redefine strategy & product portfolio | Align revenue mix with digital focus | Higher margin digital services | CEO & Strategy Office |
| **2** | Restructure to product‑centric, decentralized org | Reduce matrix friction | Faster time‑to‑market | COO |
| **3** | Deploy unified lakehouse & analytics | Real‑time insights, break silos | Data‑driven decisions | CTO |
| **4** | Launch Kotter‑based culture‑change program | Address engagement, empowerment | Higher retention | CHRO |
| **5** | Upskill staff in AI/ML & digital sales | Close skill gaps, enable end‑to‑end delivery | Increased win rates | L&D |
| **6** | Implement agile governance & OKR framework | Faster approvals, alignment | Reduced project delays | PMO |
| **7** | Revise talent acquisition to emphasize soft skills | Build holistic teams | Better fit, lower turnover | Talent Acquisition |

**ROI Justification**  

| Driver | Incremental EBITDA | Payback |
|--------|-------------------|---------|
| Digital services margin +4 % | $8 M | 12 mo |
| Project delay savings 15 % | $3 M | 9 mo |
| Retention lift 10 % | $2 M | 12 mo |
| Upskilling win‑rate +15 % | $5 M | 15 mo |
| **Total** | **$25 M** | **≈18 mo** |

---

## 4. Implementation Plan  

| Phase | Duration | Key Actions | Milestones | Owner |
|-------|----------|-------------|------------|-------|
| **Phase 1 – Foundation (0‑6 mo)** | 6 mo | • Strategy workshop<br>• Org design blueprint<br>• Data platform assessment | • Approved strategy<br>• Org chart<br>• Vendor selection | CEO, COO, CTO |
| **Phase 2 – Execution (6‑18 mo)** | 12 mo | • Roll‑out product org<br>• Deploy lakehouse<br>• Launch culture program | • Product teams operational<br>• 80 % data integration<br>• Engagement ↑10 pts | COO, CTO, CHRO |
| **Phase 3 – Optimization (18‑36 mo)** | 18 mo | • Upskill programs<br>• Agile governance rollout<br>• OKR implementation | • 90 % staff trained<br>• Agile ceremonies in place<br>• OKRs aligned | L&D, PMO, HR |
| **Phase 4 – Sustain (36‑48 mo)** | 12 mo | • Continuous improvement loop<br>• Governance reviews<br>• Scale best practices | • 3‑yr KPI targets met | All |

**Governance & Accountability**  

- **Steering Committee** (CEO, COO, CTO, CHRO) – monthly review.  
- **Change Advisory Board** – quarterly progress, risk mitigation.  
- **KPIs** – Strategy, Structure, Systems, Shared Values, Style, Staff, Skills (see Table 5).  

---

## 5. Governance & Metrics  

| Dimension | Metric | Target | Frequency |
|-----------|--------|--------|-----------|
| **Strategy** | % revenue from digital services | 60 % | Quarterly |
| **Structure** | Project delay % | <10 % | Monthly |
| **Systems** | Avg. query latency | <5 min | Monthly |
| **Shared Values** | Employee engagement | 80 % | Semi‑annual |
| **Style** | Leadership empowerment index | 75 % | Quarterly |
| **Staff** | High‑skill turnover | <5 % | Annual |
| **Skills** | Digital win rate | +15 % vs baseline | Quarterly |

**Dashboard** – Real‑time Power BI portal accessible to all stakeholders.

---

## 6. Appendices  

### Appendix A – Assumptions  
| Assumption | Rationale |
|------------|-----------|
| Revenue 2024: $500 M | Mid‑size tech services firm |
| Workforce: 2000 employees | Industry norm |
| Digital services margin: 25 % | Benchmark |
| Turnover cost: $120 k/employee | Average hiring & training cost |

### Appendix B – Data Sources (Industry Benchmarks)  
- IDC Digital Transformation Report 2024  
- McKinsey 7S Case Studies 2023  
- Gartner Unified Analytics 2023  
- Deloitte Workforce Engagement 2024  
- Forrester AI‑Enabled Sales 2024  

### Appendix C – Quick Reference Tables  
- **Skill Gap Matrix** – Technical vs Commercial  
- **Org Chart Draft** – Product‑centric vs Matrix  

---

**Prepared by:**  
*World‑Renowned Management Consultant*  
*Lead Author – 7S Assessment*  

---

<details><summary>Model answers</summary>

```json
{
  "gpt-oss": "# 7S Organizational Assessment Report  \n**Prepared for:** Board of Directors – *TechNova Solutions*  \n**Prepared by:** World‑Renowned Management Consultant (WRC)  \n**Date:** 25 Oct 2025  \n\n---\n\n## Executive Summary  \n\n| Theme | Current State | Gap | Impact | Recommendation | ROI (3‑yr) |\n|-------|---------------|-----|--------|----------------|------------|\n| **Strategy** | Growth via “Digital‑First” portfolio, but unclear market‑segmentation | 30 % of revenue from legacy services | Missed opportunities, diluted brand | Refine strategy to 3 core verticals; launch “Digital‑First” roadmap | +12 % CAGR |\n| **Structure** | Functional matrix, high inter‑departmental friction | 25 % of projects delayed | Inefficiency, cost drift | Adopt “Product‑Centric” org; clear product‑owner lines | -$3 M operating cost |\n| **Systems** | Legacy ERP, siloed data lakes | 40 % data latency | Poor decision‑making | Implement unified data platform & analytics | +8 % productivity |\n| **Shared Values** | “Customer‑first” espoused, but low employee engagement | 15 % turnover in high‑skill roles | Talent drain | Launch culture‑change program (Kotter) | +10 % retention |\n| **Style** | Top‑down, reactive leadership | 20 % of initiatives stalled | Slow response to market | Adopt coaching & empowerment style | +5 % NPS |\n| **Staff** | 2000 employees, 70 % technical | 25 % skill‑gap in AI/ML | Lagging innovation | Upskill & reskill plan | +15 % innovation output |\n| **Skills** | Strong engineering, weak commercial | 30 % of sales lost to competitors | Revenue erosion | Cross‑functional training & hiring | +10 % win rate |\n\n**Key Take‑aways**\n\n1. **Misalignment between strategy and structure** – the matrix design hampers the “Digital‑First” strategy.  \n2. **Systems lag** – data silos prevent real‑time insights, undermining both strategy and shared values.  \n3. **Cultural inertia** – leadership style and low engagement hinder execution.  \n\n**Top‑Priority Actions (Phase 1)**  \n\n| Action | Owner | Timeline | KPI |\n|--------|-------|----------|-----|\n| Redefine strategy & product portfolio | CEO & Strategy Office | 3 mo | 3‑yr revenue target |\n| Restructure to product‑centric org | COO | 6 mo | Project delay % |\n| Deploy unified data platform | CTO | 9 mo | Data latency |\n| Launch Kotter‑based culture program | CHRO | 12 mo | Engagement score |\n\nThese actions are expected to generate a **+12 % CAGR** and **$25 M incremental EBITDA** over the next three years, with a payback period of **18 months**.\n\n---\n\n## 1. Detailed 7S Analysis  \n\n### 1.1 Strategy  \n- **Current State:** TechNova’s strategic intent is “Digital‑First” – expanding cloud, AI, and IoT services. However, the strategy remains high‑level, with no clear segmentation or differentiated value proposition.  \n- **Gap Analysis:** 30 % of revenue still derives from legacy on‑premise consulting, which conflicts with the digital narrative.  \n- **Impact:** Brand dilution, inability to attract digital‑savvy clients, and internal confusion over resource allocation.  \n- **Evidence:** Market research (IDC, 2024) shows digital‑first firms outperform legacy‑based peers by 8 % CAGR.  \n\n### 1.2 Structure  \n- **Current State:** A functional matrix with separate R&D, Sales, and Operations functions. Decision rights are shared but often ambiguous.  \n- **Gap Analysis:** 25 % of cross‑functional projects experience scope creep and delays.  \n- **Impact:** Increased cost of capital, delayed time‑to‑market, and reduced agility.  \n- **Evidence:** McKinsey’s 7S studies show that product‑centric structures deliver 30 % faster innovation cycles.  \n\n### 1.3 Systems  \n- **Current State:** ERP (SAP ECC) integrated with legacy CRM; data lakes are siloed per business unit.  \n- **Gap Analysis:** 40 % of data queries take >30 min, leading to delayed insights.  \n- **Impact:** Poor forecasting, sub‑optimal pricing, and missed upsell opportunities.  \n- **Evidence:** Gartner (2023) reports that unified analytics platforms cut decision latency by 70 %.  \n\n### 1.4 Shared Values  \n- **Current State:** “Customer‑first” is a stated value, but employee engagement scores have fallen from 78 % to 63 % over the past 18 months.  \n- **Gap Analysis:** 15 % turnover among high‑skill roles, especially in data science.  \n- **Impact:** Loss of institutional knowledge, higher hiring costs, and reduced innovation.  \n- **Evidence:** Deloitte’s 2024 Workforce Study links engagement to 21 % higher productivity.  \n\n### 1.5 Style  \n- **Current State:** Leadership operates in a top‑down, reactive mode. Decision approvals often require multiple sign‑offs.  \n- **Gap Analysis:** 20 % of initiatives stall at the “approval” stage.  \n- **Impact:** Missed market windows, low morale, and stifled experimentation.  \n- **Evidence:** Kotter’s Change Model emphasizes empowering leaders at all levels to accelerate change.  \n\n### 1.6 Staff  \n- **Current State:** 2000 employees, 70 % technical, 30 % commercial.  \n- **Gap Analysis:** 25 % of technical staff lack formal AI/ML training; 30 % of commercial staff lack digital sales skills.  \n- **Impact:** Inability to deliver end‑to‑end digital solutions, leading to lost deals.  \n- **Evidence:** PwC (2023) indicates that firms with cross‑skill teams see 15 % higher deal closure rates.  \n\n### 1.7 Skills  \n- **Current State:** Strong engineering base; weak commercial and data analytics capabilities.  \n- **Gap Analysis:** 30 % of sales lost to competitors with better AI‑enabled solutions.  \n- **Impact:** Revenue erosion and lower market share.  \n- **Evidence:** Forrester (2024) reports that AI‑enabled sales teams outperform peers by 12 % in win rates.  \n\n---\n\n## 2. Interdependencies Overview  \n\n| Interdependency | Reinforcing | Conflicting | Mitigation |\n|-----------------|-------------|-------------|------------|\n| **Strategy ↔ Structure** | Clear product lines reinforce focused org design | Matrix structure dilutes product focus | Shift to product‑centric org |\n| **Systems ↔ Shared Values** | Unified data platform supports “customer‑first” insights | Siloed data undermines shared value | Deploy data lakehouse |\n| **Style ↔ Staff** | Empowered leaders boost engagement | Top‑down style stifles initiative | Leadership coaching |\n| **Skills ↔ Strategy** | Digital strategy requires AI/ML skills | Skill gaps hinder strategy execution | Upskill & hire |\n| **Structure ↔ Systems** | Product org aligns with cross‑functional data pipelines | Legacy ERP hampers product data flow | Modernize ERP |\n\n---\n\n## 3. Strategic Recommendations  \n\n| Priority | Recommendation | Rationale | ROI Driver | Implementation Lead |\n|----------|----------------|-----------|------------|---------------------|\n| **1** | **Redefine Strategy & Product Portfolio** | Align revenue mix with digital focus; clear value propositions | Higher margin digital services | CEO & Strategy Office |\n| **2** | **Restructure to Product‑Centric Org** | Reduce matrix friction; clear product ownership | Faster time‑to‑market | COO |\n| **3** | **Deploy Unified Data Platform (Lakehouse)** | Real‑time analytics; break silos | Data‑driven decisions | CTO |\n| **4** | **Launch Kotter‑Based Culture Change Program** | Address engagement, empowerment | Higher retention | CHRO |\n| **5** | **Upskill Staff in AI/ML & Digital Sales** | Close skill gaps; enable end‑to‑end delivery | Increased win rates | Learning & Development |\n| **6** | **Implement Agile Governance Framework** | Faster approvals, iterative delivery | Reduced project delays | PMO |\n| **7** | **Introduce OKR‑Based Performance Management** | Align individual goals with strategy | Higher accountability | HR |\n\n**ROI Justification**  \n- **Digital Services Margin:** +4 % (industry benchmark) → $8 M incremental EBITDA.  \n- **Reduced Project Delays:** 15 % cost savings → $3 M.  \n- **Higher Retention:** 10 % reduction in turnover → $2 M savings.  \n- **Upskilling ROI:** 15 % increase in deal closure → $5 M.  \n- **Total 3‑yr Incremental EBITDA:** ~$25 M; Payback ≈ 18 months.\n\n---\n\n## 4. Implementation Plan  \n\n| Phase | Duration | Key Actions | Milestones | Owner |\n|-------|----------|-------------|------------|-------|\n| **Phase 1 – Foundation (0‑6 mo)** | 6 mo | • Strategy workshop<br>• Org design blueprint<br>• Data platform assessment | • Approved strategy<br>• Org chart<br>• Data platform vendor selection | CEO, COO, CTO |\n| **Phase 2 – Execution (6‑18 mo)** | 12 mo | • Roll‑out product org<br>• Deploy data lakehouse<br>• Launch culture program | • Product teams operational<br>• 80 % data integration<br>• Engagement score ↑10 pts | COO, CTO, CHRO |\n| **Phase 3 – Optimization (18‑36 mo)** | 18 mo | • Upskill programs<br>• Agile governance rollout<br>• OKR implementation | • 90 % staff trained<br>• Agile ceremonies in place<br>• OKRs aligned | L&D, PMO, HR |\n| **Phase 4 – Sustain (36‑48 mo)** | 12 mo | • Continuous improvement loop<br>• Governance reviews<br>• Scale best practices | • 3‑yr KPI targets met | All |\n\n**Governance & Accountability**  \n- **Steering Committee** (CEO, COO, CTO, CHRO) – monthly review.  \n- **Change Advisory Board** – quarterly progress, risk mitigation.  \n- **KPIs** – Strategy: revenue mix; Structure: project delay %; Systems: data latency; Shared Values: engagement; Style: NPS; Staff: turnover; Skills: win rate.\n\n---\n\n## 5. Governance & Metrics  \n\n| Dimension | Metric | Target | Frequency |\n|-----------|--------|--------|-----------|\n| **Strategy** | % of revenue from digital services | 60 % | Quarterly |\n| **Structure** | Project delay % | <10 % | Monthly |\n| **Systems** | Data latency (avg. query time) | <5 min | Monthly |\n| **Shared Values** | Employee engagement score | 80 % | Semi‑annual |\n| **Style** | Leadership empowerment index | 75 % | Quarterly |\n| **Staff** | Turnover (high‑skill) | <5 % | Annual |\n| **Skills** | Deal win rate for digital solutions | 15 % higher than baseline | Quarterly |\n\n**Dashboard** – Real‑time visualization via Power BI, accessible to all stakeholders.\n\n---\n\n## 6. Appendices  \n\n### Appendix A – Assumptions  \n| Assumption | Rationale |\n|------------|-----------|\n| Revenue 2024: $500 M | Mid‑size tech services firm |\n| Workforce: 2000 employees | Typical for industry |\n| Digital services margin: 25 % | Industry average |\n| Turnover cost: $120 k per employee | Average hiring & training cost |\n\n### Appendix B – Data Sources (Industry Benchmarks)  \n- IDC Digital Transformation Report 2024  \n- McKinsey 7S Case Studies 2023  \n- Gartner Unified Analytics 2023  \n- Deloitte Workforce Engagement 2024  \n- Forrester AI‑Enabled Sales 2024  \n\n### Appendix C – Quick Reference Tables  \n- **Skill Gap Matrix** – Technical vs Commercial  \n- **Org Chart Draft** – Product‑centric vs Matrix  \n\n---\n\n**Prepared by:**  \n*World‑Renowned Management Consultant*  \n*Lead Author – 7S Assessment*  \n\n---",
  "mistral": "# XYZ Corporation 7S Organizational Assessment Report\n\n## Executive Summary\n\nThis report presents an in-depth analysis of XYZ Corporation's organizational health using the McKinsey 7S framework. The assessment reveals misalignments across several S elements that have contributed to strategic challenges and recent initiatives' underperformance. This report proposes targeted interventions with justifiable returns on investment (ROI) to optimize XYZ Corporation's performance, positioning it for sustained growth and success.\n\n## Detailed 7S Analysis\n\n### Strategy (S1)\nThe current strategy focuses on market expansion, yet the organization lacks a clear competitive advantage and differentiation in its target markets. The strategy should be revised to emphasize unique selling points and identify opportunities for competitive advantage.\n\n### Structure (S2)\nXYZ Corporation's structure is centralized, which may hinder agility and responsiveness to market changes. A more decentralized approach with empowered business units could improve adaptability and decision-making speed.\n\n### Systems (S3)\nInadequate IT systems and processes create bottlenecks in data management, hindering informed decision-making. Upgrading the technology infrastructure and streamlining processes are essential to support strategic objectives.\n\n### Shared Values (S4)\nAlthough XYZ Corporation has a well-defined mission and vision, there is a lack of alignment between these values and employee behavior. A comprehensive cultural transformation program should be initiated to reinforce shared values and foster a high-performance culture.\n\n### Style (S5)\nA command-and-control leadership style stifles innovation and creativity. Adopting a more collaborative and empowering leadership approach will encourage employee engagement, boost morale, and stimulate innovative thinking.\n\n### Staff (S6)\nThe workforce lacks the necessary skills to execute the current strategy effectively. Investments in training and development programs are essential to close the skill gap and ensure that employees have the competencies required for success.\n\n### Skills (S7)\nXYZ Corporation's recruitment practices focus on technical expertise, neglecting soft skills. A more holistic approach to talent acquisition should be implemented to emphasize both hard and soft skills.\n\n## Interdependencies Overview\nMisalignments in the S elements create a reinforcing cycle of underperformance, as strategic ambiguity (S1) leads to unclear roles and responsibilities (S2), which in turn hinders effective use of systems (S3). The resulting lack of shared values (S4) and an inappropriate leadership style (S5) further discourages employee engagement and competency development (S6 & S7).\n\n## Strategic Recommendations\n1. Revise the strategy to emphasize unique selling points and competitive advantage.\n2. Implement a more decentralized organizational structure to improve adaptability.\n3. Upgrade technology infrastructure and streamline processes for informed decision-making.\n4. Launch a comprehensive cultural transformation program to reinforce shared values.\n5. Adopt a collaborative and empowering leadership approach to stimulate innovation.\n6. Invest in training and development programs to close the skill gap.\n7. Emphasize both hard and soft skills during talent acquisition.\n\n## Implementation Plan\n1. Strategy Revision: Q2 2023 - Q4 2023\n2. Organizational Structure Change: Q3 2023 - Q1 2024\n3. Technology Infrastructure Upgrade & Process Improvement: Q1 2024 - Q3 2024\n4. Cultural Transformation Program: Q2 2024 - Q4 2025\n5. Leadership Style Change: Ongoing, with targeted workshops and coaching sessions\n6. Training & Development Programs: Ongoing, integrated into performance management system\n7. Talent Acquisition Strategy Overhaul: Q1 2024 - Q2 2024\n\n## Governance & Metrics\nEstablish a cross-functional governance committee to oversee the implementation of strategic recommendations and monitor progress using key performance indicators (KPIs) aligned with the Balanced Scorecard framework.\n\n## Appendices\n1. Detailed S-element assessments with supporting data\n2. KPI definitions and target setting for each recommendation\n3. Timeline and milestones for each phase of the implementation plan",
  "llama3": "**Executive Summary**\n\nOur comprehensive 7S organizational assessment reveals that XYZ Inc., a mid-sized company with $500 million in revenue and a strong market position, faces strategic challenges that impact its overall performance. Our analysis highlights misalignments across all seven elements: Strategy, Structure, Systems, Shared Values, Style, Staff, and Skills.\n\nKey findings include:\n\n* A mismatch between the company's current strategy and its market position, leading to stagnation in growth.\n* An outdated organizational structure that hinders collaboration and innovation.\n* Inefficient systems and processes, causing delays and increased costs.\n* A lack of shared values and a culture that prioritizes individual goals over collective success.\n* A leadership style that is autocratic and resistant to change.\n* Underutilized staff with limited opportunities for growth and development.\n* Skills gaps in key areas, such as digital transformation and data analytics.\n\nTo address these challenges, we propose targeted interventions with ROI justification. Our recommendations prioritize the development of a new strategy, restructuring of the organization, implementation of efficient systems, and cultivation of shared values and a collaborative culture. We also suggest changes to leadership style and staff development programs to ensure the company's future success.\n\n**Detailed 7S Analysis**\n\n### Strategy\n\nOur analysis reveals that XYZ Inc.'s current strategy is focused on short-term gains, rather than long-term growth and sustainability. The company's market position is strong, but its products and services are not innovative enough to drive significant revenue increases. To address this challenge, we recommend developing a new strategy that prioritizes innovation, digital transformation, and customer-centricity.\n\n### Structure\n\nThe company's organizational structure is outdated and hinders collaboration and innovation. We recommend restructuring the organization into smaller, agile units that can respond quickly to changing market conditions. This will require changes to roles and responsibilities, as well as the development of new processes and systems.\n\n### Systems\n\nOur analysis reveals that XYZ Inc.'s systems and processes are inefficient and cause delays and increased costs. We recommend implementing new systems and processes that prioritize automation, standardization, and continuous improvement. This will require investments in technology and training for employees.\n\n### Shared Values\n\nThe company's culture is not aligned with its stated values of innovation, collaboration, and customer-centricity. We recommend developing a shared values framework that prioritizes these values and provides a clear direction for the organization. This will require changes to leadership style and employee engagement programs.\n\n### Style\n\nOur analysis reveals that XYZ Inc.'s leadership style is autocratic and resistant to change. We recommend developing a more collaborative leadership style that encourages open communication, innovation, and continuous improvement. This will require changes to leadership development programs and employee engagement initiatives.\n\n### Staff\n\nThe company's staff is underutilized and lacks opportunities for growth and development. We recommend developing new roles and responsibilities that provide employees with the autonomy to make decisions and take calculated risks. This will require investments in training and development programs, as well as changes to performance management systems.\n\n### Skills\n\nOur analysis reveals that XYZ Inc. has skills gaps in key areas, such as digital transformation and data analytics. We recommend developing new skills and competencies through training and development programs, as well as partnerships with external organizations and vendors.\n\n**Interdependencies Overview**\n\nOur analysis reveals that the seven S elements are interconnected and interdependent. For example, a new strategy requires changes to structure, systems, shared values, style, staff, and skills. Similarly, changes to leadership style require changes to employee engagement programs and performance management systems. We recommend developing a comprehensive plan that addresses these interdependencies and prioritizes the development of new strategies, structures, systems, and cultures.\n\n**Strategic Recommendations**\n\nOur strategic recommendations prioritize the development of a new strategy, restructuring of the organization, implementation of efficient systems, and cultivation of shared values and a collaborative culture. We also suggest changes to leadership style and staff development programs to ensure the company's future success.\n\n1. Develop a new strategy that prioritizes innovation, digital transformation, and customer-centricity.\n2. Restructure the organization into smaller, agile units that can respond quickly to changing market conditions.\n3. Implement new systems and processes that prioritize automation, standardization, and continuous improvement.\n4. Cultivate shared values and a collaborative culture that prioritizes innovation, collaboration, and customer-centricity.\n5. Develop a more collaborative leadership style that encourages open communication, innovation, and continuous improvement.\n6. Provide employees with the autonomy to make decisions and take calculated risks through new roles and responsibilities.\n7. Develop new skills and competencies through training and development programs, as well as partnerships with external organizations and vendors.\n\n**Implementation Plan**\n\nOur implementation plan prioritizes phased actions with responsible functions. We recommend:\n\n1. Developing a new strategy and restructuring the organization (Phase 1).\n2. Implementing new systems and processes (Phase 2).\n3. Cultivating shared values and a collaborative culture (Phase 3).\n4. Developing a more collaborative leadership style and providing employees with autonomy (Phase 4).\n5. Building new skills and competencies through training and development programs (Phase 5).\n\n**Governance & Metrics**\n\nOur governance and metrics plan prioritizes success tracking and monitoring. We recommend:\n\n1. Establishing a steering committee to oversee the implementation of our recommendations.\n2. Developing key performance indicators (KPIs) that track progress towards our strategic objectives.\n3. Conducting regular reviews and assessments to ensure the company is on track to achieve its goals.\n\n**Appendices**\n\nOur appendices include tables, charts, and data references that support our analysis and recommendations. We have included:\n\n1. A table summarizing our findings across all seven S elements.\n2. A chart illustrating the interdependencies between the seven S elements.\n3. Data references supporting our analysis and recommendations."
}
```
</details>

